In [ ]:
# Library list🤖
import glob, logging, warnings, polars as pl, datetime, os, zipfile, xml.dom.minidom
from datetime import datetime as dt, time as t, timedelta
import pandas as pd, numpy as np, sqlalchemy as sa, xlsxwriter
from sqlalchemy import create_engine, text
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from polars.exceptions import ColumnNotFoundError, PanicException
from pathlib import Path
from IPython.display import HTML
from tabulate import tabulate
# -----------------------------------------------------------------------------------------------#
# --- Logging configuration📜 ---
log_directory = Path(os.environ['USERPROFILE']) / r'Concentrix Corporation//CNXVN - WFM Team - Documents//DataBase//DataFrame//BKN//ScriptLogs//'
log_directory.mkdir(parents=True, exist_ok=True) 
log_filename = log_directory / f"import_log_{dt.now():%Y%m%d_%H%M%S}.log"
logging.basicConfig(
    level=logging.INFO,  # (DEBUG, INFO, WARNING, ERROR, CRITICAL)
    format='%(asctime)s - %(levelname)s - %(name)s - %(message)s',
    handlers=[
        logging.FileHandler(log_filename, encoding='utf-8'), 
    ],force=True)
# Create logger object
logger = logging.getLogger('ServerImportScript')
# -----------------------------------------------------------------------------------------------#
# Source collection📥
user_credential = Path(os.environ['USERPROFILE']) / r'Concentrix Corporation//CNXVN - WFM Team - Documents//'

# 0️⃣1️⃣[BKN]AHT2🗃️
AHT2_TABLE_NAME = "BCOM.AHT2"
Folder_AHT2 = user_credential / r'DataBase//DataRaw//BKN//AHT2//'
log_AHT2_path = user_credential / r'DataBase//DataRaw//BKN//MODIFIED_LOG//AHT2_log.xlsx'
AHT2_schema = ['FileName', 'ModifiedDate', 'Date', 'Agent Name Display', 'Answered Language Name', 'Measure Names', 'Measure Values']
# 0️⃣2️⃣[BKN]ROSTER🗃️
ROSTER_TABLE_NAME = "BCOM.ROSTER"
Folder_ROSTER = user_credential / r'DataBase//DataRaw//BKN//ROSTER//'
log_ROSTER_path = user_credential / r'DataBase//DataRaw//BKN//MODIFIED_LOG//ROSTER_log.xlsx'
ROSTER_schema = ['FileName', 'ModifiedDate', 'Emp ID', 'Name', 'Attribute', 'Value', 'LOB', 
                 'team_leader', 'week_shift', 'week_off', 'OM', 'DPE', 'Work Type', 'QA']
# 0️⃣3️⃣[BKN]EPS🗃️
EPS_TABLE_NAME = "BCOM.EPS"
Folder_EPS = user_credential / r'DataBase//DataRaw//BKN//EPS//'
log_EPS_path = user_credential / r'DataBase//DataRaw//BKN//MODIFIED_LOG//EPS_log.xlsx'
EPS_schema = ['FileName', 'ModifiedDate', 'sitecode', 'manager_username', 'Username', 'Date', 'Session Login', 
              'Session Logout', 'Session Time', 'BPE Code', 'Total Time', 'SessionLogin_VN', 'SessionLogout_VN',
              'NightTime', 'DayTime', 'Night_BPE', 'Day_BPE']
# 0️⃣4️⃣[BKN]CPI🗃️
CPI_TABLE_NAME = "BCOM.CPI"
Folder_CPI = user_credential / r'DataBase//DataRaw//BKN//CPI//'
log_CPI_path = user_credential / r'DataBase//DataRaw//BKN//MODIFIED_LOG//CPI_log.xlsx'
CPI_schema = ['FileName', 'ModifiedDate', 'Date', 'Staff Name', 'Hour Interval Selected', 'Channel', 
              'Item Label', 'Item ID', "'Item ID'", 'Time Alert', 'Nr. Contacts', 'Item Link', 'Time']
# 0️⃣5️⃣[GLB]RAMCO🗃️
RAMCO_TABLE_NAME = "GLB.RAMCO"
Folder_RAMCO = user_credential / r'DataBase//DataRaw//GLOBAL//RAMCO//'
log_RAMCO_path = user_credential / r'DataBase//DataRaw//BKN//MODIFIED_LOG//RAMCO_log.xlsx'
RAMCO_schema = ['FileName', 'ModifiedDate', 'EID', 'Employee_Name', 'Employee_type', 'Date', 'Code']
# 0️⃣6️⃣[GLB]OT_RAMCO🗃️
OT_RAMCO_TABLE_NAME = "GLB.OT_RAMCO"
Folder_OT_RAMCO = user_credential / r'DataBase//DataRaw//GLOBAL//OT_RAMCO//'
log_OT_RAMCO_path = user_credential / r'DataBase//DataRaw//BKN//MODIFIED_LOG//OT_RAMCO_log.xlsx'
OT_RAMCO_schema = ['FileName', 'ModifiedDate', 'employee_code', 'employee_name', 'Employee Type', 'OT Type', 'Date', 'Status', 'Hours']
# 0️⃣7️⃣[GLB]PremHdays🗃️
PremHdays_TABLE_NAME = "GLB.PremHdays"
Folder_PremHdays = user_credential / r'DataBase//DataRaw//GLOBAL//HOLIDAY_MAPPING//'
log_PremHdays_path = user_credential / r'DataBase//DataRaw//BKN//MODIFIED_LOG//PremHdays_log.xlsx'
PremHdays_schema = ['FileName', 'ModifiedDate', 'Date', 'Holiday']
# 0️⃣8️⃣[GLB]NormHdays🗃️
NormHdays_TABLE_NAME = "GLB.NormHdays"
Folder_NormHdays = user_credential / r'DataBase//DataRaw//GLOBAL//HOLIDAY_MAPPING_NONBILLABLE//'
log_NormHdays_path = user_credential / r'DataBase//DataRaw//BKN//MODIFIED_LOG//NormHdays_log.xlsx'
NormHdays_schema = ['FileName', 'ModifiedDate', 'Solar Day', 'Lunar Day', 'Holiday']
# 0️⃣9️⃣[GLB]EmpMaster🗃️
EmpMaster_TABLE_NAME = "GLB.EmpMaster"
Folder_EmpMaster = user_credential / r'DataBase//DataRaw//GLOBAL//WDD//'
log_EmpMaster_path = user_credential / r'DataBase//DataRaw//BKN//MODIFIED_LOG//EmpMaster_log.xlsx'
EmpMaster_schema = ['FileName', 'ModifiedDate', 'EMPLOYEE_NUMBER', 'PREVIOUS_PAYROLL_ID', 'FIRST_NAME', 'MIDDLE_NAME', 'LAST_NAME', 
                    'FULL_NAME', 'Work Related Status', 'Work Related (Extended Status)', 'Service Type', 'WAH & Hybrid Platform', 	
                    'ORIGINAL_DATE_OF_HIRE', 'LEGAL_EMPLOYER_HIRE_DATE', 'Continuous Service Date', 'Fixed Term Hire End Date', 
                    'Contract End Date', 'PERSON_TYPE', 'WORKER_CATEGORY', 'Time Type', 'Employee Type', 'Last Promotion Date', 
                    'Assignment Category', 'Email - Work', 'BUSINESS_UNIT', 'Job Code', 'Job Title', 'Business Title', 'Cost Center - ID', 
                    'Cost Center - Name', 'LOCATION_CODE', 'LOCATION_NAME', 'CNX BU', 'Concentrix LOB', 'Process', 'COMPANY', 
                    'MANAGEMENT_LEVEL', 'Job Level', 'Compensation Grade', 'JOB_FUNCTION_DESCRIPTION', 'JOB_FAMILY', 'MSA', 'MSA Client', 
                    'MSA Program', 'ACTIVITY ID', 'SUPERVISOR_ID', 'SUPERVISOR_FULL_NAME', 'SUPERVISOR_EMAIL_ID', 'MANAGER_02_ID', 
                    'MANAGER_02_FULL_NAME', 'MANAGER_02_EMAIL_ID', 'COMP_CODE', 'CITY', 'Location', 'Country', 'Employee Status', 'Work Shift']
# 1️⃣0️⃣[GLB]Termination🗃️
Termination_TABLE_NAME = "GLB.Termination"
Folder_Termination = user_credential / r'DataBase//DataRaw//GLOBAL//WDD//'
log_Termination_path = user_credential / r'DataBase//DataRaw//BKN//MODIFIED_LOG//Termination_log.xlsx'
Termination_schema = ['FileName', 'ModifiedDate', 'EMPLOYEE_ID', 'PREVIOUS_PAYROLL_ID', 'FIRST_NAME', 'MIDDLE_NAME', 'LAST_NAME', 'FULL_NAME', 
                      'EMAIL_ADDRESS', 'HIRE_DATE', 'ORIGINAL_HIRE_DATE', 'END EMPLOYMENT DATE', 'Contract End Date', 'Termination Date', 
                      'Termination Date (DD/MM/YY)', 'Eligible for Rehire', 'LWD', 'MOST RECENT TERMINATION - DATE INITIATED', 
                      'MOST RECENT TERMINATION - DATE COMPLETED', 'MOST RECENT TERMINATION - EFFECTIVE DATE', 'MOST RECENT TERMINATION - REASON', 
                      'Action date', 'DATE INITIATED', 'COMPELETED DATE AND TIME', 'TERMINATION DATE 2', 'Is Initiated through Resignation', 
                      'Termination Reason', 'Resignation Reason', 'Secondary Termination Reasons', 'Resignation Notice served', 'PERSON_TYPE', 
                      'Time Type', 'Employee Type', 'Worker Type', 'Assignment Category', 'WORKER_CATEGORY', 'BUSINESS_UNIT', 'Cost Center', 
                      'Cost Center - ID', 'JOB_CODE', 'JOB_TITLE', 'BUSINESS_TITLE', 'LOCATION_NAME', 'LOCATION_CODE', 'COUNTRY', 'COMPANY', 
                      'MANAGEMENT LEVEL', 'JOB LEVEL', 'JOB_FAMILY', 'JOB_FUNCTION', 'JOB_ROLE', 'MSA', 'CNX BU', 'Concentrix LOB', 'Process', 
                      'Client Name ( Process )', 'Compensation Grade', 'SUPERVISOR_ID', 'SUPERVISOR_FULL_NAME', 'SUPERVISOR_EMAIL_ID', 'COMP_CODE', 
                      'CITY', 'LOCATION_DESCRIPTION', 'EMPLOYEE STATUS', 'Continuous Service Date', 'Work Related Status', 
                      'Work Related (Extended Status)', 'Activity', 'MSA Legacy Project ID']
# 1️⃣1️⃣[GLB]Resignation🗃️
Resignation_TABLE_NAME = "GLB.Resignation"
Folder_Resignation = user_credential / r'DataBase//DataRaw//GLOBAL//WDD//'
log_Resignation_path = user_credential / r'DataBase//DataRaw//BKN//MODIFIED_LOG//Resignation_log.xlsx'
Resignation_schema = ['FileName', 'ModifiedDate', 'Employee ID', 'Full Name', 'Job Family', 'MSA Client', 'Country', 'Location', 'Action', 
                      'Action Date', 'Date and Time Initiated', 'Status', 'Primary Reason', 'Secondary Reasons', 'Notification Date', 'Awaiting Persons', 
                      'Resignation Primary Reason', 'Hire Date', 'Proposed Termination Date', 'Notice Served', 'Sup ID', 'Supervisor Name', 
                      'Employee Status', 'Activity', 'MSA Legacy Project ID', 'Initiated By']
# 1️⃣2️⃣[BKN]CPI_PEGA🗃️
CPI_PEGA_TABLE_NAME = "BCOM.CPI_PEGA"
Folder_CPI_PEGA = user_credential / r'DataBase//DataRaw//BKN//CPI_PEGA//'
log_CPI_PEGA_path = user_credential / r'DataBase//DataRaw//BKN//MODIFIED_LOG//CPI_PEGA_log.xlsx'
CPI_PEGA_schema = ['FileName', 'ModifiedDate', 'Staff Name', 'Operator Def', 'Service Case Type New', 'Channel Def',	
                   'Lang Def', 'Reason For No Service Case', 'Topic Def New', 'Subtopics', 'Case Id', 'Reservation Id Def',
                   'Day of Date', 'Blank', '# Swivels', 'Count of ServiceCase or Interaction']
# 1️⃣3️⃣[BKN]Staff🗃️
Staff_TABLE_NAME = "BCOM.Staff"
Folder_Staff = user_credential / r'DataBase//DataRaw//BKN//AGENTS//'
log_Staff_path = user_credential / r'DataBase//DataRaw//BKN//MODIFIED_LOG//Staff_log.xlsx'
Staff_schema = ['FileName', 'ModifiedDate', 'Employee_ID', 'GEO', 'Site_ID', 'Employee_Last_Name', 'Employee_First_Name', 'Status', 'Wave #', 
                'Role', 'Booking Login ID', 'Language Start Date', 'TED Name', 'CUIC Name', 'EnterpriseName', 'Hire_Date', 'PST_Start_Date',
                'Production_Start_Date', 'LWD', 'Termination_Date', 'Designation', 'cnx_email', 'Booking Email', 'WAH Category', 'Full name',
                'IEX', 'serial_number', 'BKN_ID', 'Extension Number']
# 1️⃣4️⃣[BKN]ConTrack🗃️
ConTrack_TABLE_NAME = "BCOM.ConTrack"
Folder_ConTrack = Path(os.environ['USERPROFILE']) / r'OneDrive - Concentrix Corporation//DataBase//DataRaw//BKN//ContactTracker//'
log_ConTrack_path = user_credential / r'DataBase//DataRaw//BKN//MODIFIED_LOG//ConTrack_log.xlsx'
ConTrack_schema = ['FileName', 'ModifiedDate', 'Id', 'Start time', 'Completion time', 'Email', 'Name', 'Reservation Number', 'Contact Types',
                   'Contact Parties', 'Unbabel Tool Used?', 'Backlog Case', 'How many days since guest contacted? (ex: 30)', 'Topics',
                   'Resolutions', 'Reason If Skipped', 'CRM used', 'Outbound to Senior', 'Outbound Status','Reason (Name - Site of Senior)',
                   'Note', 'Reason for cannot make OB call to Guest', 'Is it possible to make Outbound call to Guest? ', 'Language']
# 1️⃣5️⃣[BKN]Quality🗃️
Quality_TABLE_NAME = "BCOM.Quality"
Folder_Quality = user_credential / r'DataBase//DataRaw//BKN//QUALITY//'
log_Quality_path = user_credential / r'DataBase//DataRaw//BKN//MODIFIED_LOG//Quality_log.xlsx'
Quality_schema = ['FileName', 'ModifiedDate', 'eps_name', 'eval_id', 'eval_date', 'agent_username', 'evaluator_username', 'result',	
                  'final_question_grouping', 'template_group', 'eval_template_name', 'sections', 'sitecode', 'score_n', 'score_question_weight',
                  'eval_language', 'eval_reference', 'tix_final_topic', 'tix_final_subtopic', 'csat_language_code', 'csat_satisfied']
# 1️⃣6️⃣[BKN]RONA🗃️
RONA_TABLE_NAME = "BCOM.RONA"
Folder_RONA = user_credential / r'DataBase//DataRaw//BKN//RONA//'
log_RONA_path = user_credential / r'DataBase//DataRaw//BKN//MODIFIED_LOG//RONA_log.xlsx'
RONA_schema = ['FileName', 'ModifiedDate', 'Agent', 'DateTime', 'RONA']
# 1️⃣7️⃣[BKN]CUIC🗃️
CUIC_TABLE_NAME = "BCOM.CUIC"
Folder_CUIC = user_credential / r'DataBase//DataRaw//BKN//CUIC//'
log_CUIC_path = user_credential / r'DataBase//DataRaw//BKN//MODIFIED_LOG//CUIC_log.xlsx'
CUIC_schema = ['FileName', 'ModifiedDate', 'FullName', 'LoginName', 'Interval', 'AgentAvailTime', 'AgentLoggedOnTime']
# 1️⃣8️⃣[BKN]KPI_Target🗃️
KPI_Target_TABLE_NAME = "BCOM.KPI_Target"
Folder_KPI_Target = user_credential / r'DataBase//DataRaw//BKN//KPI_TARGET//'
log_KPI_Target_path = user_credential / r'DataBase//DataRaw//BKN//MODIFIED_LOG//KPI_Target_log.xlsx'
KPI_Target_schema = ['FileName', 'ModifiedDate', 'LOB', 'LOB Group', 'Week', 'Tenure days', 'Overall CPH tar', 'Phone CPH tar', 'Non Phone CPH tar',	
                     'Quality - Customer Impact tar', 'Quality - Business Impact tar', 'Quality - Compliance Impact tar', 'Quality - Overall tar', 'AHT Phone tar',	
                     'AHT Non-phone tar', 'AHT Overall tar', 'Hold (phone) tar', 'AACW (phone) tar', 'Avg Talk Time tar', 'Phone CSAT tar', 'Non phone CSAT tar',	
                     'Overall CSAT tar', 'PSAT tar', 'PSAT Vietnamese tar', 'PSAT English (American) tar', 'PSAT English (Great Britain) tar', 'CSAT Reso tar',
                     'Quality - personalization tar', 'Quality - proactivity tar', 'Quality - resolution tar']
# 1️⃣9️⃣[BKN]LogoutCount🗃️
LogoutCount_TABLE_NAME = "BCOM.LogoutCount"
Folder_LogoutCount = user_credential / r'DataBase//DataRaw//BKN//LOGOUT_COUNT//'
log_LogoutCount_path = user_credential / r'DataBase//DataRaw//BKN//MODIFIED_LOG//LogoutCount_log.xlsx'
LogoutCount_schema = ['FileName', 'ModifiedDate', 'Aggregation', 'TimeDimension', 'KPI Value Formatted']
# 2️⃣0️⃣[BKN]WpDetail🗃️
WpDetail_TABLE_NAME = "BCOM.WpDetail"
Folder_WpDetail = user_credential / r'DataBase//DataRaw//BKN//WP_DETAIL//'
log_WpDetail_path = user_credential / r'DataBase//DataRaw//BKN//MODIFIED_LOG//WpDetail_log.xlsx'
WpDetail_schema = ['FileName', 'ModifiedDate', 'LOB', 'ID', 'DateTime_Start', 'DateTime_End', 'Date_Start', 'Date_end', 'Time_Start', 'Time_End', 
                   'Dur', 'Action', 'DateTime_Act_Start', 'DateTime_Act_End', 'Date_Act_Start', 'Date_Act_End', 'Time_Act_Start', 'Time_Act_End', 'Act_Dur']
# 2️⃣1️⃣[BKN]WpSummary🗃️
WpSummary_TABLE_NAME = "BCOM.WpSummary"
Folder_WpSummary = user_credential / r'DataBase//DataRaw//BKN//WP_SUMMARY//'
log_WpSummary_path = user_credential / r'DataBase//DataRaw//BKN//MODIFIED_LOG//WpSummary_log.xlsx'
WpSummary_schema = ['FileName', 'ModifiedDate', 'LOB', 'Date', 'Agent ID', 'Agent Name', 'Scheduled Activity', 'Length', 'Percent']
# 2️⃣2️⃣[BKN]RegisteredOT🗃️
RegisteredOT_TABLE_NAME = "BCOM.RegisteredOT"
Folder_RegisteredOT = user_credential / r'DataBase//DataRaw//BKN//OVERTIME//'
log_RegisteredOT_path = user_credential / r'DataBase//DataRaw//BKN//MODIFIED_LOG//RegisteredOT_log.xlsx'
RegisteredOT_schema = ['FileName', 'ModifiedDate', 'Emp ID', 'Name', 'Date', 'Value', 'OT', 'LOB','Type']
# 2️⃣3️⃣[BKN]CSAT_TP🗃️
CSAT_TP_TABLE_NAME = "BCOM.CSAT_TP"
Folder_CSAT_TP = user_credential / r'DataBase//DataRaw//BKN//CSAT//'
log_CSAT_TP_path = user_credential / r'DataBase//DataRaw//BKN//MODIFIED_LOG//CSAT_TP_log.xlsx'
CSAT_TP_schema = ['FileName', 'ModifiedDate', 'Sort by Dimension', 'Survey Id', 'Reservation', 'Team', 'Channel', 'Staff', 'Type', 'Date',
                  'Topic of the first Ticket', 'Language', 'Csat 2.0 Score', 'Has Comment', '"Comment"', 'Reservation Link', 'View comment',
                  'Sort by Dimension (copy)', 'Max. Sort by Dimension']
# 2️⃣4️⃣[BKN]CSAT_RS🗃️
CSAT_RS_TABLE_NAME = "BCOM.CSAT_RS"
Folder_CSAT_RS = user_credential / r'DataBase//DataRaw//BKN//CSAT_RESO//'
log_CSAT_RS_path = user_credential / r'DataBase//DataRaw//BKN//MODIFIED_LOG//CSAT_RS_log.xlsx'
CSAT_RS_schema = ['FileName', 'ModifiedDate', 'Sort by Dimension', 'Survey Id', 'Reservation', 'Team', 'Channel', 'Staff', 'Type', 'Date',
                  'Topic of the first Ticket', 'Language', 'Csat 2.0 Score', 'Has Comment', '"Comment"', 'Reservation Link', 'View comment',
                  'Sort by Dimension (copy)', 'Max. Sort by Dimension']
# 2️⃣5️⃣[BKN]PSAT🗃️
PSAT_TABLE_NAME = "BCOM.PSAT"
Folder_PSAT = user_credential / r'DataBase//DataRaw//BKN//PSAT//'
log_PSAT_path = user_credential / r'DataBase//DataRaw//BKN//MODIFIED_LOG//PSAT_log.xlsx'
PSAT_schema = ['FileName', 'ModifiedDate', 'Sorted By Dimension', 'Survey Id', 'Date', 'Staff Name', 'Language', 'Final Topics',
               'How satisfied were you with our service?', 'How difficult did we make it or you to solve your issue?', 'Agent understood my question',
               'Agent did everything possible to help me', 'Did we fully resolve your issue?', 'Channel', 'Hotel Id', '"Comment"',
               'Has Comment', 'Sorted BY Dimension (copy)']
# 2️⃣6️⃣[BKN]IEX_Hrs🗃️
IEX_Hrs_TABLE_NAME = "BCOM.IEX_Hrs"
Folder_IEX_Hrs = user_credential / r'DataBase//DataRaw//BKN//WP_INTERVAL//'
log_IEX_Hrs_path = user_credential / r'DataBase//DataRaw//BKN//MODIFIED_LOG//IEX_Hrs_log.xlsx'
IEX_Hrs_schema = ['FileName', 'ModifiedDate', 'LOB', 'VNT', 'CET', 'HC', 'Hour']
# 2️⃣7️⃣[BKN]IntervalReq🗃️
IntervalReq_TABLE_NAME = "BCOM.IntervalReq"
Folder_IntervalReq = user_credential / r'DataBase//DataRaw//BKN//INTERVAL_REQUIREMENT//'
log_IntervalReq_path = user_credential / r'DataBase//DataRaw//BKN//MODIFIED_LOG//IntervalReq_log.xlsx'
IntervalReq_schema = ['FileName', 'ModifiedDate', 'LOB', 'Datetime_CET', 'Datetime_VN', 'Value', 'Delivery_Req']
# 2️⃣8️⃣[BKN]ExceptionReq🗃️
ExceptionReq_TABLE_NAME = "BCOM.ExceptionReq"
Folder_ExceptionReq = user_credential / r'DataBase//DataRaw//BKN//EXCEPTION_REQ//'
log_ExceptionReq_path = user_credential / r'DataBase//DataRaw//BKN//MODIFIED_LOG//ExceptionReq_log.xlsx'
ExceptionReq_schema = ['FileName', 'ModifiedDate', 'Emp ID', 'Date (MM/DD/YYYY)', 'Exception request (Minute)', 'Reason', 'TL', 'OM']
# 2️⃣9️⃣[BKN]LTTransfers🗃️
LTTransfers_TABLE_NAME = "BCOM.LTTransfers"
Folder_LTTransfers = user_credential / r'DataBase//DataRaw//BKN//HC_TRANSFER//'
log_LTTransfers_path = user_credential / r'DataBase//DataRaw//BKN//MODIFIED_LOG//LTTransfers_log.xlsx'
LTTransfers_schema = ['FileName', 'ModifiedDate', 'EID', 'Full Name', 'Employee Status', 'LWD', 'Remarks']
# 3️⃣0️⃣[BKN]DailyReq🗃️
DailyReq_TABLE_NAME = "BCOM.DailyReq"
Folder_DailyReq = user_credential / r'DataBase//DataRaw//BKN//REQUIREMENT_HOURS//'
log_DailyReq_path = user_credential / r'DataBase//DataRaw//BKN//MODIFIED_LOG//DailyReq_log.xlsx'
DailyReq_schema = ['FileName', 'ModifiedDate', 'LOB', 'Date', 'Daily Requirement', 'Prod Requirement']
# 3️⃣1️⃣[BKN]ProjectedShrink🗃️
ProjectedShrink_TABLE_NAME = "BCOM.ProjectedShrink"
Folder_ProjectedShrink = user_credential / r'DataBase//DataRaw//BKN//SHRINKAGE_TARGET//'
log_ProjectedShrink_path = user_credential / r'DataBase//DataRaw//BKN//MODIFIED_LOG//ProjectedShrink_log.xlsx'
ProjectedShrink_schema = ['FileName', 'ModifiedDate', 'LOB', 'Week', 'Ratio']
# 3️⃣2️⃣[BKN]OTReq🗃️
OTReq_TABLE_NAME = "BCOM.OTReq"
Folder_OTReq = user_credential / r'DataBase//DataRaw//BKN//OT_REQ//'
log_OTReq_path = user_credential / r'DataBase//DataRaw//BKN//MODIFIED_LOG//OTReq_log.xlsx'
OTReq_schema = ['FileName', 'ModifiedDate', 'Date', 'LOB', 'OT Hour', 'Type']
# 3️⃣3️⃣[BKN]CapHC🗃️
CapHC_TABLE_NAME = "BCOM.CapHC"
Folder_CapHC = user_credential / r'DataBase//DataRaw//BKN//CAPACITY_HC//'
log_CapHC_path = user_credential / r'DataBase//DataRaw//BKN//MODIFIED_LOG//CapHC_log.xlsx'
CapHC_schema = ['FileName', 'ModifiedDate', 'LOB', 'Date', 'Client Requirement (Hours)']
# 3️⃣4️⃣[BKN]ProjectedHC🗃️
ProjectedHC_TABLE_NAME = "BCOM.ProjectedHC"
Folder_ProjectedHC = user_credential / r'DataBase//DataRaw//BKN//PROJECTED_HEADCOUNT//'
log_ProjectedHC_path = user_credential / r'DataBase//DataRaw//BKN//MODIFIED_LOG//ProjectedHC_log.xlsx'
ProjectedHC_schema = ['FileName', 'ModifiedDate', 'Date', 'LOB', 'FTE Required', 'Projected HC', 'Plan Leave', 
                      'Actual Projected HC', '%OO', '%IO', 'Projected HC with Shrink', 'OT', 'Leave allow for Shrink', '% Deli']
# 3️⃣5️⃣[BKN]RampHC🗃️
RampHC_TABLE_NAME = "BCOM.RampHC"
Folder_RampHC = user_credential / r'DataBase//DataRaw//BKN//RAMPUP_HC//'
log_RampHC_path = user_credential / r'DataBase//DataRaw//BKN//MODIFIED_LOG//RampHC_log.xlsx'
RampHC_schema = ['FileName', 'ModifiedDate', 'Date', 'LOB', 'Headcount', 'Hours']

# 3️⃣6️⃣[BKN]SEAT🗃️
SEAT_TABLE_NAME = "BCOM.SEAT"
Folder_SEAT = user_credential / r'DataBase//DataFrame//BKN//SEAT_MAP//History data//'
Folder_SEAT2 = user_credential / r'DataBase//DataFrame//BKN//SEAT_MAP//SEAT_MAP_DATA//'
log_SEAT_path = user_credential / r'DataBase//DataRaw//BKN//MODIFIED_LOG//SEAT_log.xlsx'
SEAT_schema = ['FileName','ModifiedDate','Date','Emp ID','TED Name','Week_day','Seat No','Floor','Building']

# 3️⃣7️⃣[BKN]SEAT_AVAIL🗃️
SEAT_AVAIL_TABLE_NAME = "BCOM.SEAT_Avail"
Folder_SEAT_AVAIL = user_credential / r'DataBase//DataFrame//BKN//SEAT_MAP//History data//'
Folder_SEAT_AVAIL2 = user_credential / r'DataBase//DataFrame//BKN//SEAT_MAP//SEAT_MAP_DATA//'
log_SEAT_AVAIL_path = user_credential / r'DataBase//DataRaw//BKN//MODIFIED_LOG//SEAT_AVAIL_log.xlsx'
SEAT_AVAIL_schema = ['FileName','ModifiedDate','Seat No','Floor','Building','Code','Date','Open Hours','Closed Hours']

# 3️⃣8️⃣[BKN]CSAT_Comp🗃️
CSAT_Comp_TABLE_NAME = "BCOM.CSAT_Comp"
Folder_CSAT_Comp = user_credential / r'DataBase//DataRaw//BKN//CSAT_Comp//'
log_CSAT_Comp_path = user_credential / r'DataBase//DataRaw//BKN//MODIFIED_LOG//CSAT_Comp_log.xlsx'
CSAT_Comp_schema = ['FileName','ModifiedDate','Date','REGION','Site type','Site Name','Company','Language',
                    'Overall Surveys','Overall CSAT','Messaging surveys','Messaging CSAT','Email surveys',
                    'Email CSAT','Phone surveys','Phone CSAT']
# -----------------------------------------------------------------------------------------------#
# Database_Connecter🧬
summary_results = {
    "new_files_imported": [],
    "updated_files_reimported": [],
    "skipped_files_unchanged": [],
    "failed_imports": []
}
server_name = "PHMANVMDEV01V"
server_ip = "10.5.11.60"
database = "wfm_vn_dev"
user = "usr_wfmvn_dev"
password = "12guWU2OdEj5kEspl9Rlfoglf"
# SQL Server Authentication 🔗
connection_string = f"mssql+pyodbc://{user}:{password}@{server_ip}/{database}?driver=ODBC+Driver+17+for+SQL+Server"
# Windows Authentication 🔗
# connection_string = f"mssql+pyodbc://{server_name}/{database}?driver=ODBC+Driver+17+for+SQL+Server&Trusted_Connection=yes"
try:
    engine = create_engine(connection_string, fast_executemany=True)
    logger.info(f"✅ Successfully connected to DB: {database} server: {server_ip}")
except Exception as e:
    logger.exception("❌ DB Connection error")
    raise

In [ ]:
# Function Definition🛠️

# Log Color view💡
def print_colored(text, color):
    display(HTML(f'<span style="color: {color};">{text}</span>'))

# Check existing log file💡
def read_or_create_log(log_path):
    with warnings.catch_warnings():
        warnings.simplefilter('ignore') # Ignor Warning📃
        try:
            logger.debug(f"Reading log file: {log_path}")
            log_df = pl.read_excel(log_path)
            log_df = log_df.with_columns([pl.col("ModifiedDate").dt.cast_time_unit("ms")], strict=False)
            logger.info(f"Success read log file: {log_path}")
        except FileNotFoundError: # Create new log if can't find log📃
            logger.warning(f"Log file not found: {log_path}. Create new log.")
            log_df = pl.DataFrame(
                {
                    "FileName": pl.Series([], dtype=pl.Utf8),
                    "ModifiedDate": pl.Series([], dtype=pl.Datetime),
                    "Error": pl.Series([], dtype=pl.Utf8),})
        except Exception as e: # Create new log if can't open log📃
            logger.exception(f"Error reading log file: {log_path}")
            print(f"Error reading log file: {e}")
            log_df = pl.DataFrame(
                {
                    "FileName": pl.Series([], dtype=pl.Utf8),
                    "ModifiedDate": pl.Series([], dtype=pl.Datetime),
                    "Error": pl.Series([], dtype=pl.Utf8),})
        return log_df
        
# Update log_df💡
def process_and_save_log(log_df, log_entries, log_path):
    if log_entries:
        new_log_df = pl.DataFrame(log_entries)
        log_df = log_df.with_columns(pl.col('ModifiedDate').dt.cast_time_unit("ms"))
        log_df = (pl.concat([log_df, new_log_df], how="diagonal_relaxed") # Combine and remove duplicate New_Log and Old_Log📃
                  .sort("ModifiedDate", descending=[False])
                  .unique(subset=["FileName"], keep="last")
                  .sort("FileName", descending=[False])
                  .select(["FileName", "ModifiedDate", "Error"]))
        try:
            log_df.write_excel(log_path, worksheet="ImportLog", autofit=True)
            print(f"Import log saved to: {log_path}")
            logger.info(f"Import log saved to: {log_path}")
        except Exception as e:
            print(f"Error writing log file: {e}")
            logger.error(f"Error writing log file: {log_path} - {e}")

# write_data💡
def write_data(engine, table_name, df): # write to database📃
     df.write_database(table_name=table_name, connection=engine, if_table_exists="append")
    
# delete_data💡
def delete_data(engine, table_name, filename):
    try:
        with engine.connect() as connection:
            print_colored(f"Prepare to delete old data for '{filename}' in '{table_name}'", "DarkTurquoise")
            logger.warning(f"Prepare to delete old data for '{filename}' in '{table_name}'")
            delete_query = text(f"DELETE FROM {table_name} WHERE [FileName] = :filename")
            connection.execute(delete_query, {"filename": filename})
            connection.commit()
            print_colored(f"Old data deleted successfully🧹", "DarkTurquoise")
            logger.info(f"'{filename}' data deleted successfully in '{table_name}' 🧹.")
    except Exception as e:
        logger.exception(f"Error while delete data for '{filename}' in '{table_name}'")
        print_colored(f"Error while delete data for '{filename}' in '{table_name}'", "DarkTurquoise")
        raise 
        
# Check Time💡
def is_time_between(begin_time, end_time, check_time=None):
    check_time = check_time or datetime.utcnow().time() # If check time is not given, default to current UTC time📃
    if begin_time < end_time:
        return check_time >= begin_time and check_time <= end_time
    else: # crosses midnight📃
        return check_time >= begin_time or check_time <= end_time
def time_difference(time1, time2):
    seconds1 = time1.hour * 3600 + time1.minute * 60 + time1.second # Convert times to seconds📃
    seconds2 = time2.hour * 3600 + time2.minute * 60 + time2.second
    diff_seconds = seconds1 - seconds2
    return diff_seconds

# Final Summary💡
def display_summary(source_name: str, error_count: int) -> None:
    """Final Notice."""
    if error_count > 0:
        print_colored(f"Finished processing all files ({error_count} have errors🛠️).", "OrangeRed")
        logger.warning(f"Finished processing all files ({error_count} have errors🛠️).")
    else:
        print_colored(f"Finished processing all files (no errors🎉).", "PaleVioletRed")
        logger.info(f"Finished processing [{source_name}] (no errors🎉).")

# Default_variable💡
def Default_variable():
    log_entries = []
    error_count = 0
    return log_entries, error_count

# parse_date💡
def parse_date(col: pl.Expr) -> pl.Expr:
    return pl.coalesce(
        col.str.strptime(pl.Date, format="%m/%d/%Y", strict=False),
        col.str.strptime(pl.Date, format="%Y-%m-%d", strict=False),
        col.str.strptime(pl.Date, format="%d %B %Y", strict=False),
        col.str.strptime(pl.Date, format="%B %d, %Y", strict=False),
        col.str.strptime(pl.Date, format="%d-%b-%y", strict=False),
        col.str.strptime(pl.Date, format="%Y%m%d", strict=False),
        col.str.strptime(pl.Date, format="%d/%m/%y", strict=False),
        col.str.strptime(pl.Date, format="%d-%m-%Y", strict=False),
    )

# validate_schema💡
def validate_schema(df: pl.DataFrame, expected_schema: list[str], filename: str) -> tuple[bool, str | None]:
    # Start validation
    start_msg = f"🔍 Starting schema validation for file: {filename}"
    logger.info(start_msg)
    print_colored(start_msg, "DodgerBlue")
    actual_columns = df.columns
    expected_set = set(expected_schema)
    actual_set = set(actual_columns)
    missing_columns = expected_set - actual_set
    extra_columns = actual_set - expected_set
    has_critical_error = False
    critical_error_message = None
    has_warnings = False
    # 1. Schema error (Missing columns)
    if missing_columns:
        has_critical_error = True
        critical_error_message = f"Schema error in the file: '{filename}'. Missing columns: {sorted(list(missing_columns))}"
        logger.error(critical_error_message)
        print_colored(f"❗️ {critical_error_message}", "OrangeRed")
    # 2. warning extra columns
    if extra_columns:
        has_warnings = True
        warning_message = f"warning schema for file '{filename}'. Extra columns: {sorted(list(extra_columns))}. These columns will be excluded from the import process."
        logger.warning(warning_message)
        print_colored(f"⚠️ {warning_message}", "Gold")
    # 3. Final results announcement
    if not has_critical_error and not has_warnings:
        final_msg = f"✅ Completely valid schema for the file: {filename}."
        logger.info(final_msg)
        print_colored(final_msg, "MediumSeaGreen")
    elif not has_critical_error and has_warnings:
        final_msg = f"⚠️ File schema check: {filename} Passed (No missing columns, extra columns warned)"
        logger.info(final_msg)
        print_colored(final_msg, "MediumSeaGreen") # Vẫn dùng màu xanh lá
    elif has_critical_error:
        final_msg = f"❌ Schema validation failed due to missing column(s) for file: {filename}."
        logger.warning(final_msg) # Log ở mức warning hoặc error tùy ý
        print_colored(final_msg, "OrangeRed")
    return has_critical_error, critical_error_message
    
# DF Info💡
def info_polars(df: pl.DataFrame):
    print_colored(f"⚙️Final structure", "Olive")
    logger.info(f"⚙️Final structure")
    shape = df.shape
    print(f"Shape: {shape}")
    print("Data columns:")  
    table_data = []
    for i, name in enumerate(df.columns):
        dtype = df.dtypes[i]
        non_null_count = df.select(pl.col(name).is_not_null().sum()).item()
        table_data.append([i, name, non_null_count, dtype])  
    headers = ["#", "Column", "Non-Null Count", "Dtype"]
    print(tabulate(table_data, headers=headers, tablefmt="grid"))
    logger.info(tabulate(table_data, headers=headers, tablefmt="grid"))

# Summary💡
def generate_summary_report(results):
    report = []
    total_processed = (len(results["new_files_imported"]) + 
                       len(results["updated_files_reimported"]) + 
                       len(results["failed_imports"]))
    
    report.append("╔══════════════════════════════╗")
    report.append("║ 📊 Server Import Summary 📊 ║")
    report.append("╚══════════════════════════════╝")
    report.append(f"\n✨ Processed a total of {total_processed} files.")
    
    # 1. Các tệp mới được nhập
    report.append("\n--- ✅ New file has been imported successfully ---")
    if results["new_files_imported"]:
        for file in results["new_files_imported"]:
            report.append(f"  - {file}")
    else:
        report.append("  (Nothing)")

    # 2. Các tệp được cập nhật
    report.append("\n--- 🔄️ File Updated Successfully ---")
    if results["updated_files_reimported"]:
        for file in results["updated_files_reimported"]:
            report.append(f"  - {file}")
    else:
        report.append("  (Nothing)")

    # 3. Các tệp xử lý lỗi
    report.append("\n--- ❗️Failed Files ---")
    if results["failed_imports"]:
        for file, error in results["failed_imports"]:
            report.append(f"  - {file}")
            report.append(f"    └─ Lỗi: {error}")
    else:
        report.append("  (All Good 🎉)")

    # 4. Các tệp được bỏ qua
    report.append("\n--- 🚀 skipped files (unchanged) ---")
    if results["skipped_files_unchanged"]:
         report.append(f"  (Tổng cộng: {len(results['skipped_files_unchanged'])} tệp)")
        # report.append("  " + ", ".join(results["skipped_files_unchanged"])) # Bỏ comment dòng này nếu bạn muốn liệt kê tất cả
    else:
        report.append("  (Nothing)")
        
    report.append("\n═════════════════════ Finish ═════════════════════")
    return "\n".join(report)

In [ ]:
# 0️⃣1️⃣[BKN]AHT2💾
logger.info("===== Start AHT2 Process =====") #🧩 Log the start of the data processing.
log_df = read_or_create_log(log_AHT2_path) #🧩 Read the existing Excel log file or create a new one if it doesn't exist.
log_entries, error_count = Default_variable() #📃 Initialize a list to store log results for the current run and an error counter.
#💡 Iterate through all files in the specified folder.
for filename in Folder_AHT2.glob("*.csv"): #🧩 Get files in the Folder directory.
    #📃 Get the current modification timestamp and filename of the current file.
    current_modified_date_microseconds = datetime.datetime.fromtimestamp(filename.stat().st_mtime) # Get modification timestamp, including microseconds.
    current_modified_date = current_modified_date_microseconds.replace(microsecond=0) # Remove microseconds for easier comparison.
    file_basename = filename.name # Get the base name of the file (e.g., "data_20230101.csv").
    #📃 File is being processed.
    print_colored(f"Processing file🔄️: {file_basename}", "NavajoWhite")
    logger.info(f"Processing file🔄️: {file_basename}")
    #📃 Check if this file has been logged before.
    log_entry = log_df.filter(pl.col("FileName") == file_basename) # Filter the old log_df to find an entry for this filename.
    is_new_file = log_entry.is_empty() # If no entry is found (log_entry is empty), it's a new file.
    previous_modified_date = datetime.datetime(1900, 1, 1, 0, 0, 1)  # Initialize the previous modification date with a very old date.
    #📃 If this file was processed before (exists in the log).  
    if not is_new_file:
        previous_modified_date = log_entry.select("ModifiedDate").item() # Get the modification date recorded in the log from the previous processing.
    #📃 Condition to process the file:
    # 1. The file is new (not in the log).
    # 2. Or, the file is old (in the log), but its current modification date (current_modified_date) is more recent than the modification date recorded in the log (previous_modified_date).
    if is_new_file or current_modified_date > previous_modified_date:
        if not is_new_file: # If it's an old file but has changed (ModifiedDate is newer).
            print_colored(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.", "Coral")
            logger.warning(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.")
        else: # If it's a completely new file.
            print_colored(f"New file detected🚦: {file_basename}. Importing.", "Coral")
            logger.info(f"New file detected🚦: {file_basename}. Importing.")
        try: # Start a try-except block to handle potential errors during the import process.
            if not is_new_file: # If this file already exists in the DB (because it's not new and its ModifiedDate changed) then delete the old data for this file from the df.
                delete_data(engine, AHT2_TABLE_NAME, file_basename)  #🧩 Call the function to delete data.
            #📃 Read data from the excel file using Polars.
            AHT2 = (pl.read_csv(filename, infer_schema_length=None) #🧩 Read the entire file to infer the schema.Add two new columns to the DataFrame: FileName and ModifiedDate.
                   .select(pl.all(),pl.lit(file_basename).alias("FileName"),pl.lit(current_modified_date).dt.cast_time_unit("ms").alias("ModifiedDate")
                   )) #🧩        
            #📃 Validate if the columns in the DataFrame match the defined schema
            has_critical_error, critical_schema_error_msg = validate_schema(AHT2, AHT2_schema, file_basename) #🧩
            if has_critical_error: # If critical columns are missing.
                raise ValueError(critical_schema_error_msg) # Raise an error to stop processing this file.
            #📃 Adjust the column structure.
            print_colored(f"⚙️Adjust column structure", "Olive")
            logger.warning(f"⚙️Adjust column structure")
            AHT2 = AHT2.with_columns(parse_date(pl.col("Date")).alias("Date"),).select(AHT2_schema) #🧩 Select columns in the correct order and names as per df_schema.
            print_colored(f"⚙️Column structure adjustment completed", "Olive")
            logger.info(f"⚙️Column structure adjustment completed")
            info_polars(AHT2) #🧩 Display information about the DataFrame (shape, columns, dtypes).
            #📃 Start writing data to the database.
            print_colored(f"💾Start import file: {file_basename}", "Coral")
            logger.info(f"💾Start import file: {file_basename}")
            write_data(engine, AHT2_TABLE_NAME, AHT2) #🧩 Call the function to write the DataFrame to SQL.
            print_colored(f"Successfully imported✨: {file_basename}", "LimeGreen")
            logger.info(f"Successfully imported✨: {file_basename}")
            #📃 Record to summary
            if is_new_file:
                summary_results["new_files_imported"].append(f"[AHT2] {file_basename}")#🧩
            else:
                summary_results["updated_files_reimported"].append(f"[AHT2] {file_basename}")#🧩    
            #📃 Update log_entries for successfully import: Add information to the log list for this run (successful).
            log_entries.append({"FileName": file_basename, "ModifiedDate": current_modified_date, "Error": None})
            print_colored(f"Updated 'Modified' time for {file_basename}", "LimeGreen")
            logger.info(f"Updated 'Modified' time for {file_basename}")
        #📃 Handle exceptions if any error occurred in the try block.
        except (
            FileNotFoundError, PermissionError, UnicodeDecodeError, IOError,                
            pl.exceptions.NoDataError, pl.exceptions.ComputeError, pl.exceptions.SchemaError, sa.exc.SQLAlchemyError,
            Exception # Catch various types of potential errors.
        ) as e:
            error_count += 1 # Increment the error counter.        
            error_msg_short = f"{type(e).__name__}: {str(e).splitlines()[0]}"  # Get a short error message (the first line).   
            #📃 Update Failed file to Summary
            summary_results["failed_imports"].append((f"[AHT2] {file_basename}", error_msg_short))#🧩   
            logger.error(f"Error importing {file_basename}: {error_msg_short}", exc_info=True) # Log the full error traceback.
            print_colored(f"❗️Error importing {file_basename}: {error_msg_short}", "OrangeRed")
            # Add information to the log list for this run (failed).
            # Note: ModifiedDate here is previous_modified_date, as this file wasn't successfully imported with current_modified_date.
            log_entries.append({"FileName": file_basename,"ModifiedDate": previous_modified_date,"Error": error_msg_short})
    else: #📃 If the file is not new and its modification date hasn't changed.
          #📃 Update Skip file to Summary
        summary_results["skipped_files_unchanged"].append(f"[AHT2] {file_basename}")#🧩
        print_colored(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.", "Aquamarine")
        logger.info(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.")
#💡 After iterating through all files in the directory.
display_summary("AHT2", error_count) #🧩 Display a summary of the results (number of errors).
process_and_save_log(log_df, log_entries, log_AHT2_path) #🧩 Update the Excel log file with information
logger.info("===== Processing of the AHT2 data source is complete =====") #🧩 Log finish data processing.

In [ ]:
# 0️⃣2️⃣[BKN]ROSTER💾
logger.info("===== Start ROSTER Process =====")  #🧩 Log the start of the data processing.
log_df = read_or_create_log(log_ROSTER_path) #🧩 Read the existing Excel log file or create a new one if it doesn't exist.
log_entries, error_count = Default_variable() #📃 Initialize a list to store log results for the current run and an error counter.
#💡 Iterate through all files in the specified folder.
for filename in Folder_ROSTER.glob("*.xlsx"): #🧩 Get files in the Folder directory.
    #📃 Get the current modification timestamp and filename of the current file.
    current_modified_date_microseconds = datetime.datetime.fromtimestamp(filename.stat().st_mtime) # Get modification timestamp, including microseconds.
    current_modified_date = current_modified_date_microseconds.replace(microsecond=0) # Remove microseconds for easier comparison.
    file_basename = filename.name # Get the base name of the file (e.g., "data_20230101.csv").
    #📃 File is being processed.
    print_colored(f"Processing file🔄️: {file_basename}", "NavajoWhite")
    logger.info(f"Processing file🔄️: {file_basename}")
    #📃 Check if this file has been logged before.
    log_entry = log_df.filter(pl.col("FileName") == file_basename) # Filter the old log_df to find an entry for this filename.
    is_new_file = log_entry.is_empty() # If no entry is found (log_entry is empty), it's a new file.
    previous_modified_date = datetime.datetime(1900, 1, 1, 0, 0, 1)  # Initialize the previous modification date with a very old date.
    #📃 If this file was processed before (exists in the log).  
    if not is_new_file:
        previous_modified_date = log_entry.select("ModifiedDate").item() # Get the modification date recorded in the log from the previous processing.
    #📃 Condition to process the file:
    # 1. The file is new (not in the log).
    # 2. Or, the file is old (in the log), but its current modification date (current_modified_date) is more recent than the modification date recorded in the log (previous_modified_date).
    if is_new_file or current_modified_date > previous_modified_date:
        if not is_new_file: # If it's an old file but has changed (ModifiedDate is newer).
            print_colored(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.", "Coral")
            logger.warning(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.")
        else: # If it's a completely new file.
            print_colored(f"New file detected🚦: {file_basename}. Importing.", "Coral")
            logger.info(f"New file detected🚦: {file_basename}. Importing.")
        try: # Start a try-except block to handle potential errors during the import process.
            if not is_new_file: # If this file already exists in the DB (because it's not new and its ModifiedDate changed) then delete the old data for this file from the df.
                delete_data(engine, ROSTER_TABLE_NAME, file_basename)  #🧩 Call the function to delete data.
            #📃 Read data from the excel file using Polars.
            ROSTER = (pl.read_excel(filename, infer_schema_length=None, engine="calamine", sheet_name="Sheet1") #🧩 Read the entire file to infer the schema.Add two new columns to the DataFrame: FileName and ModifiedDate.
                    .select(pl.all(),pl.lit(file_basename).alias("FileName"),pl.lit(current_modified_date).dt.cast_time_unit("ms").alias("ModifiedDate")
                   )) #🧩
            #📃 Validate if the columns in the DataFrame match the defined schema
            has_critical_error, critical_schema_error_msg = validate_schema(ROSTER, ROSTER_schema, file_basename) #🧩
            if has_critical_error: # If critical columns are missing.
                raise ValueError(critical_schema_error_msg) # Raise an error to stop processing this file.
            #📃 Adjust the column structure.
            print_colored(f"⚙️Adjust column structure", "Olive")
            logger.warning(f"⚙️Adjust column structure")
            ROSTER = ROSTER.with_columns(pl.col("Attribute").cast(pl.Date)).select(ROSTER_schema) #🧩 Select columns in the correct order and names as per df_schema.
            print_colored(f"⚙️Column structure adjustment completed", "Olive")
            logger.info(f"⚙️Column structure adjustment completed")
            info_polars(ROSTER) #🧩 Display information about the DataFrame (shape, columns, dtypes).
            #📃 Start writing data to the database.
            print_colored(f"💾Start import file: {file_basename}", "Coral")
            logger.info(f"💾Start import file: {file_basename}")
            write_data(engine, ROSTER_TABLE_NAME, ROSTER) #🧩 Call the function to write the DataFrame to SQL.
            print_colored(f"Successfully imported✨: {file_basename}", "LimeGreen")
            logger.info(f"Successfully imported✨: {file_basename}")
            #📃 Record to summary
            if is_new_file:
                summary_results["new_files_imported"].append(f"[ROSTER] {file_basename}")#🧩
            else:
                summary_results["updated_files_reimported"].append(f"[ROSTER] {file_basename}")#🧩
            #📃 Update log_entries for successfully import: Add information to the log list for this run (successful).
            log_entries.append({"FileName": file_basename, "ModifiedDate": current_modified_date, "Error": None}) 
            print_colored(f"Updated 'Modified' time for {file_basename}", "LimeGreen")
            logger.info(f"Updated 'Modified' time for {file_basename}")
            #📃 Handle exceptions if any error occurred in the try block.
        except (
            FileNotFoundError, PermissionError, UnicodeDecodeError, IOError,                
            pl.exceptions.NoDataError, pl.exceptions.ComputeError, pl.exceptions.SchemaError, sa.exc.SQLAlchemyError,
            Exception # Catch various types of potential errors.
        ) as e:
            error_count += 1 # Increment the error counter.        
            error_msg_short = f"{type(e).__name__}: {str(e).splitlines()[0]}"  # Get a short error message (the first line).
            #📃 Update Failed file to Summary
            summary_results["failed_imports"].append((f"[ROSTER] {file_basename}", error_msg_short))#🧩    
            logger.error(f"Error importing {file_basename}: {error_msg_short}", exc_info=True) # Log the full error traceback.
            print_colored(f"❗️Error importing {file_basename}: {error_msg_short}", "OrangeRed")
            # Add information to the log list for this run (failed).
            # Note: ModifiedDate here is previous_modified_date, as this file wasn't successfully imported with current_modified_date.
            log_entries.append({"FileName": file_basename,"ModifiedDate": previous_modified_date,"Error": error_msg_short})
    else: #📃 If the file is not new and its modification date hasn't changed.
          #📃 Update Skip file to Summary
        summary_results["skipped_files_unchanged"].append(f"[ROSTER] {file_basename}")#🧩       
        print_colored(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.", "Aquamarine")
        logger.info(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.")
#💡 After iterating through all files in the directory.
display_summary("ROSTER", error_count) #🧩 Display a summary of the results (number of errors).
process_and_save_log(log_df, log_entries, log_ROSTER_path) #🧩 Update the Excel log file with information
logger.info("===== Processing of the ROSTER data source is complete =====") #🧩 Log finish data processing.

In [ ]:
# 0️⃣3️⃣[BKN]EPS💾
logger.info("===== Start EPS Process =====")  #🧩 Log the start of the data processing.
log_df = read_or_create_log(log_EPS_path) #🧩 Read the existing Excel log file or create a new one if it doesn't exist.
log_entries, error_count = Default_variable() #📃 Initialize a list to store log results for the current run and an error counter.
#💡 Iterate through all files in the specified folder.
for filename in Folder_EPS.glob("*.csv"): #🧩 Get files in the Folder directory.
    #📃 Get the current modification timestamp and filename of the current file.
    current_modified_date_microseconds = datetime.datetime.fromtimestamp(filename.stat().st_mtime) # Get modification timestamp, including microseconds.
    current_modified_date = current_modified_date_microseconds.replace(microsecond=0) # Remove microseconds for easier comparison.
    file_basename = filename.name # Get the base name of the file (e.g., "data_20230101.csv").
    #📃 File is being processed.
    print_colored(f"Processing file🔄️: {file_basename}", "NavajoWhite")
    logger.info(f"Processing file🔄️: {file_basename}")
    #📃 Check if this file has been logged before.
    log_entry = log_df.filter(pl.col("FileName") == file_basename) # Filter the old log_df to find an entry for this filename.
    is_new_file = log_entry.is_empty() # If no entry is found (log_entry is empty), it's a new file.
    previous_modified_date = datetime.datetime(1900, 1, 1, 0, 0, 1)  # Initialize the previous modification date with a very old date.
    #📃 If this file was processed before (exists in the log).  
    if not is_new_file:
        previous_modified_date = log_entry.select("ModifiedDate").item() # Get the modification date recorded in the log from the previous processing.
    #📃 Condition to process the file:
    # 1. The file is new (not in the log).
    # 2. Or, the file is old (in the log), but its current modification date (current_modified_date) is more recent than the modification date recorded in the log (previous_modified_date).
    if is_new_file or current_modified_date > previous_modified_date:
        if not is_new_file: # If it's an old file but has changed (ModifiedDate is newer).
            print_colored(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.", "Coral")
            logger.warning(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.")
        else: # If it's a completely new file.
            print_colored(f"New file detected🚦: {file_basename}. Importing.", "Coral")
            logger.info(f"New file detected🚦: {file_basename}. Importing.")
        try: # Start a try-except block to handle potential errors during the import process.
            if not is_new_file: # If this file already exists in the DB (because it's not new and its ModifiedDate changed) then delete the old data for this file from the df.
                delete_data(engine, EPS_TABLE_NAME, file_basename)  #🧩 Call the function to delete data.
            #📃 Read data from the excel file using Polars.
            EPS = (pl.read_csv(filename, infer_schema_length=0, encoding='latin-1') #🧩 Read the entire file to infer the schema.Add two new columns to the DataFrame: FileName and ModifiedDate.
                   .select(pl.all(),pl.lit(file_basename).alias("FileName"),pl.lit(current_modified_date).dt.cast_time_unit("ms").alias("ModifiedDate")
                   ))
            try:
                EPS = EPS.rename({'ï»¿sitecode': 'sitecode'})
            except pl.exceptions.ColumnNotFoundError:
                pass           
            EPS = EPS.with_columns(pl.col('Session Login', 'Session Logout').str.strptime(pl.Datetime, format='%m/%d/%Y %H:%M'))
            EPS = EPS.with_columns(pl.col('Session Login' #Setup Login VN📃
                                ).dt.replace_time_zone("Europe/Berlin", ambiguous="earliest"
                                ).dt.convert_time_zone("Asia/Bangkok").alias("SessionLogin_VN"))
            EPS = EPS.with_columns(pl.col('Session Logout' #Setup Logout VN📃
                                ).dt.replace_time_zone("Europe/Berlin", ambiguous="earliest"
                                ).dt.convert_time_zone("Asia/Bangkok").alias("SessionLogout_VN"))
            EPS = EPS.with_columns(pl.col('SessionLogin_VN', 'SessionLogout_VN').dt.strftime('%Y-%m-%d %H:%M:%S')) 
            EPS = EPS.to_pandas()
            time_difference(t(23,0,0),t(7,0,0))
            EPS['SessionLogin_VN'] =  pd.to_datetime(EPS['SessionLogin_VN'])
            EPS['SessionLogout_VN'] =  pd.to_datetime(EPS['SessionLogout_VN'])
            EPS['Hour Difference'] = (EPS['SessionLogout_VN'] - EPS['SessionLogin_VN']).dt.total_seconds()
            EPS['is_night_login'] = [is_time_between(t(23,0),t(7,0),i) for i in EPS['SessionLogin_VN'].dt.time]
            EPS['is_night_logout'] = [is_time_between(t(23,0),t(7,0),i) for i in EPS['SessionLogout_VN'].dt.time]
            EPS['Log In Adjusted'] =  [time_difference(t(23,0),i) if a == False and b == True else 0 for i,a,b in zip(EPS['SessionLogin_VN'].dt.time,EPS['is_night_login'],EPS['is_night_logout'])]
            EPS['Log Out Adjusted'] =  [time_difference(i,t(7,0)) if a == True and b == False else 0 for i,a,b in zip(EPS['SessionLogout_VN'].dt.time,EPS['is_night_login'],EPS['is_night_logout'])]
            EPS['NightTime'] = [i-j-k if a != False or b != False else 0 for i,j,k,a,b in zip(EPS['Hour Difference'],EPS['Log In Adjusted'],EPS['Log Out Adjusted'],EPS['is_night_login'],EPS['is_night_logout'])]
            EPS['Hour Difference'] = [time_difference(i,j) for i,j in zip(EPS['SessionLogout_VN'].dt.time,EPS['SessionLogin_VN'].dt.time)]
            EPS['is_day_login'] = [is_time_between(t(7,0),t(23,0),i) for i in EPS['SessionLogin_VN'].dt.time]
            EPS['is_day_logout'] = [is_time_between(t(7,0),t(23,0),i) for i in EPS['SessionLogout_VN'].dt.time]
            EPS['Log In Adjusted'] =  [time_difference(t(7,0),i) if a == False and b == True else 0 for i,a,b in zip(EPS['SessionLogin_VN'].dt.time,EPS['is_day_login'],EPS['is_day_logout'])]
            EPS['Log Out Adjusted'] =  [time_difference(i,t(23,0)) if a == True and b == False else 0 for i,a,b in zip(EPS['SessionLogout_VN'].dt.time,EPS['is_day_login'],EPS['is_day_logout'])]
            EPS['DayTime'] = [i-j-k if a != False or b != False else 0 for i,j,k,a,b in zip(EPS['Hour Difference'],EPS['Log In Adjusted'],EPS['Log Out Adjusted'],EPS['is_day_login'],EPS['is_day_logout'])]
            EPS = pl.from_pandas(EPS)
            EPS = EPS.with_columns(pl.col("Total Time").cast(pl.Int64))
            EPS = EPS.with_columns(pl.when(pl.col("Total Time") - pl.col("NightTime") < 0).then(pl.col("Total Time")).otherwise(pl.col("NightTime")).alias("Night_BPE"))
            EPS = EPS.with_columns(pl.when(pl.col("Total Time") - pl.col("NightTime") < 0).then(0).otherwise(pl.col("Total Time") - pl.col("Night_BPE")).alias("Day_BPE"))
            #📃 Validate if the columns in the DataFrame match the defined schema
            has_critical_error, critical_schema_error_msg = validate_schema(EPS, EPS_schema, file_basename) #🧩
            if has_critical_error: # If critical columns are missing.
                raise ValueError(critical_schema_error_msg) # Raise an error to stop processing this file.
            #📃 Adjust the column structure.
            print_colored(f"⚙️Adjust column structure", "Olive")
            logger.warning(f"⚙️Adjust column structure")    
            EPS = (EPS.select(EPS_schema)) #🧩 Select columns in the correct order and names as per df_schema.
            print_colored(f"⚙️Column structure adjustment completed", "Olive")
            logger.info(f"⚙️Column structure adjustment completed")
            info_polars(EPS) #🧩 Display information about the DataFrame (shape, columns, dtypes).
            #📃 Start writing data to the database.
            print_colored(f"💾Start import file: {file_basename}", "Coral")
            logger.info(f"💾Start import file: {file_basename}")
            write_data(engine, EPS_TABLE_NAME, EPS) #🧩 Call the function to write the DataFrame to SQL.
            print_colored(f"Successfully imported✨: {file_basename}", "LimeGreen")
            logger.info(f"Successfully imported✨: {file_basename}")
            #📃 Record to summary
            if is_new_file:
                summary_results["new_files_imported"].append(f"[EPS] {file_basename}")#🧩
            else:
                summary_results["updated_files_reimported"].append(f"[EPS] {file_basename}")#🧩
            #📃 Update log_entries for successfully import: Add information to the log list for this run (successful).
            log_entries.append({"FileName": file_basename, "ModifiedDate": current_modified_date, "Error": None})
            print_colored(f"Updated 'Modified' time for {file_basename}", "LimeGreen")
            logger.info(f"Updated 'Modified' time for {file_basename}")
            #📃 Handle exceptions if any error occurred in the try block.
        except (
            FileNotFoundError, PermissionError, UnicodeDecodeError, IOError,                
            pl.exceptions.NoDataError, pl.exceptions.ComputeError, pl.exceptions.SchemaError, sa.exc.SQLAlchemyError,
            Exception # Catch various types of potential errors.
        ) as e:
            error_count += 1 # Increment the error counter.        
            error_msg_short = f"{type(e).__name__}: {str(e).splitlines()[0]}"  # Get a short error message (the first line).
            #📃 Update Failed file to Summary
            summary_results["failed_imports"].append((f"[EPS] {file_basename}", error_msg_short))#🧩
            logger.error(f"Error importing {file_basename}: {error_msg_short}", exc_info=True) # Log the full error traceback.
            print_colored(f"❗️Error importing {file_basename}: {error_msg_short}", "OrangeRed")
            # Add information to the log list for this run (failed).
            # Note: ModifiedDate here is previous_modified_date, as this file wasn't successfully imported with current_modified_date.
            log_entries.append({"FileName": file_basename,"ModifiedDate": previous_modified_date,"Error": error_msg_short})
    else: #📃 If the file is not new and its modification date hasn't changed.
          #📃 Update Skip file to Summary
        summary_results["skipped_files_unchanged"].append(f"[EPS] {file_basename}")#🧩
        print_colored(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.", "Aquamarine")
        logger.info(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.")
#💡 After iterating through all files in the directory.
display_summary("EPS", error_count) #🧩 Display a summary of the results (number of errors).
process_and_save_log(log_df, log_entries, log_EPS_path) #🧩 Update the Excel log file with information
logger.info("===== Processing of the EPS data source is complete =====") #🧩 Log finish data processing.

In [ ]:
# 0️⃣4️⃣[BKN]CPI💾
logger.info("===== Start CPI Process =====")  #🧩 Log the start of the data processing.
log_df = read_or_create_log(log_CPI_path) #🧩 Read the existing Excel log file or create a new one if it doesn't exist.
log_entries, error_count = Default_variable() #📃 Initialize a list to store log results for the current run and an error counter.
#💡 Iterate through all files in the specified folder.
for filename in Folder_CPI.glob("*.csv"): #🧩 Get files in the Folder directory.
    #📃 Get the current modification timestamp and filename of the current file.
    current_modified_date_microseconds = datetime.datetime.fromtimestamp(filename.stat().st_mtime) # Get modification timestamp, including microseconds.
    current_modified_date = current_modified_date_microseconds.replace(microsecond=0) # Remove microseconds for easier comparison.
    file_basename = filename.name # Get the base name of the file (e.g., "data_20230101.csv").
    #📃 File is being processed.
    print_colored(f"Processing file🔄️: {file_basename}", "NavajoWhite")
    logger.info(f"Processing file🔄️: {file_basename}")
    #📃 Check if this file has been logged before.
    log_entry = log_df.filter(pl.col("FileName") == file_basename) # Filter the old log_df to find an entry for this filename.
    is_new_file = log_entry.is_empty() # If no entry is found (log_entry is empty), it's a new file.
    previous_modified_date = datetime.datetime(1900, 1, 1, 0, 0, 1)  # Initialize the previous modification date with a very old date.
    #📃 If this file was processed before (exists in the log).  
    if not is_new_file:
        previous_modified_date = log_entry.select("ModifiedDate").item() # Get the modification date recorded in the log from the previous processing.
    #📃 Condition to process the file:
    # 1. The file is new (not in the log).
    # 2. Or, the file is old (in the log), but its current modification date (current_modified_date) is more recent than the modification date recorded in the log (previous_modified_date).
    if is_new_file or current_modified_date > previous_modified_date:
        if not is_new_file: # If it's an old file but has changed (ModifiedDate is newer).
            print_colored(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.", "Coral")
            logger.warning(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.")
        else: # If it's a completely new file.
            print_colored(f"New file detected🚦: {file_basename}. Importing.", "Coral")
            logger.info(f"New file detected🚦: {file_basename}. Importing.")
        try: # Start a try-except block to handle potential errors during the import process.
            if not is_new_file: # If this file already exists in the DB (because it's not new and its ModifiedDate changed) then delete the old data for this file from the df.
                delete_data(engine, CPI_TABLE_NAME, file_basename)  #🧩 Call the function to delete data.
            #📃 Read data from the excel file using Polars.
            CPI = (pl.read_csv(filename, infer_schema_length=None) #🧩 Read the entire file to infer the schema.Add two new columns to the DataFrame: FileName and ModifiedDate.
                   .select(pl.all(),pl.lit(file_basename).alias("FileName"),pl.lit(current_modified_date).dt.cast_time_unit("ms").alias("ModifiedDate")
                   ))
            try:
                CPI = CPI.with_columns(pl.col('Nr. Contacts').cast(pl.Int64))
            except:
                CPI = CPI.with_columns(pl.col("Nr. Contacts").str.replace_all(r"[()]", "").cast(pl.Int64).alias("Nr. Contacts"))
            CPI = CPI.with_columns(
                pl.when(pl.col("Nr. Contacts").is_null()).then(1).otherwise(pl.col("Nr. Contacts").abs()).alias("Nr. Contacts"))
            # --- "Date" processing ---
            if "Date" in CPI.columns:
                logger.info(f"'Date' column found in {file_basename}. Current dtype: {CPI['Date'].dtype}")
                print_colored(f"'Date' column found in {file_basename}. Current dtype: {CPI['Date'].dtype}", "Olive")
                # Parse if "Date" in Utf8 or Object
                if CPI["Date"].dtype in [pl.Utf8, pl.Object]:
                    try:
                        logger.info(f"Attempting to parse 'Date' column from string in {file_basename}.")
                        print_colored(f"Attempting to parse 'Date' column from string in {file_basename}.", "Olive")
                        CPI = CPI.with_columns(parse_date(pl.col("Date")).alias("Date"),)
                        logger.info(f"✅ Parse 'Date' successfully in {file_basename}.")
                        print_colored(f"✅ Parse 'Date' successfully in {file_basename}.", "MediumSeaGreen")
                    except Exception as parse_err:
                        logger.error(f"Could not parse string 'Date' column in {file_basename}: {parse_err}", exc_info=True)
                        print_colored(f"❗️ Failed to parse 'Date' column in {file_basename}. It will remain as string/object or become null.", "OrangeRed")
                        CPI = CPI.with_columns(pl.lit(None).cast(pl.Date).alias("Date")) # add Date null
                elif CPI["Date"].dtype not in [pl.Date, pl.Datetime]:
                    # if "Date" not in Utf8,Object or Date/Datetime (Ex: Int)
                    logger.warning(f"Existing 'Date' column in {file_basename} has unexpected type: {CPI['Date'].dtype}. Attempting direct cast to Date.")
                    print_colored(f"Existing 'Date' column in {file_basename} has unexpected type: {CPI['Date'].dtype}. Attempting direct cast to Date.", "OrangeRed")
                    try:
                        CPI = CPI.with_columns(pl.col("Date").cast(pl.Date, strict=False))
                        logger.info(f"✅ Cast 'Date' successfully in {file_basename}.")
                        print_colored(f"✅ Carse 'Date' successfully in {file_basename}.", "MediumSeaGreen")
                    except Exception as cast_err:
                        logger.error(f"Could not cast 'Date' column of type {CPI['Date'].dtype} to Date in {file_basename}: {cast_err}")
                        print_colored(f"Could not cast 'Date' column of type {CPI['Date'].dtype} to Date in {file_basename}: {cast_err}", "OrangeRed")
                        CPI = CPI.with_columns(pl.lit(None).cast(pl.Date).alias("Date")) # add Date null
                else:
                    # if "Date" in Date/Datetime
                    logger.info(f"'Date' column in {file_basename} is already Date/Datetime. Ensuring pl.Date type.")
                    print_colored(f"'Date' column in {file_basename} is already Date/Datetime. Ensuring pl.Date type.", "Olive")
                    CPI = CPI.with_columns(pl.col("Date").cast(pl.Date, strict=False))
                    logger.info(f"✅ Cast 'Date' successfully in {file_basename}.")
                    print_colored(f"✅ Carse 'Date' successfully in {file_basename}.", "MediumSeaGreen")
            else:
                # If "Date" not exist, extract from file name
                logger.warning(f"'Date' column not found in {file_basename}. Attempting to extract from filename.")
                print_colored(f"'Date' column not found in {file_basename}. Attempting to extract from filename.", "Olive")
                try:
                    date_from_filename = (
                         pl.lit(file_basename)
                        .str.replace(r"\.csv$", "", literal=False) # remove .csv
                        .str.extract(r"(\d{8})", 1) # extract YYYYMMDD
                        .str.strptime(pl.Date, "%Y%m%d", strict=False) # Parse YYYYMMDD
                     )
                    CPI = CPI.with_columns(date_from_filename.alias("Date"))
                    logger.info(f"Successfully extracted 'Date' from filename {file_basename}.")
                    print_colored(f"Successfully extracted 'Date' from filename {file_basename}.", "Olive")
                except Exception as extract_err:
                    logger.error(f"Could not extract date from filename {file_basename}: {extract_err}. Adding null 'Date' column.", exc_info=True)
                    print_colored(f"Could not extract date from filename {file_basename}: {extract_err}. Adding null 'Date' column.", "OrangeRed")
                    CPI = CPI.with_columns(pl.lit(None).cast(pl.Date).alias("Date")) # add Date null           
            CPI = CPI.rename({'Time ': 'Time'}) #🧩
            #📃 Validate if the columns in the DataFrame match the defined schema
            has_critical_error, critical_schema_error_msg = validate_schema(CPI, CPI_schema, file_basename) #🧩
            if has_critical_error: # If critical columns are missing.
                raise ValueError(critical_schema_error_msg) # Raise an error to stop processing this file.
            #📃 Adjust the column structure.
            print_colored(f"⚙️Adjust column structure", "Olive")
            logger.warning(f"⚙️Adjust column structure")
            CPI = CPI.select(CPI_schema) #🧩 Select columns in the correct order and names as per df_schema.
            print_colored(f"⚙️Column structure adjustment completed", "Olive")
            logger.info(f"⚙️Column structure adjustment completed")
            info_polars(CPI) #🧩 Display information about the DataFrame (shape, columns, dtypes).
            #📃 Start writing data to the database.
            print_colored(f"💾Start import file: {file_basename}", "Coral")
            logger.info(f"💾Start import file: {file_basename}")
            write_data(engine, CPI_TABLE_NAME, CPI) #🧩 Call the function to write the DataFrame to SQL.
            print_colored(f"Successfully imported✨: {file_basename}", "LimeGreen")
            logger.info(f"Successfully imported✨: {file_basename}")
            #📃 Record to summary
            if is_new_file:
                summary_results["new_files_imported"].append(f"[CPI] {file_basename}")#🧩
            else:
                summary_results["updated_files_reimported"].append(f"[CPI] {file_basename}")#🧩
            #📃 Update log_entries for successfully import: Add information to the log list for this run (successful).
            log_entries.append({"FileName": file_basename, "ModifiedDate": current_modified_date, "Error": None})
            print_colored(f"Updated 'Modified' time for {file_basename}", "LimeGreen")
            logger.info(f"Updated 'Modified' time for {file_basename}")
            #📃 Handle exceptions if any error occurred in the try block. 
        except (
            FileNotFoundError, PermissionError, UnicodeDecodeError, IOError,                
            pl.exceptions.NoDataError, pl.exceptions.ComputeError, pl.exceptions.SchemaError, sa.exc.SQLAlchemyError,
            Exception # Catch various types of potential errors.
        ) as e:
            error_count += 1 # Increment the error counter.        
            error_msg_short = f"{type(e).__name__}: {str(e).splitlines()[0]}"  # Get a short error message (the first line).

            #📃 Update Failed file to Summary
            summary_results["failed_imports"].append((f"[CPI] {file_basename}", error_msg_short))#🧩       
            logger.error(f"Error importing {file_basename}: {error_msg_short}", exc_info=True) # Log the full error traceback.
            print_colored(f"❗️Error importing {file_basename}: {error_msg_short}", "OrangeRed")
            # Add information to the log list for this run (failed).
            # Note: ModifiedDate here is previous_modified_date, as this file wasn't successfully imported with current_modified_date.
            log_entries.append({"FileName": file_basename,"ModifiedDate": previous_modified_date,"Error": error_msg_short})
    else: #📃 If the file is not new and its modification date hasn't changed.
          #📃 Update Skip file to Summary
        summary_results["skipped_files_unchanged"].append(f"[CPI] {file_basename}")#🧩
        print_colored(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.", "Aquamarine")
        logger.info(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.")
#💡 After iterating through all files in the directory.
display_summary("CPI", error_count) #🧩 Display a summary of the results (number of errors).
process_and_save_log(log_df, log_entries, log_CPI_path) #🧩 Update the Excel log file with information
logger.info("===== Processing of the CPI data source is complete =====") #🧩 Log finish data processing.

In [ ]:
# 0️⃣5️⃣[GLB]RAMCO💾
logger.info("===== Start RAMCO Process =====")  #🧩 Log the start of the data processing.
log_df = read_or_create_log(log_RAMCO_path) #🧩 Read the existing Excel log file or create a new one if it doesn't exist.
log_entries, error_count = Default_variable() #📃 Initialize a list to store log results for the current run and an error counter.
#💡 Iterate through all files in the specified folder.
for filename in Folder_RAMCO.glob("*.csv"): #🧩 Get files in the Folder directory.
    #📃 Get the current modification timestamp and filename of the current file.
    current_modified_date_microseconds = datetime.datetime.fromtimestamp(filename.stat().st_mtime) # Get modification timestamp, including microseconds.
    current_modified_date = current_modified_date_microseconds.replace(microsecond=0) # Remove microseconds for easier comparison.
    file_basename = filename.name # Get the base name of the file (e.g., "data_20230101.csv").
    #📃 File is being processed.
    print_colored(f"Processing file🔄️: {file_basename}", "NavajoWhite")
    logger.info(f"Processing file🔄️: {file_basename}")
    #📃 Check if this file has been logged before.
    log_entry = log_df.filter(pl.col("FileName") == file_basename) # Filter the old log_df to find an entry for this filename.
    is_new_file = log_entry.is_empty() # If no entry is found (log_entry is empty), it's a new file.
    previous_modified_date = datetime.datetime(1900, 1, 1, 0, 0, 1)  # Initialize the previous modification date with a very old date.
    #📃 If this file was processed before (exists in the log).  
    if not is_new_file:
        previous_modified_date = log_entry.select("ModifiedDate").item() # Get the modification date recorded in the log from the previous processing.
    #📃 Condition to process the file:
    # 1. The file is new (not in the log).
    # 2. Or, the file is old (in the log), but its current modification date (current_modified_date) is more recent than the modification date recorded in the log (previous_modified_date).
    if is_new_file or current_modified_date > previous_modified_date:
        if not is_new_file: # If it's an old file but has changed (ModifiedDate is newer).
            print_colored(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.", "Coral")
            logger.warning(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.")
        else: # If it's a completely new file.
            print_colored(f"New file detected🚦: {file_basename}. Importing.", "Coral")
            logger.info(f"New file detected🚦: {file_basename}. Importing.")
        try: # Start a try-except block to handle potential errors during the import process.
            if not is_new_file: # If this file already exists in the DB (because it's not new and its ModifiedDate changed) then delete the old data for this file from the df.
                delete_data(engine, RAMCO_TABLE_NAME, file_basename)  #🧩 Call the function to delete data.
            #📃 Read data from the excel file using Polars.
            RAMCO = (pl.read_csv(filename, infer_schema_length=0, encoding='latin-1') #🧩 Read the entire file to infer the schema.Add two new columns to the DataFrame: FileName and ModifiedDate.
                       .select(pl.all(),pl.lit(file_basename).alias("FileName"),pl.lit(current_modified_date).dt.cast_time_unit("ms").alias("ModifiedDate")))
            RAMCO = RAMCO.with_columns(
                pl.col('Attribute').str.strptime(pl.Date, format='%m/%d/%Y'),
                pl.col('EID').cast(pl.Int64)).rename({'Attribute': 'Date', 'Value': 'Code'}) #🧩
            #📃 Validate if the columns in the DataFrame match the defined schema
            has_critical_error, critical_schema_error_msg = validate_schema(RAMCO, RAMCO_schema, file_basename) #🧩
            if has_critical_error: # If critical columns are missing.
                raise ValueError(critical_schema_error_msg) # Raise an error to stop processing this file.
            #📃 Adjust the column structure.
            print_colored(f"⚙️Adjust column structure", "Olive")
            logger.warning(f"⚙️Adjust column structure")
            RAMCO = RAMCO.select(RAMCO_schema) #🧩 Select columns in the correct order and names as per df_schema.
            print_colored(f"⚙️Column structure adjustment completed", "Olive")
            logger.info(f"⚙️Column structure adjustment completed")
            info_polars(RAMCO) #🧩 Display information about the DataFrame (shape, columns, dtypes).
            #📃 Start writing data to the database.
            print_colored(f"💾Start import file: {file_basename}", "Coral")
            logger.info(f"💾Start import file: {file_basename}")
            write_data(engine, RAMCO_TABLE_NAME, RAMCO) #🧩 Call the function to write the DataFrame to SQL.
            print_colored(f"Successfully imported✨: {file_basename}", "LimeGreen")
            logger.info(f"Successfully imported✨: {file_basename}")
            #📃 Record to summary
            if is_new_file:
                summary_results["new_files_imported"].append(f"[RAMCO] {file_basename}")#🧩
            else:
                summary_results["updated_files_reimported"].append(f"[RAMCO] {file_basename}")#🧩
            #📃 Update log_entries for successfully import: Add information to the log list for this run (successful).
            log_entries.append({"FileName": file_basename, "ModifiedDate": current_modified_date, "Error": None})
            print_colored(f"Updated 'Modified' time for {file_basename}", "LimeGreen")
            logger.info(f"Updated 'Modified' time for {file_basename}")
            #📃 Handle exceptions if any error occurred in the try block.
        except (
            FileNotFoundError, PermissionError, UnicodeDecodeError, IOError,                
            pl.exceptions.NoDataError, pl.exceptions.ComputeError, pl.exceptions.SchemaError, sa.exc.SQLAlchemyError,
            Exception # Catch various types of potential errors.
        ) as e:
            error_count += 1 # Increment the error counter.        
            error_msg_short = f"{type(e).__name__}: {str(e).splitlines()[0]}"  # Get a short error message (the first line).
            #📃 Update Failed file to Summary
            summary_results["failed_imports"].append((f"[RAMCO] {file_basename}", error_msg_short))#🧩  
            logger.error(f"Error importing {file_basename}: {error_msg_short}", exc_info=True) # Log the full error traceback.
            print_colored(f"❗️Error importing {file_basename}: {error_msg_short}", "OrangeRed")
            # Add information to the log list for this run (failed).
            # Note: ModifiedDate here is previous_modified_date, as this file wasn't successfully imported with current_modified_date.
            log_entries.append({"FileName": file_basename,"ModifiedDate": previous_modified_date,"Error": error_msg_short})
    else: #📃 If the file is not new and its modification date hasn't changed.
          #📃 Update Skip file to Summary
        summary_results["skipped_files_unchanged"].append(f"[RAMCO] {file_basename}")#🧩  
        print_colored(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.", "Aquamarine")
        logger.info(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.")
#💡 After iterating through all files in the directory.
display_summary("RAMCO", error_count) #🧩 Display a summary of the results (number of errors).
process_and_save_log(log_df, log_entries, log_RAMCO_path) #🧩 Update the Excel log file with information
logger.info("===== Processing of the RAMCO data source is complete =====") #🧩 Log finish data processing.

In [ ]:
# 0️⃣6️⃣[GLB]OT_RAMCO💾
logger.info("===== Start OT_ROSTER Process =====")  #🧩 Log the start of the data processing.
log_df = read_or_create_log(log_OT_RAMCO_path) #🧩 Read the existing Excel log file or create a new one if it doesn't exist.
log_entries, error_count = Default_variable() #📃 Initialize a list to store log results for the current run and an error counter.
#💡 Iterate through all files in the specified folder.
for filename in Folder_OT_RAMCO.glob("*.xlsx"): #🧩 Get files in the Folder directory.
    #📃 Get the current modification timestamp and filename of the current file.
    current_modified_date_microseconds = datetime.datetime.fromtimestamp(filename.stat().st_mtime) # Get modification timestamp, including microseconds.
    current_modified_date = current_modified_date_microseconds.replace(microsecond=0) # Remove microseconds for easier comparison.
    file_basename = filename.name # Get the base name of the file (e.g., "data_20230101.csv").
    #📃 File is being processed.
    print_colored(f"Processing file🔄️: {file_basename}", "NavajoWhite")
    logger.info(f"Processing file🔄️: {file_basename}")
    #📃 Check if this file has been logged before.
    log_entry = log_df.filter(pl.col("FileName") == file_basename) # Filter the old log_df to find an entry for this filename.
    is_new_file = log_entry.is_empty() # If no entry is found (log_entry is empty), it's a new file.
    previous_modified_date = datetime.datetime(1900, 1, 1, 0, 0, 1)  # Initialize the previous modification date with a very old date.
    #📃 If this file was processed before (exists in the log).  
    if not is_new_file:
        previous_modified_date = log_entry.select("ModifiedDate").item() # Get the modification date recorded in the log from the previous processing.
    #📃 Condition to process the file:
    # 1. The file is new (not in the log).
    # 2. Or, the file is old (in the log), but its current modification date (current_modified_date) is more recent than the modification date recorded in the log (previous_modified_date).
    if is_new_file or current_modified_date > previous_modified_date:
        if not is_new_file: # If it's an old file but has changed (ModifiedDate is newer).
            print_colored(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.", "Coral")
            logger.warning(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.")
        else: # If it's a completely new file.
            print_colored(f"New file detected🚦: {file_basename}. Importing.", "Coral")
            logger.info(f"New file detected🚦: {file_basename}. Importing.")
        try: # Start a try-except block to handle potential errors during the import process.
            if not is_new_file: # If this file already exists in the DB (because it's not new and its ModifiedDate changed) then delete the old data for this file from the df.
                delete_data(engine, OT_RAMCO_TABLE_NAME, file_basename)  #🧩 Call the function to delete data.
            #📃 Read data from the excel file using Polars.
            OT_RAMCO = (pl.read_excel(filename, infer_schema_length=None, engine="calamine", sheet_name="Sheet1") #🧩 Read the entire file to infer the schema.Add two new columns to the DataFrame: FileName and ModifiedDate.
                   .select(pl.all(),pl.lit(file_basename).alias("FileName"),pl.lit(current_modified_date).dt.cast_time_unit("ms").alias("ModifiedDate")
                   ).rename({'Attribute': 'Date'})) #🧩
            #📃 Validate if the columns in the DataFrame match the defined schema
            has_critical_error, critical_schema_error_msg = validate_schema(OT_RAMCO, OT_RAMCO_schema, file_basename) #🧩
            if has_critical_error: # If critical columns are missing.
                raise ValueError(critical_schema_error_msg) # Raise an error to stop processing this file.
            #📃 Adjust the column structure.
            print_colored(f"⚙️Adjust column structure", "Olive")
            logger.warning(f"⚙️Adjust column structure")
            OT_RAMCO = OT_RAMCO.select(OT_RAMCO_schema) #🧩 Select columns in the correct order and names as per df_schema.
            print_colored(f"⚙️Column structure adjustment completed", "Olive")
            logger.info(f"⚙️Column structure adjustment completed")
            info_polars(OT_RAMCO) #🧩 Display information about the DataFrame (shape, columns, dtypes).
            #📃 Start writing data to the database.
            print_colored(f"💾Start import file: {file_basename}", "Coral")
            logger.info(f"💾Start import file: {file_basename}")
            write_data(engine, OT_RAMCO_TABLE_NAME, OT_RAMCO) #🧩 Call the function to write the DataFrame to SQL.
            print_colored(f"Successfully imported✨: {file_basename}", "LimeGreen")
            logger.info(f"Successfully imported✨: {file_basename}")
            #📃 Record to summary
            if is_new_file:
                summary_results["new_files_imported"].append(f"[OT_RAMCO] {file_basename}")#🧩
            else:
                summary_results["updated_files_reimported"].append(f"[OT_RAMCO] {file_basename}")#🧩   
            #📃 Update log_entries for successfully import: Add information to the log list for this run (successful).
            log_entries.append({"FileName": file_basename, "ModifiedDate": current_modified_date, "Error": None})
            print_colored(f"Updated 'Modified' time for {file_basename}", "LimeGreen")
            logger.info(f"Updated 'Modified' time for {file_basename}")
            #📃 Handle exceptions if any error occurred in the try block.
        except (
            FileNotFoundError, PermissionError, UnicodeDecodeError, IOError,                
            pl.exceptions.NoDataError, pl.exceptions.ComputeError, pl.exceptions.SchemaError, sa.exc.SQLAlchemyError,
            Exception # Catch various types of potential errors.
        ) as e:
            error_count += 1 # Increment the error counter.        
            error_msg_short = f"{type(e).__name__}: {str(e).splitlines()[0]}"  # Get a short error message (the first line).
            #📃 Update Failed file to Summary
            summary_results["failed_imports"].append((f"[OT_RAMCO] {file_basename}", error_msg_short))#🧩
            logger.error(f"Error importing {file_basename}: {error_msg_short}", exc_info=True) # Log the full error traceback.
            print_colored(f"❗️Error importing {file_basename}: {error_msg_short}", "OrangeRed")
            # Add information to the log list for this run (failed).
            # Note: ModifiedDate here is previous_modified_date, as this file wasn't successfully imported with current_modified_date.
            log_entries.append({"FileName": file_basename,"ModifiedDate": previous_modified_date,"Error": error_msg_short})
    else: #📃 If the file is not new and its modification date hasn't changed.
          #📃 Update Skip file to Summary
        summary_results["skipped_files_unchanged"].append(f"[OT_RAMCO] {file_basename}")#🧩
        print_colored(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.", "Aquamarine")
        logger.info(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.")
#💡 After iterating through all files in the directory.
display_summary("OT_RAMCO", error_count) #🧩 Display a summary of the results (number of errors).
process_and_save_log(log_df, log_entries, log_OT_RAMCO_path) #🧩 Update the Excel log file with information
logger.info("===== Processing of the OT_RAMCO data source is complete =====") #🧩 Log finish data processing.

In [ ]:
# 0️⃣7️⃣[GLB]PremHdays💾
logger.info("===== Start PremHdays Process =====")  #🧩 Log the start of the data processing.
log_df = read_or_create_log(log_PremHdays_path) #🧩 Read the existing Excel log file or create a new one if it doesn't exist.
log_entries, error_count = Default_variable() #📃 Initialize a list to store log results for the current run and an error counter.
#💡 Iterate through all files in the specified folder.
for filename in Folder_PremHdays.glob("*.csv"): #🧩 Get files in the Folder directory.
    #📃 Get the current modification timestamp and filename of the current file.
    current_modified_date_microseconds = datetime.datetime.fromtimestamp(filename.stat().st_mtime) # Get modification timestamp, including microseconds.
    current_modified_date = current_modified_date_microseconds.replace(microsecond=0) # Remove microseconds for easier comparison.
    file_basename = filename.name # Get the base name of the file (e.g., "data_20230101.csv").
    #📃 File is being processed.
    print_colored(f"Processing file🔄️: {file_basename}", "NavajoWhite")
    logger.info(f"Processing file🔄️: {file_basename}")
    #📃 Check if this file has been logged before.
    log_entry = log_df.filter(pl.col("FileName") == file_basename) # Filter the old log_df to find an entry for this filename.
    is_new_file = log_entry.is_empty() # If no entry is found (log_entry is empty), it's a new file.
    previous_modified_date = datetime.datetime(1900, 1, 1, 0, 0, 1)  # Initialize the previous modification date with a very old date.
    #📃 If this file was processed before (exists in the log).  
    if not is_new_file:
        previous_modified_date = log_entry.select("ModifiedDate").item() # Get the modification date recorded in the log from the previous processing.
    #📃 Condition to process the file:
    # 1. The file is new (not in the log).
    # 2. Or, the file is old (in the log), but its current modification date (current_modified_date) is more recent than the modification date recorded in the log (previous_modified_date).
    if is_new_file or current_modified_date > previous_modified_date:
        if not is_new_file: # If it's an old file but has changed (ModifiedDate is newer).
            print_colored(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.", "Coral")
            logger.warning(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.")
        else: # If it's a completely new file.
            print_colored(f"New file detected🚦: {file_basename}. Importing.", "Coral")
            logger.info(f"New file detected🚦: {file_basename}. Importing.")
        try: # Start a try-except block to handle potential errors during the import process.
            if not is_new_file: # If this file already exists in the DB (because it's not new and its ModifiedDate changed) then delete the old data for this file from the df.
                delete_data(engine, PremHdays_TABLE_NAME, file_basename)  #🧩 Call the function to delete data.
            #📃 Read data from the excel file using Polars.
            PremHdays = (pl.read_csv(filename, infer_schema_length=None) #🧩 Read the entire file to infer the schema.Add two new columns to the DataFrame: FileName and ModifiedDate.
                    .select(pl.all(),pl.lit(file_basename).alias("FileName"),pl.lit(current_modified_date).dt.cast_time_unit("ms").alias("ModifiedDate")
                   )) # Import Schema🧩
            #📃 Validate if the columns in the DataFrame match the defined schema
            has_critical_error, critical_schema_error_msg = validate_schema(PremHdays, PremHdays_schema, file_basename) #🧩
            if has_critical_error: # If critical columns are missing.
                raise ValueError(critical_schema_error_msg) # Raise an error to stop processing this file.
            #📃 Adjust the column structure.
            print_colored(f"⚙️Adjust column structure", "Olive")
            logger.warning(f"⚙️Adjust column structure")
            PremHdays = PremHdays.with_columns(pl.col('Date').str.strptime(pl.Date, format='%m/%d/%Y')) #🧩
            PremHdays = PremHdays.select(PremHdays_schema) #🧩 Select columns in the correct order and names as per df_schema.
            print_colored(f"⚙️Column structure adjustment completed", "Olive")
            logger.info(f"⚙️Column structure adjustment completed")
            info_polars(PremHdays) #🧩 Display information about the DataFrame (shape, columns, dtypes).
            #📃 Start writing data to the database.
            print_colored(f"💾Start import file: {file_basename}", "Coral")
            logger.info(f"💾Start import file: {file_basename}")
            write_data(engine, PremHdays_TABLE_NAME, PremHdays) #🧩 Call the function to write the DataFrame to SQL.
            print_colored(f"Successfully imported✨: {file_basename}", "LimeGreen")
            logger.info(f"Successfully imported✨: {file_basename}")
            #📃 Record to summary
            if is_new_file:
                summary_results["new_files_imported"].append(f"[PremHdays] {file_basename}")#🧩
            else:
                summary_results["updated_files_reimported"].append(f"[PremHdays] {file_basename}")#🧩       
            #📃 Update log_entries for successfully import: Add information to the log list for this run (successful).
            log_entries.append({"FileName": file_basename, "ModifiedDate": current_modified_date, "Error": None})
            print_colored(f"Updated 'Modified' time for {file_basename}", "LimeGreen")
            logger.info(f"Updated 'Modified' time for {file_basename}")
            #📃 Handle exceptions if any error occurred in the try block.
        except (
            FileNotFoundError, PermissionError, UnicodeDecodeError, IOError,                
            pl.exceptions.NoDataError, pl.exceptions.ComputeError, pl.exceptions.SchemaError, sa.exc.SQLAlchemyError,
            Exception # Catch various types of potential errors.
        ) as e:
            error_count += 1 # Increment the error counter.        
            error_msg_short = f"{type(e).__name__}: {str(e).splitlines()[0]}"  # Get a short error message (the first line).
            #📃 Update Failed file to Summary
            summary_results["failed_imports"].append((f"[PremHdays] {file_basename}", error_msg_short))#🧩       
            logger.error(f"Error importing {file_basename}: {error_msg_short}", exc_info=True) # Log the full error traceback.
            print_colored(f"❗️Error importing {file_basename}: {error_msg_short}", "OrangeRed")
            # Add information to the log list for this run (failed).
            # Note: ModifiedDate here is previous_modified_date, as this file wasn't successfully imported with current_modified_date.
            log_entries.append({"FileName": file_basename,"ModifiedDate": previous_modified_date,"Error": error_msg_short})
    else: #📃 If the file is not new and its modification date hasn't changed.
          #📃 Update Skip file to Summary
        summary_results["skipped_files_unchanged"].append(f"[PremHdays] {file_basename}")#🧩
        print_colored(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.", "Aquamarine")
        logger.info(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.")
#💡 After iterating through all files in the directory.
display_summary("PremHdays", error_count) #🧩 Display a summary of the results (number of errors).
process_and_save_log(log_df, log_entries, log_PremHdays_path) #🧩 Update the Excel log file with information
logger.info("===== Processing of the PremHdays data source is complete =====") #🧩 Log finish data processing.

In [ ]:
# 0️⃣8️⃣[GLB]NormHdays💾
logger.info("===== Start NormHdays Process =====")  #🧩 Log the start of the data processing.
log_df = read_or_create_log(log_NormHdays_path) #🧩 Read the existing Excel log file or create a new one if it doesn't exist.
log_entries, error_count = Default_variable() #📃 Initialize a list to store log results for the current run and an error counter.
#💡 Iterate through all files in the specified folder.
for filename in Folder_NormHdays.glob("*.csv"): #🧩 Get files in the Folder directory.
    #📃 Get the current modification timestamp and filename of the current file.
    current_modified_date_microseconds = datetime.datetime.fromtimestamp(filename.stat().st_mtime) # Get modification timestamp, including microseconds.
    current_modified_date = current_modified_date_microseconds.replace(microsecond=0) # Remove microseconds for easier comparison.
    file_basename = filename.name # Get the base name of the file (e.g., "data_20230101.csv").
    #📃 File is being processed.
    print_colored(f"Processing file🔄️: {file_basename}", "NavajoWhite")
    logger.info(f"Processing file🔄️: {file_basename}")
    #📃 Check if this file has been logged before.
    log_entry = log_df.filter(pl.col("FileName") == file_basename) # Filter the old log_df to find an entry for this filename.
    is_new_file = log_entry.is_empty() # If no entry is found (log_entry is empty), it's a new file.
    previous_modified_date = datetime.datetime(1900, 1, 1, 0, 0, 1)  # Initialize the previous modification date with a very old date.
    #📃 If this file was processed before (exists in the log).  
    if not is_new_file:
        previous_modified_date = log_entry.select("ModifiedDate").item() # Get the modification date recorded in the log from the previous processing.
    #📃 Condition to process the file:
    # 1. The file is new (not in the log).
    # 2. Or, the file is old (in the log), but its current modification date (current_modified_date) is more recent than the modification date recorded in the log (previous_modified_date).
    if is_new_file or current_modified_date > previous_modified_date:
        if not is_new_file: # If it's an old file but has changed (ModifiedDate is newer).
            print_colored(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.", "Coral")
            logger.warning(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.")
        else: # If it's a completely new file.
            print_colored(f"New file detected🚦: {file_basename}. Importing.", "Coral")
            logger.info(f"New file detected🚦: {file_basename}. Importing.")
        try: # Start a try-except block to handle potential errors during the import process.
            if not is_new_file: # If this file already exists in the DB (because it's not new and its ModifiedDate changed) then delete the old data for this file from the df.
                delete_data(engine, NormHdays_TABLE_NAME, file_basename)  #🧩 Call the function to delete data.
            #📃 Read data from the excel file using Polars.
            NormHdays = (pl.read_csv(filename, infer_schema_length=None) #🧩 Read the entire file to infer the schema.Add two new columns to the DataFrame: FileName and ModifiedDate.
                    .select(pl.all(),pl.lit(file_basename).alias("FileName"),pl.lit(current_modified_date).dt.cast_time_unit("ms").alias("ModifiedDate")
                   )) #🧩
            #📃 Validate if the columns in the DataFrame match the defined schema
            has_critical_error, critical_schema_error_msg = validate_schema(NormHdays, NormHdays_schema, file_basename) #🧩
            if has_critical_error: # If critical columns are missing.
                raise ValueError(critical_schema_error_msg) # Raise an error to stop processing this file.
            #📃 Adjust the column structure.
            print_colored(f"⚙️Adjust column structure", "Olive")
            logger.warning(f"⚙️Adjust column structure")
            NormHdays = NormHdays.with_columns(pl.col('Solar Day', 'Lunar Day').str.strptime(pl.Date, format='%m/%d/%Y')) #🧩
            NormHdays = NormHdays.select(NormHdays_schema) #🧩 Select columns in the correct order and names as per df_schema.
            print_colored(f"⚙️Column structure adjustment completed", "Olive")
            logger.info(f"⚙️Column structure adjustment completed")
            info_polars(NormHdays) #🧩 Display information about the DataFrame (shape, columns, dtypes).
            #📃 Start writing data to the database.
            print_colored(f"💾Start import file: {file_basename}", "Coral")
            logger.info(f"💾Start import file: {file_basename}")
            write_data(engine, NormHdays_TABLE_NAME, NormHdays) #🧩 Call the function to write the DataFrame to SQL.
            print_colored(f"Successfully imported✨: {file_basename}", "LimeGreen")
            logger.info(f"Successfully imported✨: {file_basename}")
            #📃 Record to summary
            if is_new_file:
                summary_results["new_files_imported"].append(f"[NormHdays] {file_basename}")#🧩
            else:
                summary_results["updated_files_reimported"].append(f"[NormHdays] {file_basename}")#🧩  
            #📃 Update log_entries for successfully import: Add information to the log list for this run (successful).
            log_entries.append({"FileName": file_basename, "ModifiedDate": current_modified_date, "Error": None})
            print_colored(f"Updated 'Modified' time for {file_basename}", "LimeGreen")
            logger.info(f"Updated 'Modified' time for {file_basename}")
            #📃 Handle exceptions if any error occurred in the try block.
        except (
            FileNotFoundError, PermissionError, UnicodeDecodeError, IOError,                
            pl.exceptions.NoDataError, pl.exceptions.ComputeError, pl.exceptions.SchemaError, sa.exc.SQLAlchemyError,
            Exception # Catch various types of potential errors.
        ) as e:
            error_count += 1 # Increment the error counter.        
            error_msg_short = f"{type(e).__name__}: {str(e).splitlines()[0]}"  # Get a short error message (the first line).
            #📃 Update Failed file to Summary
            summary_results["failed_imports"].append((f"[NormHdays] {file_basename}", error_msg_short))#🧩
            logger.error(f"Error importing {file_basename}: {error_msg_short}", exc_info=True) # Log the full error traceback.
            print_colored(f"❗️Error importing {file_basename}: {error_msg_short}", "OrangeRed")
            # Add information to the log list for this run (failed).
            # Note: ModifiedDate here is previous_modified_date, as this file wasn't successfully imported with current_modified_date.
            log_entries.append({"FileName": file_basename,"ModifiedDate": previous_modified_date,"Error": error_msg_short})
    else: #📃 If the file is not new and its modification date hasn't changed.
          #📃 Update Skip file to Summary
        summary_results["skipped_files_unchanged"].append(f"[NormHdays] {file_basename}")#🧩
        print_colored(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.", "Aquamarine")
        logger.info(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.")
#💡 After iterating through all files in the directory.
display_summary("NormHdays", error_count) #🧩 Display a summary of the results (number of errors).
process_and_save_log(log_df, log_entries, log_NormHdays_path) #🧩 Update the Excel log file with information
logger.info("===== Processing of the NormHdays data source is complete =====") #🧩 Log finish data processing.

In [ ]:
# 0️⃣9️⃣[GLB]EmpMaster💾
logger.info("===== Start EmpMaster Process =====")  #🧩 Log the start of the data processing.
log_df = read_or_create_log(log_EmpMaster_path) #🧩 Read the existing Excel log file or create a new one if it doesn't exist.
log_entries, error_count = Default_variable() #📃 Initialize a list to store log results for the current run and an error counter.
#💡 Iterate through all files in the specified folder.
for filename in Folder_EmpMaster.glob("*.xlsx"): #🧩 Get files in the Folder directory.
    #📃 Get the current modification timestamp and filename of the current file.
    current_modified_date_microseconds = datetime.datetime.fromtimestamp(filename.stat().st_mtime) # Get modification timestamp, including microseconds.
    current_modified_date = current_modified_date_microseconds.replace(microsecond=0) # Remove microseconds for easier comparison.
    file_basename = filename.name # Get the base name of the file (e.g., "data_20230101.csv").
    #📃 File is being processed.
    print_colored(f"Processing file🔄️: {file_basename}", "NavajoWhite")
    logger.info(f"Processing file🔄️: {file_basename}")
    #📃 Check if this file has been logged before.
    log_entry = log_df.filter(pl.col("FileName") == file_basename) # Filter the old log_df to find an entry for this filename.
    is_new_file = log_entry.is_empty() # If no entry is found (log_entry is empty), it's a new file.
    previous_modified_date = datetime.datetime(1900, 1, 1, 0, 0, 1)  # Initialize the previous modification date with a very old date.
    #📃 If this file was processed before (exists in the log).  
    if not is_new_file:
        previous_modified_date = log_entry.select("ModifiedDate").item() # Get the modification date recorded in the log from the previous processing.
    #📃 Condition to process the file:
    # 1. The file is new (not in the log).
    # 2. Or, the file is old (in the log), but its current modification date (current_modified_date) is more recent than the modification date recorded in the log (previous_modified_date).
    if is_new_file or current_modified_date > previous_modified_date:
        if not is_new_file: # If it's an old file but has changed (ModifiedDate is newer).
            print_colored(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.", "Coral")
            logger.warning(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.")
        else: # If it's a completely new file.
            print_colored(f"New file detected🚦: {file_basename}. Importing.", "Coral")
            logger.info(f"New file detected🚦: {file_basename}. Importing.")
        try: # Start a try-except block to handle potential errors during the import process.
            if not is_new_file: # If this file already exists in the DB (because it's not new and its ModifiedDate changed) then delete the old data for this file from the df.
                delete_data(engine, EmpMaster_TABLE_NAME, file_basename)  #🧩 Call the function to delete data.
            #📃 Read data from the excel file using Polars.
            EmpMaster = (pl.read_excel(filename, infer_schema_length=None, engine="calamine", sheet_name="Employee Master") #🧩 Read the entire file to infer the schema.Add two new columns to the DataFrame: FileName and ModifiedDate.
                   .select(pl.all(),pl.lit(file_basename).alias("FileName"),pl.lit(current_modified_date).dt.cast_time_unit("ms").alias("ModifiedDate")
                   )) #🧩
            #📃 Validate if the columns in the DataFrame match the defined schema
            has_critical_error, critical_schema_error_msg = validate_schema(EmpMaster, EmpMaster_schema, file_basename) #🧩
            if has_critical_error: # If critical columns are missing.
                raise ValueError(critical_schema_error_msg) # Raise an error to stop processing this file.
            #📃 Adjust the column structure.
            print_colored(f"⚙️Adjust column structure", "Olive")
            logger.warning(f"⚙️Adjust column structure")
            EmpMaster = EmpMaster.with_columns(pl.col("ORIGINAL_DATE_OF_HIRE", "LEGAL_EMPLOYER_HIRE_DATE", "Continuous Service Date", \
                                                      "Fixed Term Hire End Date", "Contract End Date", "Last Promotion Date",).cast(pl.Date)) #🧩
            EmpMaster = EmpMaster.select(EmpMaster_schema) #🧩 Select columns in the correct order and names as per df_schema.
            print_colored(f"⚙️Column structure adjustment completed", "Olive")
            logger.info(f"⚙️Column structure adjustment completed")
            info_polars(EmpMaster) #🧩 Display information about the DataFrame (shape, columns, dtypes).
            #📃 Start writing data to the database.
            print_colored(f"💾Start import file: {file_basename}", "Coral")
            logger.info(f"💾Start import file: {file_basename}")
            write_data(engine, EmpMaster_TABLE_NAME, EmpMaster) #🧩 Call the function to write the DataFrame to SQL.
            print_colored(f"Successfully imported✨: {file_basename}", "LimeGreen")
            logger.info(f"Successfully imported✨: {file_basename}")
            #📃 Record to summary
            if is_new_file:
                summary_results["new_files_imported"].append(f"[EmpMaster] {file_basename}")#🧩
            else:
                summary_results["updated_files_reimported"].append(f"[EmpMaster] {file_basename}")#🧩
            #📃 Update log_entries for successfully import: Add information to the log list for this run (successful).
            log_entries.append({"FileName": file_basename, "ModifiedDate": current_modified_date, "Error": None})
            print_colored(f"Updated 'Modified' time for {file_basename}", "LimeGreen")
            logger.info(f"Updated 'Modified' time for {file_basename}")
            #📃 Handle exceptions if any error occurred in the try block.
        except (
            FileNotFoundError, PermissionError, UnicodeDecodeError, IOError,                
            pl.exceptions.NoDataError, pl.exceptions.ComputeError, pl.exceptions.SchemaError, sa.exc.SQLAlchemyError,
            Exception # Catch various types of potential errors.
        ) as e:
            error_count += 1 # Increment the error counter.        
            error_msg_short = f"{type(e).__name__}: {str(e).splitlines()[0]}"  # Get a short error message (the first line).
            #📃 Update Failed file to Summary
            summary_results["failed_imports"].append((f"[EmpMaster] {file_basename}", error_msg_short))#🧩
            logger.error(f"Error importing {file_basename}: {error_msg_short}", exc_info=True) # Log the full error traceback.
            print_colored(f"❗️Error importing {file_basename}: {error_msg_short}", "OrangeRed")
            # Add information to the log list for this run (failed).
            # Note: ModifiedDate here is previous_modified_date, as this file wasn't successfully imported with current_modified_date.
            log_entries.append({"FileName": file_basename,"ModifiedDate": previous_modified_date,"Error": error_msg_short})
    else: #📃 If the file is not new and its modification date hasn't changed.
          #📃 Update Skip file to Summary
        summary_results["skipped_files_unchanged"].append(f"[EmpMaster] {file_basename}")#🧩
        print_colored(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.", "Aquamarine")
        logger.info(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.")
#💡 After iterating through all files in the directory.
display_summary("EmpMaster", error_count) #🧩 Display a summary of the results (number of errors).
process_and_save_log(log_df, log_entries, log_EmpMaster_path) #🧩 Update the Excel log file with information
logger.info("===== Processing of the EmpMaster data source is complete =====") #🧩 Log finish data processing.

In [ ]:
# 1️⃣0️⃣[GLB]Termination💾
logger.info("===== Start Termination Process =====")  #🧩 Log the start of the data processing.
log_df = read_or_create_log(log_Termination_path) #🧩 Read the existing Excel log file or create a new one if it doesn't exist.
log_entries, error_count = Default_variable() #📃 Initialize a list to store log results for the current run and an error counter.
#💡 Iterate through all files in the specified folder.
for filename in Folder_Termination.glob("*.xlsx"): #🧩 Get files in the Folder directory.
    #📃 Get the current modification timestamp and filename of the current file.
    current_modified_date_microseconds = datetime.datetime.fromtimestamp(filename.stat().st_mtime) # Get modification timestamp, including microseconds.
    current_modified_date = current_modified_date_microseconds.replace(microsecond=0) # Remove microseconds for easier comparison.
    file_basename = filename.name # Get the base name of the file (e.g., "data_20230101.csv").
    #📃 File is being processed.
    print_colored(f"Processing file🔄️: {file_basename}", "NavajoWhite")
    logger.info(f"Processing file🔄️: {file_basename}")
    #📃 Check if this file has been logged before.
    log_entry = log_df.filter(pl.col("FileName") == file_basename) # Filter the old log_df to find an entry for this filename.
    is_new_file = log_entry.is_empty() # If no entry is found (log_entry is empty), it's a new file.
    previous_modified_date = datetime.datetime(1900, 1, 1, 0, 0, 1)  # Initialize the previous modification date with a very old date.
    #📃 If this file was processed before (exists in the log).  
    if not is_new_file:
        previous_modified_date = log_entry.select("ModifiedDate").item() # Get the modification date recorded in the log from the previous processing.
    #📃 Condition to process the file:
    # 1. The file is new (not in the log).
    # 2. Or, the file is old (in the log), but its current modification date (current_modified_date) is more recent than the modification date recorded in the log (previous_modified_date).
    if is_new_file or current_modified_date > previous_modified_date:
        if not is_new_file: # If it's an old file but has changed (ModifiedDate is newer).
            print_colored(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.", "Coral")
            logger.warning(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.")
        else: # If it's a completely new file.
            print_colored(f"New file detected🚦: {file_basename}. Importing.", "Coral")
            logger.info(f"New file detected🚦: {file_basename}. Importing.")
        try: # Start a try-except block to handle potential errors during the import process.
            if not is_new_file: # If this file already exists in the DB (because it's not new and its ModifiedDate changed) then delete the old data for this file from the df.
                delete_data(engine, Termination_TABLE_NAME, file_basename)  #🧩 Call the function to delete data.
            #📃 Read data from the excel file using Polars.
            Termination = (pl.read_excel(filename, infer_schema_length=None, engine="calamine", sheet_name="Termination Dump") #🧩 Read the entire file to infer the schema.Add two new columns to the DataFrame: FileName and ModifiedDate.
                    .select(pl.all(),pl.lit(file_basename).alias("FileName"),pl.lit(current_modified_date).dt.cast_time_unit("ms").alias("ModifiedDate")
                   )) #🧩
            #📃 Validate if the columns in the DataFrame match the defined schema
            has_critical_error, critical_schema_error_msg = validate_schema(Termination, Termination_schema, file_basename) #🧩
            if has_critical_error: # If critical columns are missing.
                raise ValueError(critical_schema_error_msg) # Raise an error to stop processing this file.
            #📃 Adjust the column structure.
            print_colored(f"⚙️Adjust column structure", "Olive")
            logger.warning(f"⚙️Adjust column structure")
            Termination = Termination.with_columns(pl.col("HIRE_DATE", "ORIGINAL_HIRE_DATE", "END EMPLOYMENT DATE", 
                                                          "Contract End Date", "Termination Date", "Termination Date (DD/MM/YY)", 
                                                          "LWD", "MOST RECENT TERMINATION - DATE INITIATED", "MOST RECENT TERMINATION - DATE COMPLETED",
                                                          "MOST RECENT TERMINATION - EFFECTIVE DATE", "DATE INITIATED",
                                                          "TERMINATION DATE 2", "Continuous Service Date").cast(pl.Date)) #🧩
            Termination = Termination.with_columns(pl.col("Action date", "COMPELETED DATE AND TIME").cast(pl.Datetime)) #🧩
            Termination = Termination.with_columns(pl.col("Resignation Notice served").cast(pl.Int64)) #🧩
            Termination = Termination.select(Termination_schema) #🧩 Select columns in the correct order and names as per df_schema.
            print_colored(f"⚙️Column structure adjustment completed", "Olive")
            logger.info(f"⚙️Column structure adjustment completed")
            info_polars(Termination) #🧩 Display information about the DataFrame (shape, columns, dtypes).
            #📃 Start writing data to the database.
            print_colored(f"💾Start import file: {file_basename}", "Coral")
            logger.info(f"💾Start import file: {file_basename}")
            write_data(engine, Termination_TABLE_NAME, Termination) #🧩 Call the function to write the DataFrame to SQL.
            print_colored(f"Successfully imported✨: {file_basename}", "LimeGreen")
            logger.info(f"Successfully imported✨: {file_basename}")
            #📃 Record to summary
            if is_new_file:
                summary_results["new_files_imported"].append(f"[Termination] {file_basename}")#🧩
            else:
                summary_results["updated_files_reimported"].append(f"[Termination] {file_basename}")#🧩    
            #📃 Update log_entries for successfully import: Add information to the log list for this run (successful).
            log_entries.append({"FileName": file_basename, "ModifiedDate": current_modified_date, "Error": None})
            print_colored(f"Updated 'Modified' time for {file_basename}", "LimeGreen")
            logger.info(f"Updated 'Modified' time for {file_basename}")
            #📃 Handle exceptions if any error occurred in the try block.
        except (
            FileNotFoundError, PermissionError, UnicodeDecodeError, IOError,                
            pl.exceptions.NoDataError, pl.exceptions.ComputeError, pl.exceptions.SchemaError, sa.exc.SQLAlchemyError,
            Exception # Catch various types of potential errors.
        ) as e:
            error_count += 1 # Increment the error counter.        
            error_msg_short = f"{type(e).__name__}: {str(e).splitlines()[0]}"  # Get a short error message (the first line).
            #📃 Update Failed file to Summary
            summary_results["failed_imports"].append((f"[Termination] {file_basename}", error_msg_short))#🧩      
            logger.error(f"Error importing {file_basename}: {error_msg_short}", exc_info=True) # Log the full error traceback.
            print_colored(f"❗️Error importing {file_basename}: {error_msg_short}", "OrangeRed")
            # Add information to the log list for this run (failed).
            # Note: ModifiedDate here is previous_modified_date, as this file wasn't successfully imported with current_modified_date.
            log_entries.append({"FileName": file_basename,"ModifiedDate": previous_modified_date,"Error": error_msg_short})
    else: #📃 If the file is not new and its modification date hasn't changed.
          #📃 Update Skip file to Summary
        summary_results["skipped_files_unchanged"].append(f"[Termination] {file_basename}")#🧩
        print_colored(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.", "Aquamarine")
        logger.info(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.")
#💡 After iterating through all files in the directory.
display_summary("Termination", error_count) #🧩 Display a summary of the results (number of errors).
process_and_save_log(log_df, log_entries, log_Termination_path) #🧩 Update the Excel log file with information
logger.info("===== Processing of the Termination data source is complete =====") #🧩 Log finish data processing.

In [ ]:
# 1️⃣1️⃣[GLB]Resignation💾
logger.info("===== Start Resignation Process =====")  #🧩 Log the start of the data processing.
log_df = read_or_create_log(log_Resignation_path) #🧩 Read the existing Excel log file or create a new one if it doesn't exist.
log_entries, error_count = Default_variable() #📃 Initialize a list to store log results for the current run and an error counter.
#💡 Iterate through all files in the specified folder.
for filename in Folder_Resignation.glob("*.xlsx"): #🧩 Get files in the Folder directory.
    #📃 Get the current modification timestamp and filename of the current file.
    current_modified_date_microseconds = datetime.datetime.fromtimestamp(filename.stat().st_mtime) # Get modification timestamp, including microseconds.
    current_modified_date = current_modified_date_microseconds.replace(microsecond=0) # Remove microseconds for easier comparison.
    file_basename = filename.name # Get the base name of the file (e.g., "data_20230101.csv").
    #📃 File is being processed.
    print_colored(f"Processing file🔄️: {file_basename}", "NavajoWhite")
    logger.info(f"Processing file🔄️: {file_basename}")
    #📃 Check if this file has been logged before.
    log_entry = log_df.filter(pl.col("FileName") == file_basename) # Filter the old log_df to find an entry for this filename.
    is_new_file = log_entry.is_empty() # If no entry is found (log_entry is empty), it's a new file.
    previous_modified_date = datetime.datetime(1900, 1, 1, 0, 0, 1)  # Initialize the previous modification date with a very old date.
    #📃 If this file was processed before (exists in the log).  
    if not is_new_file:
        previous_modified_date = log_entry.select("ModifiedDate").item() # Get the modification date recorded in the log from the previous processing.
    #📃 Condition to process the file:
    # 1. The file is new (not in the log).
    # 2. Or, the file is old (in the log), but its current modification date (current_modified_date) is more recent than the modification date recorded in the log (previous_modified_date).
    if is_new_file or current_modified_date > previous_modified_date:
        if not is_new_file: # If it's an old file but has changed (ModifiedDate is newer).
            print_colored(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.", "Coral")
            logger.warning(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.")
        else: # If it's a completely new file.
            print_colored(f"New file detected🚦: {file_basename}. Importing.", "Coral")
            logger.info(f"New file detected🚦: {file_basename}. Importing.")
        try: # Start a try-except block to handle potential errors during the import process.
            if not is_new_file: # If this file already exists in the DB (because it's not new and its ModifiedDate changed) then delete the old data for this file from the df.
                delete_data(engine, Resignation_TABLE_NAME, file_basename)  #🧩 Call the function to delete data.
            #📃 Read data from the excel file using Polars.
            Resignation = (pl.read_excel(filename, infer_schema_length=None, engine="calamine", sheet_name="Resignation") #🧩 Read the entire file to infer the schema.Add two new columns to the DataFrame: FileName and ModifiedDate.
                   .select(pl.all(),pl.lit(file_basename).alias("FileName"),pl.lit(current_modified_date).dt.cast_time_unit("ms").alias("ModifiedDate")
                   )) #🧩
            #📃 Validate if the columns in the DataFrame match the defined schema
            has_critical_error, critical_schema_error_msg = validate_schema(Resignation, Resignation_schema, file_basename) #🧩
            if has_critical_error: # If critical columns are missing.
                raise ValueError(critical_schema_error_msg) # Raise an error to stop processing this file.
            #📃 Adjust the column structure.
            print_colored(f"⚙️Adjust column structure", "Olive")
            logger.warning(f"⚙️Adjust column structure")
            Resignation = Resignation.with_columns(pl.col("Notification Date", "Hire Date", "Proposed Termination Date",).cast(pl.Date)) #🧩
            Resignation = Resignation.with_columns(pl.col("Action Date", "Date and Time Initiated").cast(pl.Datetime)) #🧩
            Resignation = Resignation.with_columns(pl.col("Notice Served").cast(pl.Int64)) #🧩
            Resignation = Resignation.select(Resignation_schema) #🧩 Select columns in the correct order and names as per df_schema.
            print_colored(f"⚙️Column structure adjustment completed", "Olive")
            logger.info(f"⚙️Column structure adjustment completed")
            info_polars(Resignation) #🧩 Display information about the DataFrame (shape, columns, dtypes).
            #📃 Start writing data to the database.
            print_colored(f"💾Start import file: {file_basename}", "Coral")
            logger.info(f"💾Start import file: {file_basename}")
            write_data(engine, Resignation_TABLE_NAME, Resignation) #🧩 Call the function to write the DataFrame to SQL.
            print_colored(f"Successfully imported✨: {file_basename}", "LimeGreen")
            logger.info(f"Successfully imported✨: {file_basename}")
            #📃 Record to summary
            if is_new_file:
                summary_results["new_files_imported"].append(f"[Resignation] {file_basename}")#🧩
            else:
                summary_results["updated_files_reimported"].append(f"[Resignation] {file_basename}")#🧩        
            #📃 Update log_entries for successfully import: Add information to the log list for this run (successful).
            log_entries.append({"FileName": file_basename, "ModifiedDate": current_modified_date, "Error": None})
            print_colored(f"Updated 'Modified' time for {file_basename}", "LimeGreen")
            logger.info(f"Updated 'Modified' time for {file_basename}")
            #📃 Handle exceptions if any error occurred in the try block.
        except (
            FileNotFoundError, PermissionError, UnicodeDecodeError, IOError,                
            pl.exceptions.NoDataError, pl.exceptions.ComputeError, pl.exceptions.SchemaError, sa.exc.SQLAlchemyError,
            Exception # Catch various types of potential errors.
        ) as e:
            error_count += 1 # Increment the error counter.        
            error_msg_short = f"{type(e).__name__}: {str(e).splitlines()[0]}"  # Get a short error message (the first line).
            #📃 Update Failed file to Summary
            summary_results["failed_imports"].append((f"[Resignation] {file_basename}", error_msg_short))#🧩
            logger.error(f"Error importing {file_basename}: {error_msg_short}", exc_info=True) # Log the full error traceback.
            print_colored(f"❗️Error importing {file_basename}: {error_msg_short}", "OrangeRed")
            # Add information to the log list for this run (failed).
            # Note: ModifiedDate here is previous_modified_date, as this file wasn't successfully imported with current_modified_date.
            log_entries.append({"FileName": file_basename,"ModifiedDate": previous_modified_date,"Error": error_msg_short})
    else: #📃 If the file is not new and its modification date hasn't changed.
          #📃 Update Skip file to Summary
        summary_results["skipped_files_unchanged"].append(f"[Resignation] {file_basename}")#🧩
        print_colored(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.", "Aquamarine")
        logger.info(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.")
#💡 After iterating through all files in the directory.
display_summary("Resignation", error_count) #🧩 Display a summary of the results (number of errors).
process_and_save_log(log_df, log_entries, log_Resignation_path) #🧩 Update the Excel log file with information
logger.info("===== Processing of the Resignation data source is complete =====") #🧩 Log finish data processing.

In [ ]:
# 1️⃣2️⃣[BKN]CPI_PEGA💾
logger.info("===== Start CPI_PEGA Process =====")  #🧩 Log the start of the data processing.
log_df = read_or_create_log(log_CPI_PEGA_path) #🧩 Read the existing Excel log file or create a new one if it doesn't exist.
log_entries, error_count = Default_variable() #📃 Initialize a list to store log results for the current run and an error counter.
#💡 Iterate through all files in the specified folder.
for filename in Folder_CPI_PEGA.glob("*.csv"): #🧩 Get files in the Folder directory.
    #📃 Get the current modification timestamp and filename of the current file.
    current_modified_date_microseconds = datetime.datetime.fromtimestamp(filename.stat().st_mtime) # Get modification timestamp, including microseconds.
    current_modified_date = current_modified_date_microseconds.replace(microsecond=0) # Remove microseconds for easier comparison.
    file_basename = filename.name # Get the base name of the file (e.g., "data_20230101.csv").
    #📃 File is being processed.
    print_colored(f"Processing file🔄️: {file_basename}", "NavajoWhite")
    logger.info(f"Processing file🔄️: {file_basename}")
    #📃 Check if this file has been logged before.
    log_entry = log_df.filter(pl.col("FileName") == file_basename) # Filter the old log_df to find an entry for this filename.
    is_new_file = log_entry.is_empty() # If no entry is found (log_entry is empty), it's a new file.
    previous_modified_date = datetime.datetime(1900, 1, 1, 0, 0, 1)  # Initialize the previous modification date with a very old date.
    #📃 If this file was processed before (exists in the log).  
    if not is_new_file:
        previous_modified_date = log_entry.select("ModifiedDate").item() # Get the modification date recorded in the log from the previous processing.
    #📃 Condition to process the file:
    # 1. The file is new (not in the log).
    # 2. Or, the file is old (in the log), but its current modification date (current_modified_date) is more recent than the modification date recorded in the log (previous_modified_date).
    if is_new_file or current_modified_date > previous_modified_date:
        if not is_new_file: # If it's an old file but has changed (ModifiedDate is newer).
            print_colored(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.", "Coral")
            logger.warning(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.")
        else: # If it's a completely new file.
            print_colored(f"New file detected🚦: {file_basename}. Importing.", "Coral")
            logger.info(f"New file detected🚦: {file_basename}. Importing.")
        try: # Start a try-except block to handle potential errors during the import process.
            if not is_new_file: # If this file already exists in the DB (because it's not new and its ModifiedDate changed) then delete the old data for this file from the df.
                delete_data(engine, CPI_PEGA_TABLE_NAME, file_basename)  #🧩 Call the function to delete data.
            #📃 Read data from the excel file using Polars.
            CPI_PEGA = (pl.read_csv(filename, infer_schema_length=None) #🧩 Read the entire file to infer the schema.Add two new columns to the DataFrame: FileName and ModifiedDate.
                    .select(pl.all(),pl.lit(file_basename).alias("FileName"),pl.lit(current_modified_date).dt.cast_time_unit("ms").alias("ModifiedDate")
                   )) #🧩
            #📃 Validate if the columns in the DataFrame match the defined schema
            has_critical_error, critical_schema_error_msg = validate_schema(CPI_PEGA, CPI_PEGA_schema, file_basename) #🧩
            if has_critical_error: # If critical columns are missing.
                raise ValueError(critical_schema_error_msg) # Raise an error to stop processing this file.
            #📃 Adjust the column structure.
            print_colored(f"⚙️Adjust column structure", "Olive")
            logger.warning(f"⚙️Adjust column structure")
            CPI_PEGA = CPI_PEGA.with_columns(parse_date(pl.col("Day of Date")).alias("Day of Date")) #🧩
            CPI_PEGA = CPI_PEGA.with_columns(pl.col("# Swivels", "Count of ServiceCase or Interaction").cast(pl.Int64)) #🧩
            CPI_PEGA = CPI_PEGA.select(CPI_PEGA_schema) #🧩 Select columns in the correct order and names as per df_schema.
            print_colored(f"⚙️Column structure adjustment completed", "Olive")
            logger.info(f"⚙️Column structure adjustment completed")
            info_polars(CPI_PEGA) #🧩 Display information about the DataFrame (shape, columns, dtypes).
            #📃 Start writing data to the database.
            print_colored(f"💾Start import file: {file_basename}", "Coral")
            logger.info(f"💾Start import file: {file_basename}")
            write_data(engine, CPI_PEGA_TABLE_NAME, CPI_PEGA) #🧩 Call the function to write the DataFrame to SQL.
            print_colored(f"Successfully imported✨: {file_basename}", "LimeGreen")
            logger.info(f"Successfully imported✨: {file_basename}")
            #📃 Record to summary
            if is_new_file:
                summary_results["new_files_imported"].append(f"[CPI_PEGA] {file_basename}")#🧩
            else:
                summary_results["updated_files_reimported"].append(f"[CPI_PEGA] {file_basename}")#🧩    
            #📃 Update log_entries for successfully import: Add information to the log list for this run (successful).
            log_entries.append({"FileName": file_basename, "ModifiedDate": current_modified_date, "Error": None})
            print_colored(f"Updated 'Modified' time for {file_basename}", "LimeGreen")
            logger.info(f"Updated 'Modified' time for {file_basename}")
            #📃 Handle exceptions if any error occurred in the try block.
        except (
            FileNotFoundError, PermissionError, UnicodeDecodeError, IOError,                
            pl.exceptions.NoDataError, pl.exceptions.ComputeError, pl.exceptions.SchemaError, sa.exc.SQLAlchemyError,
            Exception # Catch various types of potential errors.
        ) as e:
            error_count += 1 # Increment the error counter.        
            error_msg_short = f"{type(e).__name__}: {str(e).splitlines()[0]}"  # Get a short error message (the first line).
            #📃 Update Failed file to Summary
            summary_results["failed_imports"].append((f"[CPI_PEGA] {file_basename}", error_msg_short))#🧩  
            logger.error(f"Error importing {file_basename}: {error_msg_short}", exc_info=True) # Log the full error traceback.
            print_colored(f"❗️Error importing {file_basename}: {error_msg_short}", "OrangeRed")
            # Add information to the log list for this run (failed).
            # Note: ModifiedDate here is previous_modified_date, as this file wasn't successfully imported with current_modified_date.
            log_entries.append({"FileName": file_basename,"ModifiedDate": previous_modified_date,"Error": error_msg_short})
    else: #📃 If the file is not new and its modification date hasn't changed.
          #📃 Update Skip file to Summary
        summary_results["skipped_files_unchanged"].append(f"[CPI_PEGA] {file_basename}")#🧩
        print_colored(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.", "Aquamarine")
        logger.info(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.")
#💡 After iterating through all files in the directory.
display_summary("CPI_PEGA", error_count) #🧩 Display a summary of the results (number of errors).
process_and_save_log(log_df, log_entries, log_CPI_PEGA_path) #🧩 Update the Excel log file with information
logger.info("===== Processing of the CPI_PEGA data source is complete =====") #🧩 Log finish data processing.

In [ ]:
# 1️⃣3️⃣[BKN]Staff💾
logger.info("===== Start Staff Process =====")  #🧩 Log the start of the data processing.
log_df = read_or_create_log(log_Staff_path) #🧩 Read the existing Excel log file or create a new one if it doesn't exist.
log_entries, error_count = Default_variable() #📃 Initialize a list to store log results for the current run and an error counter.
#💡 Iterate through all files in the specified folder.
for filename in Folder_Staff.glob("*.xlsx"): #🧩 Get files in the Folder directory.
    #📃 Get the current modification timestamp and filename of the current file.
    current_modified_date_microseconds = datetime.datetime.fromtimestamp(filename.stat().st_mtime) # Get modification timestamp, including microseconds.
    current_modified_date = current_modified_date_microseconds.replace(microsecond=0) # Remove microseconds for easier comparison.
    file_basename = filename.name # Get the base name of the file (e.g., "data_20230101.csv").
    #📃 File is being processed.
    print_colored(f"Processing file🔄️: {file_basename}", "NavajoWhite")
    logger.info(f"Processing file🔄️: {file_basename}")
    #📃 Check if this file has been logged before.
    log_entry = log_df.filter(pl.col("FileName") == file_basename) # Filter the old log_df to find an entry for this filename.
    is_new_file = log_entry.is_empty() # If no entry is found (log_entry is empty), it's a new file.
    previous_modified_date = datetime.datetime(1900, 1, 1, 0, 0, 1)  # Initialize the previous modification date with a very old date.
    #📃 If this file was processed before (exists in the log).  
    if not is_new_file:
        previous_modified_date = log_entry.select("ModifiedDate").item() # Get the modification date recorded in the log from the previous processing.
    #📃 Condition to process the file:
    # 1. The file is new (not in the log).
    # 2. Or, the file is old (in the log), but its current modification date (current_modified_date) is more recent than the modification date recorded in the log (previous_modified_date).
    if is_new_file or current_modified_date > previous_modified_date:
        if not is_new_file: # If it's an old file but has changed (ModifiedDate is newer).
            print_colored(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.", "Coral")
            logger.warning(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.")
        else: # If it's a completely new file.
            print_colored(f"New file detected🚦: {file_basename}. Importing.", "Coral")
            logger.info(f"New file detected🚦: {file_basename}. Importing.")
        try: # Start a try-except block to handle potential errors during the import process.
            if not is_new_file: # If this file already exists in the DB (because it's not new and its ModifiedDate changed) then delete the old data for this file from the df.
                delete_data(engine, Staff_TABLE_NAME, file_basename)  #🧩 Call the function to delete data.
            #📃 Read data from the excel file using Polars.
            Staff = (pl.read_excel(filename, infer_schema_length=None, engine="calamine", sheet_name="Sheet1") #🧩 Read the entire file to infer the schema.Add two new columns to the DataFrame: FileName and ModifiedDate.
                   .select(pl.all(),pl.lit(file_basename).alias("FileName"),pl.lit(current_modified_date).dt.cast_time_unit("ms").alias("ModifiedDate")
                   )) #🧩
            #📃 Validate if the columns in the DataFrame match the defined schema
            has_critical_error, critical_schema_error_msg = validate_schema(Staff, Staff_schema, file_basename) #🧩
            if has_critical_error: # If critical columns are missing.
                raise ValueError(critical_schema_error_msg) # Raise an error to stop processing this file.
            #📃 Adjust the column structure.
            print_colored(f"⚙️Adjust column structure", "Olive")
            logger.warning(f"⚙️Adjust column structure")
            Staff = Staff.with_columns(pl.col("Language Start Date", "Hire_Date", "PST_Start_Date", "Production_Start_Date", 
                                              "LWD", "Termination_Date").cast(pl.Date))
            Staff = Staff.select(Staff_schema) #🧩 Select columns in the correct order and names as per df_schema.
            print_colored(f"⚙️Column structure adjustment completed", "Olive")
            logger.info(f"⚙️Column structure adjustment completed")
            info_polars(Staff) #🧩 Display information about the DataFrame (shape, columns, dtypes).
            #📃 Start writing data to the database.
            print_colored(f"💾Start import file: {file_basename}", "Coral")
            logger.info(f"💾Start import file: {file_basename}")
            write_data(engine, Staff_TABLE_NAME, Staff) #🧩 Call the function to write the DataFrame to SQL.
            print_colored(f"Successfully imported✨: {file_basename}", "LimeGreen")
            logger.info(f"Successfully imported✨: {file_basename}")
            #📃 Record to summary
            if is_new_file:
                summary_results["new_files_imported"].append(f"[Staff] {file_basename}")#🧩
            else:
                summary_results["updated_files_reimported"].append(f"[Staff] {file_basename}")#🧩     
            #📃 Update log_entries for successfully import: Add information to the log list for this run (successful).
            log_entries.append({"FileName": file_basename, "ModifiedDate": current_modified_date, "Error": None})
            print_colored(f"Updated 'Modified' time for {file_basename}", "LimeGreen")
            logger.info(f"Updated 'Modified' time for {file_basename}")
            #📃 Handle exceptions if any error occurred in the try block.
        except (
            FileNotFoundError, PermissionError, UnicodeDecodeError, IOError,                
            pl.exceptions.NoDataError, pl.exceptions.ComputeError, pl.exceptions.SchemaError, sa.exc.SQLAlchemyError,
            Exception # Catch various types of potential errors.
        ) as e:
            error_count += 1 # Increment the error counter.        
            error_msg_short = f"{type(e).__name__}: {str(e).splitlines()[0]}"  # Get a short error message (the first line).
            #📃 Update Failed file to Summary
            summary_results["failed_imports"].append((f"[Staff] {file_basename}", error_msg_short))#🧩        
            logger.error(f"Error importing {file_basename}: {error_msg_short}", exc_info=True) # Log the full error traceback.
            print_colored(f"❗️Error importing {file_basename}: {error_msg_short}", "OrangeRed")
            # Add information to the log list for this run (failed).
            # Note: ModifiedDate here is previous_modified_date, as this file wasn't successfully imported with current_modified_date.
            log_entries.append({"FileName": file_basename,"ModifiedDate": previous_modified_date,"Error": error_msg_short})
    else: #📃 If the file is not new and its modification date hasn't changed.
          #📃 Update Skip file to Summary
        summary_results["skipped_files_unchanged"].append(f"[Staff] {file_basename}")#🧩      
        print_colored(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.", "Aquamarine")
        logger.info(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.")
#💡 After iterating through all files in the directory.
display_summary("Staff", error_count) #🧩 Display a summary of the results (number of errors).
process_and_save_log(log_df, log_entries, log_Staff_path) #🧩 Update the Excel log file with information
logger.info("===== Processing of the Staff data source is complete =====") #🧩 Log finish data processing.

In [ ]:
# 1️⃣4️⃣[BKN]ConTrack💾
logger.info("===== Start ConTrack Process =====")  #🧩 Log the start of the data processing.
log_df = read_or_create_log(log_ConTrack_path) #🧩 Read the existing Excel log file or create a new one if it doesn't exist.
log_entries, error_count = Default_variable() #📃 Initialize a list to store log results for the current run and an error counter.
#💡 Iterate through all files in the specified folder.
for filename in Folder_ConTrack.glob("*.xlsx"): #🧩 Get files in the Folder directory.
    #📃 Get the current modification timestamp and filename of the current file.
    current_modified_date_microseconds = datetime.datetime.fromtimestamp(filename.stat().st_mtime) # Get modification timestamp, including microseconds.
    current_modified_date = current_modified_date_microseconds.replace(microsecond=0) # Remove microseconds for easier comparison.
    file_basename = filename.name # Get the base name of the file (e.g., "data_20230101.csv").
    #📃 File is being processed.
    print_colored(f"Processing file🔄️: {file_basename}", "NavajoWhite")
    logger.info(f"Processing file🔄️: {file_basename}")
    #📃 Check if this file has been logged before.
    log_entry = log_df.filter(pl.col("FileName") == file_basename) # Filter the old log_df to find an entry for this filename.
    is_new_file = log_entry.is_empty() # If no entry is found (log_entry is empty), it's a new file.
    previous_modified_date = datetime.datetime(1900, 1, 1, 0, 0, 1)  # Initialize the previous modification date with a very old date.
    #📃 If this file was processed before (exists in the log).  
    if not is_new_file:
        previous_modified_date = log_entry.select("ModifiedDate").item() # Get the modification date recorded in the log from the previous processing.
    #📃 Condition to process the file:
    # 1. The file is new (not in the log).
    # 2. Or, the file is old (in the log), but its current modification date (current_modified_date) is more recent than the modification date recorded in the log (previous_modified_date).
    if is_new_file or current_modified_date > previous_modified_date:
        if not is_new_file: # If it's an old file but has changed (ModifiedDate is newer).
            print_colored(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.", "Coral")
            logger.warning(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.")
        else: # If it's a completely new file.
            print_colored(f"New file detected🚦: {file_basename}. Importing.", "Coral")
            logger.info(f"New file detected🚦: {file_basename}. Importing.")
        try: # Start a try-except block to handle potential errors during the import process.
            if not is_new_file: # If this file already exists in the DB (because it's not new and its ModifiedDate changed) then delete the old data for this file from the df.
                delete_data(engine, ConTrack_TABLE_NAME, file_basename)  #🧩 Call the function to delete data.
            #📃 Read data from the excel file using Polars.
            ConTrack = (pl.read_excel(filename, infer_schema_length=1000, sheet_name="Sheet1",
                         schema_overrides={"Reservation Number": pl.String, "Note": pl.String, 
                                           "Reason If Skipped": pl.String}) #🧩 Read the entire file to infer the schema.Add two new columns to the DataFrame: FileName and ModifiedDate.
                   .select(pl.all(),pl.lit(file_basename).alias("FileName"),pl.lit(current_modified_date).dt.cast_time_unit("ms").alias("ModifiedDate")
                   )) #🧩
            #📃 Validate if the columns in the DataFrame match the defined schema
            has_critical_error, critical_schema_error_msg = validate_schema(ConTrack, ConTrack_schema, file_basename) #🧩
            if has_critical_error: # If critical columns are missing.
                raise ValueError(critical_schema_error_msg) # Raise an error to stop processing this file.
            #📃 Adjust the column structure.
            print_colored(f"⚙️Adjust column structure", "Olive")
            logger.warning(f"⚙️Adjust column structure")
            ConTrack = ConTrack.with_columns(pl.col("Start time", "Completion time").cast(pl.Datetime)) #🧩
            ConTrack = ConTrack.select(ConTrack_schema) #🧩 Select columns in the correct order and names as per df_schema.
            print_colored(f"⚙️Column structure adjustment completed", "Olive")
            logger.info(f"⚙️Column structure adjustment completed")
            info_polars(ConTrack) #🧩 Display information about the DataFrame (shape, columns, dtypes).
            #📃 Start writing data to the database.
            print_colored(f"💾Start import file: {file_basename}", "Coral")
            logger.info(f"💾Start import file: {file_basename}")
            write_data(engine, ConTrack_TABLE_NAME, ConTrack) #🧩 Call the function to write the DataFrame to SQL.
            print_colored(f"Successfully imported✨: {file_basename}", "LimeGreen")
            logger.info(f"Successfully imported✨: {file_basename}")
            #📃 Record to summary
            if is_new_file:
                summary_results["new_files_imported"].append(f"[ConTrack] {file_basename}")#🧩
            else:
                summary_results["updated_files_reimported"].append(f"[ConTrack] {file_basename}")#🧩 
            #📃 Update log_entries for successfully import: Add information to the log list for this run (successful).
            log_entries.append({"FileName": file_basename, "ModifiedDate": current_modified_date, "Error": None})
            print_colored(f"Updated 'Modified' time for {file_basename}", "LimeGreen")
            logger.info(f"Updated 'Modified' time for {file_basename}")
            #📃 Handle exceptions if any error occurred in the try block.
        except (
            FileNotFoundError, PermissionError, UnicodeDecodeError, IOError,                
            pl.exceptions.NoDataError, pl.exceptions.ComputeError, pl.exceptions.SchemaError, sa.exc.SQLAlchemyError,
            Exception # Catch various types of potential errors.
        ) as e:
            error_count += 1 # Increment the error counter.        
            error_msg_short = f"{type(e).__name__}: {str(e).splitlines()[0]}"  # Get a short error message (the first line).
            #📃 Update Failed file to Summary
            summary_results["failed_imports"].append((f"[ConTrack] {file_basename}", error_msg_short))#🧩       
            logger.error(f"Error importing {file_basename}: {error_msg_short}", exc_info=True) # Log the full error traceback.
            print_colored(f"❗️Error importing {file_basename}: {error_msg_short}", "OrangeRed")
            # Add information to the log list for this run (failed).
            # Note: ModifiedDate here is previous_modified_date, as this file wasn't successfully imported with current_modified_date.
            log_entries.append({"FileName": file_basename,"ModifiedDate": previous_modified_date,"Error": error_msg_short})
    else: #📃 If the file is not new and its modification date hasn't changed.
          #📃 Update Skip file to Summary
        summary_results["skipped_files_unchanged"].append(f"[ConTrack] {file_basename}")#🧩        
        print_colored(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.", "Aquamarine")
        logger.info(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.")
#💡 After iterating through all files in the directory.
display_summary("ConTrack", error_count) #🧩 Display a summary of the results (number of errors).
process_and_save_log(log_df, log_entries, log_ConTrack_path) #🧩 Update the Excel log file with information
logger.info("===== Processing of the ConTrack data source is complete =====") #🧩 Log finish data processing.

In [ ]:
# 1️⃣5️⃣[BKN]Quality💾
logger.info("===== Start Quality Process =====")  #🧩 Log the start of the data processing.
log_df = read_or_create_log(log_Quality_path) #🧩 Read the existing Excel log file or create a new one if it doesn't exist.
log_entries, error_count = Default_variable() #📃 Initialize a list to store log results for the current run and an error counter.
#💡 Iterate through all files in the specified folder.
for filename in Folder_Quality.glob("*.xlsx"): #🧩 Get files in the Folder directory.
    #📃 Get the current modification timestamp and filename of the current file.
    current_modified_date_microseconds = datetime.datetime.fromtimestamp(filename.stat().st_mtime) # Get modification timestamp, including microseconds.
    current_modified_date = current_modified_date_microseconds.replace(microsecond=0) # Remove microseconds for easier comparison.
    file_basename = filename.name # Get the base name of the file (e.g., "data_20230101.csv").
    #📃 File is being processed.
    print_colored(f"Processing file🔄️: {file_basename}", "NavajoWhite")
    logger.info(f"Processing file🔄️: {file_basename}")
    #📃 Check if this file has been logged before.
    log_entry = log_df.filter(pl.col("FileName") == file_basename) # Filter the old log_df to find an entry for this filename.
    is_new_file = log_entry.is_empty() # If no entry is found (log_entry is empty), it's a new file.
    previous_modified_date = datetime.datetime(1900, 1, 1, 0, 0, 1)  # Initialize the previous modification date with a very old date.
    #📃 If this file was processed before (exists in the log).  
    if not is_new_file:
        previous_modified_date = log_entry.select("ModifiedDate").item() # Get the modification date recorded in the log from the previous processing.
    #📃 Condition to process the file:
    # 1. The file is new (not in the log).
    # 2. Or, the file is old (in the log), but its current modification date (current_modified_date) is more recent than the modification date recorded in the log (previous_modified_date).
    if is_new_file or current_modified_date > previous_modified_date:
        if not is_new_file: # If it's an old file but has changed (ModifiedDate is newer).
            print_colored(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.", "Coral")
            logger.warning(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.")
        else: # If it's a completely new file.
            print_colored(f"New file detected🚦: {file_basename}. Importing.", "Coral")
            logger.info(f"New file detected🚦: {file_basename}. Importing.")
        try: # Start a try-except block to handle potential errors during the import process.
            if not is_new_file: # If this file already exists in the DB (because it's not new and its ModifiedDate changed) then delete the old data for this file from the df.
                delete_data(engine, Quality_TABLE_NAME, file_basename)  #🧩 Call the function to delete data.
            #📃 Read data from the excel file using Polars.
            Quality = (pl.read_excel(filename, infer_schema_length=None, engine="calamine", sheet_name="Sheet1") #🧩 Read the entire file to infer the schema.Add two new columns to the DataFrame: FileName and ModifiedDate.
                   .select(pl.all(),pl.lit(file_basename).alias("FileName"),pl.lit(current_modified_date).dt.cast_time_unit("ms").alias("ModifiedDate")
                   ).rename({' score_question_weight': 'score_question_weight'})) #🧩
            #📃 Validate if the columns in the DataFrame match the defined schema
            has_critical_error, critical_schema_error_msg = validate_schema(Quality, Quality_schema, file_basename) #🧩
            if has_critical_error: # If critical columns are missing.
                raise ValueError(critical_schema_error_msg) # Raise an error to stop processing this file.
            #📃 Adjust the column structure.
            print_colored(f"⚙️Adjust column structure", "Olive")
            logger.warning(f"⚙️Adjust column structure")
            Quality = Quality.with_columns(pl.col("eval_date").cast(pl.Date)) #🧩
            Quality = Quality.with_columns(pl.col("score_n","score_question_weight").cast(pl.Int64)) #🧩
            Quality = Quality.select(Quality_schema) #🧩 Select columns in the correct order and names as per df_schema.
            print_colored(f"⚙️Column structure adjustment completed", "Olive")
            logger.info(f"⚙️Column structure adjustment completed")
            info_polars(Quality) #🧩 Display information about the DataFrame (shape, columns, dtypes).
            #📃 Start writing data to the database.
            print_colored(f"💾Start import file: {file_basename}", "Coral")
            logger.info(f"💾Start import file: {file_basename}")
            write_data(engine, Quality_TABLE_NAME, Quality) #🧩 Call the function to write the DataFrame to SQL.
            print_colored(f"Successfully imported✨: {file_basename}", "LimeGreen")
            logger.info(f"Successfully imported✨: {file_basename}")
            #📃 Record to summary
            if is_new_file:
                summary_results["new_files_imported"].append(f"[Quality] {file_basename}")#🧩
            else:
                summary_results["updated_files_reimported"].append(f"[Quality] {file_basename}")#🧩          
            #📃 Update log_entries for successfully import: Add information to the log list for this run (successful).
            log_entries.append({"FileName": file_basename, "ModifiedDate": current_modified_date, "Error": None})
            print_colored(f"Updated 'Modified' time for {file_basename}", "LimeGreen")
            logger.info(f"Updated 'Modified' time for {file_basename}")
            #📃 Handle exceptions if any error occurred in the try block.
        except (
            FileNotFoundError, PermissionError, UnicodeDecodeError, IOError,                
            pl.exceptions.NoDataError, pl.exceptions.ComputeError, pl.exceptions.SchemaError, sa.exc.SQLAlchemyError,
            Exception # Catch various types of potential errors.
        ) as e:
            error_count += 1 # Increment the error counter.        
            error_msg_short = f"{type(e).__name__}: {str(e).splitlines()[0]}"  # Get a short error message (the first line).
            #📃 Update Failed file to Summary
            summary_results["failed_imports"].append((f"[Quality] {file_basename}", error_msg_short))#🧩           
            logger.error(f"Error importing {file_basename}: {error_msg_short}", exc_info=True) # Log the full error traceback.
            print_colored(f"❗️Error importing {file_basename}: {error_msg_short}", "OrangeRed")
            # Add information to the log list for this run (failed).
            # Note: ModifiedDate here is previous_modified_date, as this file wasn't successfully imported with current_modified_date.
            log_entries.append({"FileName": file_basename,"ModifiedDate": previous_modified_date,"Error": error_msg_short})
    else: #📃 If the file is not new and its modification date hasn't changed.
          #📃 Update Skip file to Summary
        summary_results["skipped_files_unchanged"].append(f"[Quality] {file_basename}")#🧩
        print_colored(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.", "Aquamarine")
        logger.info(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.")
#💡 After iterating through all files in the directory.
display_summary("Quality", error_count) #🧩 Display a summary of the results (number of errors).
process_and_save_log(log_df, log_entries, log_Quality_path) #🧩 Update the Excel log file with information
logger.info("===== Processing of the Quality data source is complete =====") #🧩 Log finish data processing.

In [ ]:
# 1️⃣6️⃣[BKN]RONA💾
logger.info("===== Start RONA Process =====")  #🧩 Log the start of the data processing.
log_df = read_or_create_log(log_RONA_path) #🧩 Read the existing Excel log file or create a new one if it doesn't exist.
log_entries, error_count = Default_variable() #📃 Initialize a list to store log results for the current run and an error counter.
#💡 Iterate through all files in the specified folder.
for filename in Folder_RONA.glob("*.xlsx"): #🧩 Get files in the Folder directory.
    #📃 Get the current modification timestamp and filename of the current file.
    current_modified_date_microseconds = datetime.datetime.fromtimestamp(filename.stat().st_mtime) # Get modification timestamp, including microseconds.
    current_modified_date = current_modified_date_microseconds.replace(microsecond=0) # Remove microseconds for easier comparison.
    file_basename = filename.name # Get the base name of the file (e.g., "data_20230101.csv").
    #📃 File is being processed.
    print_colored(f"Processing file🔄️: {file_basename}", "NavajoWhite")
    logger.info(f"Processing file🔄️: {file_basename}")
    #📃 Check if this file has been logged before.
    log_entry = log_df.filter(pl.col("FileName") == file_basename) # Filter the old log_df to find an entry for this filename.
    is_new_file = log_entry.is_empty() # If no entry is found (log_entry is empty), it's a new file.
    previous_modified_date = datetime.datetime(1900, 1, 1, 0, 0, 1)  # Initialize the previous modification date with a very old date.
    #📃 If this file was processed before (exists in the log).  
    if not is_new_file:
        previous_modified_date = log_entry.select("ModifiedDate").item() # Get the modification date recorded in the log from the previous processing.
    #📃 Condition to process the file:
    # 1. The file is new (not in the log).
    # 2. Or, the file is old (in the log), but its current modification date (current_modified_date) is more recent than the modification date recorded in the log (previous_modified_date).
    if is_new_file or current_modified_date > previous_modified_date:
        if not is_new_file: # If it's an old file but has changed (ModifiedDate is newer).
            print_colored(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.", "Coral")
            logger.warning(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.")
        else: # If it's a completely new file.
            print_colored(f"New file detected🚦: {file_basename}. Importing.", "Coral")
            logger.info(f"New file detected🚦: {file_basename}. Importing.")
        try: # Start a try-except block to handle potential errors during the import process.
            if not is_new_file: # If this file already exists in the DB (because it's not new and its ModifiedDate changed) then delete the old data for this file from the df.
                delete_data(engine, RONA_TABLE_NAME, file_basename)  #🧩 Call the function to delete data.
            #📃 Read data from the excel file using Polars.
            RONA = (pl.read_excel(filename, infer_schema_length=None, engine="calamine", sheet_name="HCM_RONA-Agent Team Historical ") #🧩 Read the entire file to infer the schema.Add two new columns to the DataFrame: FileName and ModifiedDate.
                   .select(pl.all(),pl.lit(file_basename).alias("FileName"),pl.lit(current_modified_date).dt.cast_time_unit("ms").alias("ModifiedDate")
                   )) #🧩
            #📃 Validate if the columns in the DataFrame match the defined schema
            has_critical_error, critical_schema_error_msg = validate_schema(RONA, RONA_schema, file_basename) #🧩
            if has_critical_error: # If critical columns are missing.
                raise ValueError(critical_schema_error_msg) # Raise an error to stop processing this file.
            #📃 Adjust the column structure.
            print_colored(f"⚙️Adjust column structure", "Olive")
            logger.warning(f"⚙️Adjust column structure")
            RONA = RONA.select(RONA_schema) #🧩 Select columns in the correct order and names as per df_schema.
            print_colored(f"⚙️Column structure adjustment completed", "Olive")
            logger.info(f"⚙️Column structure adjustment completed")
            info_polars(RONA) #🧩 Display information about the DataFrame (shape, columns, dtypes).
            #📃 Start writing data to the database.
            print_colored(f"💾Start import file: {file_basename}", "Coral")
            logger.info(f"💾Start import file: {file_basename}")
            write_data(engine, RONA_TABLE_NAME, RONA) #🧩 Call the function to write the DataFrame to SQL.
            print_colored(f"Successfully imported✨: {file_basename}", "LimeGreen")
            logger.info(f"Successfully imported✨: {file_basename}")
            #📃 Record to summary
            if is_new_file:
                summary_results["new_files_imported"].append(f"[RONA] {file_basename}")#🧩
            else:
                summary_results["updated_files_reimported"].append(f"[RONA] {file_basename}")#🧩    
            #📃 Update log_entries for successfully import: Add information to the log list for this run (successful).
            log_entries.append({"FileName": file_basename, "ModifiedDate": current_modified_date, "Error": None})
            print_colored(f"Updated 'Modified' time for {file_basename}", "LimeGreen")
            logger.info(f"Updated 'Modified' time for {file_basename}")
            #📃 Handle exceptions if any error occurred in the try block.
        except (
            FileNotFoundError, PermissionError, UnicodeDecodeError, IOError,                
            pl.exceptions.NoDataError, pl.exceptions.ComputeError, pl.exceptions.SchemaError, sa.exc.SQLAlchemyError,
            Exception # Catch various types of potential errors.
        ) as e:
            error_count += 1 # Increment the error counter.        
            error_msg_short = f"{type(e).__name__}: {str(e).splitlines()[0]}"  # Get a short error message (the first line).
            #📃 Update Failed file to Summary
            summary_results["failed_imports"].append((f"[RONA] {file_basename}", error_msg_short))#🧩
            logger.error(f"Error importing {file_basename}: {error_msg_short}", exc_info=True) # Log the full error traceback.
            print_colored(f"❗️Error importing {file_basename}: {error_msg_short}", "OrangeRed")
            # Add information to the log list for this run (failed).
            # Note: ModifiedDate here is previous_modified_date, as this file wasn't successfully imported with current_modified_date.
            log_entries.append({"FileName": file_basename,"ModifiedDate": previous_modified_date,"Error": error_msg_short})
    else: #📃 If the file is not new and its modification date hasn't changed.
          #📃 Update Skip file to Summary
        summary_results["skipped_files_unchanged"].append(f"[RONA] {file_basename}")#🧩
        print_colored(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.", "Aquamarine")
        logger.info(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.")
#💡 After iterating through all files in the directory.
display_summary("RONA", error_count) #🧩 Display a summary of the results (number of errors).
process_and_save_log(log_df, log_entries, log_RONA_path) #🧩 Update the Excel log file with information
logger.info("===== Processing of the RONA data source is complete =====") #🧩 Log finish data processing.

In [ ]:
# 1️⃣7️⃣[BKN]CUIC💾
logger.info("===== Start CUIC Process =====")  #🧩 Log the start of the data processing.
log_df = read_or_create_log(log_CUIC_path) #🧩 Read the existing Excel log file or create a new one if it doesn't exist.
log_entries, error_count = Default_variable() #📃 Initialize a list to store log results for the current run and an error counter.
#💡 Iterate through all files in the specified folder.
for filename in Folder_CUIC.glob("*.xlsx"): #🧩 Get files in the Folder directory.
    #📃 Get the current modification timestamp and filename of the current file.
    current_modified_date_microseconds = datetime.datetime.fromtimestamp(filename.stat().st_mtime) # Get modification timestamp, including microseconds.
    current_modified_date = current_modified_date_microseconds.replace(microsecond=0) # Remove microseconds for easier comparison.
    file_basename = filename.name # Get the base name of the file (e.g., "data_20230101.csv").
    #📃 File is being processed.
    print_colored(f"Processing file🔄️: {file_basename}", "NavajoWhite")
    logger.info(f"Processing file🔄️: {file_basename}")
    #📃 Check if this file has been logged before.
    log_entry = log_df.filter(pl.col("FileName") == file_basename) # Filter the old log_df to find an entry for this filename.
    is_new_file = log_entry.is_empty() # If no entry is found (log_entry is empty), it's a new file.
    previous_modified_date = datetime.datetime(1900, 1, 1, 0, 0, 1)  # Initialize the previous modification date with a very old date.
    #📃 If this file was processed before (exists in the log).  
    if not is_new_file:
        previous_modified_date = log_entry.select("ModifiedDate").item() # Get the modification date recorded in the log from the previous processing.
    #📃 Condition to process the file:
    # 1. The file is new (not in the log).
    # 2. Or, the file is old (in the log), but its current modification date (current_modified_date) is more recent than the modification date recorded in the log (previous_modified_date).
    if is_new_file or current_modified_date > previous_modified_date:
        if not is_new_file: # If it's an old file but has changed (ModifiedDate is newer).
            print_colored(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.", "Coral")
            logger.warning(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.")
        else: # If it's a completely new file.
            print_colored(f"New file detected🚦: {file_basename}. Importing.", "Coral")
            logger.info(f"New file detected🚦: {file_basename}. Importing.")
        try: # Start a try-except block to handle potential errors during the import process.
            if not is_new_file: # If this file already exists in the DB (because it's not new and its ModifiedDate changed) then delete the old data for this file from the df.
                delete_data(engine, CUIC_TABLE_NAME, file_basename)  #🧩 Call the function to delete data.
            #📃 Read data from the excel file using Polars.
            CUIC = (pl.read_excel(filename, infer_schema_length=None, engine="calamine", sheet_name="Attendance Check-Sample_Attenda") #🧩 Read the entire file to infer the schema.Add two new columns to the DataFrame: FileName and ModifiedDate.
                    .select(pl.all(),pl.lit(file_basename).alias("FileName"),pl.lit(current_modified_date).dt.cast_time_unit("ms").alias("ModifiedDate")
                   )) #🧩
            #📃 Validate if the columns in the DataFrame match the defined schema
            has_critical_error, critical_schema_error_msg = validate_schema(CUIC, CUIC_schema, file_basename) #🧩
            if has_critical_error: # If critical columns are missing.
                raise ValueError(critical_schema_error_msg) # Raise an error to stop processing this file.
            #📃 Adjust the column structure.
            print_colored(f"⚙️Adjust column structure", "Olive")
            logger.warning(f"⚙️Adjust column structure")
            CUIC = CUIC.with_columns(pl.col("Interval").str.strptime(pl.Datetime, "%m/%d/%y %I:%M:%S %p")) #🧩
            CUIC = CUIC.with_columns(pl.col("AgentAvailTime", "AgentLoggedOnTime").cast(pl.Float64)) #🧩
            CUIC = CUIC.select(CUIC_schema) #🧩 Select columns in the correct order and names as per df_schema.
            print_colored(f"⚙️Column structure adjustment completed", "Olive")
            logger.info(f"⚙️Column structure adjustment completed")
            info_polars(CUIC) #🧩 Display information about the DataFrame (shape, columns, dtypes).
            #📃 Start writing data to the database.
            print_colored(f"💾Start import file: {file_basename}", "Coral")
            logger.info(f"💾Start import file: {file_basename}")
            write_data(engine, CUIC_TABLE_NAME, CUIC) #🧩 Call the function to write the DataFrame to SQL.
            print_colored(f"Successfully imported✨: {file_basename}", "LimeGreen")
            logger.info(f"Successfully imported✨: {file_basename}")
            #📃 Record to summary
            if is_new_file:
                summary_results["new_files_imported"].append(f"[CUIC] {file_basename}")#🧩
            else:
                summary_results["updated_files_reimported"].append(f"[CUIC] {file_basename}")#🧩 
            #📃 Update log_entries for successfully import: Add information to the log list for this run (successful).
            log_entries.append({"FileName": file_basename, "ModifiedDate": current_modified_date, "Error": None})
            print_colored(f"Updated 'Modified' time for {file_basename}", "LimeGreen")
            logger.info(f"Updated 'Modified' time for {file_basename}")
            #📃 Handle exceptions if any error occurred in the try block.
        except (
            FileNotFoundError, PermissionError, UnicodeDecodeError, IOError,                
            pl.exceptions.NoDataError, pl.exceptions.ComputeError, pl.exceptions.SchemaError, sa.exc.SQLAlchemyError,
            Exception # Catch various types of potential errors.
        ) as e:
            error_count += 1 # Increment the error counter.        
            error_msg_short = f"{type(e).__name__}: {str(e).splitlines()[0]}"  # Get a short error message (the first line).
            #📃 Update Failed file to Summary
            summary_results["failed_imports"].append((f"[CUIC] {file_basename}", error_msg_short))#🧩       
            logger.error(f"Error importing {file_basename}: {error_msg_short}", exc_info=True) # Log the full error traceback.
            print_colored(f"❗️Error importing {file_basename}: {error_msg_short}", "OrangeRed")
            # Add information to the log list for this run (failed).
            # Note: ModifiedDate here is previous_modified_date, as this file wasn't successfully imported with current_modified_date.
            log_entries.append({"FileName": file_basename,"ModifiedDate": previous_modified_date,"Error": error_msg_short})
    else: #📃 If the file is not new and its modification date hasn't changed.
          #📃 Update Skip file to Summary
        summary_results["skipped_files_unchanged"].append(f"[CUIC] {file_basename}")#🧩
        print_colored(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.", "Aquamarine")
        logger.info(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.")
#💡 After iterating through all files in the directory.
display_summary("CUIC", error_count) #🧩 Display a summary of the results (number of errors).
process_and_save_log(log_df, log_entries, log_CUIC_path) #🧩 Update the Excel log file with information
logger.info("===== Processing of the CUIC data source is complete =====") #🧩 Log finish data processing.

In [ ]:
# 1️⃣8️⃣[BKN]KPI_Target💾
logger.info("===== Start KPI_Target Process =====")  #🧩 Log the start of the data processing.
log_df = read_or_create_log(log_KPI_Target_path) #🧩 Read the existing Excel log file or create a new one if it doesn't exist.
log_entries, error_count = Default_variable() #📃 Initialize a list to store log results for the current run and an error counter.
#💡 Iterate through all files in the specified folder.
for filename in Folder_KPI_Target.glob("*.xlsx"): #🧩 Get files in the Folder directory.
    #📃 Get the current modification timestamp and filename of the current file.
    current_modified_date_microseconds = datetime.datetime.fromtimestamp(filename.stat().st_mtime) # Get modification timestamp, including microseconds.
    current_modified_date = current_modified_date_microseconds.replace(microsecond=0) # Remove microseconds for easier comparison.
    file_basename = filename.name # Get the base name of the file (e.g., "data_20230101.csv").
    #📃 File is being processed.
    print_colored(f"Processing file🔄️: {file_basename}", "NavajoWhite")
    logger.info(f"Processing file🔄️: {file_basename}")
    #📃 Check if this file has been logged before.
    log_entry = log_df.filter(pl.col("FileName") == file_basename) # Filter the old log_df to find an entry for this filename.
    is_new_file = log_entry.is_empty() # If no entry is found (log_entry is empty), it's a new file.
    previous_modified_date = datetime.datetime(1900, 1, 1, 0, 0, 1)  # Initialize the previous modification date with a very old date.
    #📃 If this file was processed before (exists in the log).  
    if not is_new_file:
        previous_modified_date = log_entry.select("ModifiedDate").item() # Get the modification date recorded in the log from the previous processing.
    #📃 Condition to process the file:
    # 1. The file is new (not in the log).
    # 2. Or, the file is old (in the log), but its current modification date (current_modified_date) is more recent than the modification date recorded in the log (previous_modified_date).
    if is_new_file or current_modified_date > previous_modified_date:
        if not is_new_file: # If it's an old file but has changed (ModifiedDate is newer).
            print_colored(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.", "Coral")
            logger.warning(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.")
        else: # If it's a completely new file.
            print_colored(f"New file detected🚦: {file_basename}. Importing.", "Coral")
            logger.info(f"New file detected🚦: {file_basename}. Importing.")
        try: # Start a try-except block to handle potential errors during the import process.
            if not is_new_file: # If this file already exists in the DB (because it's not new and its ModifiedDate changed) then delete the old data for this file from the df.
                delete_data(engine, KPI_Target_TABLE_NAME, file_basename)  #🧩 Call the function to delete data.
            #📃 Read data from the excel file using Polars.
            KPI_Target = (pl.read_excel(filename, infer_schema_length=None, engine="calamine", sheet_name="Sheet1") #🧩 Read the entire file to infer the schema.Add two new columns to the DataFrame: FileName and ModifiedDate.
                   .select(pl.all(),pl.lit(file_basename).alias("FileName"),pl.lit(current_modified_date).dt.cast_time_unit("ms").alias("ModifiedDate")
                   )) #🧩
            #📃 Validate if the columns in the DataFrame match the defined schema
            has_critical_error, critical_schema_error_msg = validate_schema(KPI_Target, KPI_Target_schema, file_basename) #🧩
            if has_critical_error: # If critical columns are missing.
                raise ValueError(critical_schema_error_msg) # Raise an error to stop processing this file.
            #📃 Adjust the column structure.
            print_colored(f"⚙️Adjust column structure", "Olive")
            logger.warning(f"⚙️Adjust column structure")
            KPI_Target = KPI_Target.with_columns(pl.col("Week").cast(pl.Int64)) #🧩
            KPI_Target = KPI_Target.with_columns(pl.col('Overall CPH tar', 'Phone CPH tar', 'Non Phone CPH tar', 'Quality - Customer Impact tar', 
                                                        'Quality - Business Impact tar', 'Quality - Compliance Impact tar', 'Quality - Overall tar', 
                                                        'AHT Phone tar', 'AHT Non-phone tar', 'AHT Overall tar', 'Hold (phone) tar', 'AACW (phone) tar', 
                                                        'Avg Talk Time tar', 'Phone CSAT tar', 'Non phone CSAT tar', 'Overall CSAT tar', 'PSAT tar', 
                                                        'PSAT Vietnamese tar', 'PSAT English (American) tar', 'PSAT English (Great Britain) tar', 'CSAT Reso tar').cast(pl.Float64)) # Import Schema🧩
            KPI_Target = KPI_Target.select(KPI_Target_schema) #🧩 Select columns in the correct order and names as per df_schema.
            print_colored(f"⚙️Column structure adjustment completed", "Olive")
            logger.info(f"⚙️Column structure adjustment completed")
            info_polars(KPI_Target) #🧩 Display information about the DataFrame (shape, columns, dtypes).
            #📃 Start writing data to the database.
            print_colored(f"💾Start import file: {file_basename}", "Coral")
            logger.info(f"💾Start import file: {file_basename}")
            write_data(engine, KPI_Target_TABLE_NAME, KPI_Target) #🧩 Call the function to write the DataFrame to SQL.
            print_colored(f"Successfully imported✨: {file_basename}", "LimeGreen")
            logger.info(f"Successfully imported✨: {file_basename}")
            #📃 Record to summary
            if is_new_file:
                summary_results["new_files_imported"].append(f"[KPI_Target] {file_basename}")#🧩
            else:
                summary_results["updated_files_reimported"].append(f"[KPI_Target] {file_basename}")#🧩 
            #📃 Update log_entries for successfully import: Add information to the log list for this run (successful).
            log_entries.append({"FileName": file_basename, "ModifiedDate": current_modified_date, "Error": None})
            print_colored(f"Updated 'Modified' time for {file_basename}", "LimeGreen")
            logger.info(f"Updated 'Modified' time for {file_basename}")
            #📃 Handle exceptions if any error occurred in the try block.
        except (
            FileNotFoundError, PermissionError, UnicodeDecodeError, IOError,                
            pl.exceptions.NoDataError, pl.exceptions.ComputeError, pl.exceptions.SchemaError, sa.exc.SQLAlchemyError,
            Exception # Catch various types of potential errors.
        ) as e:
            error_count += 1 # Increment the error counter.        
            error_msg_short = f"{type(e).__name__}: {str(e).splitlines()[0]}"  # Get a short error message (the first line).
            #📃 Update Failed file to Summary
            summary_results["failed_imports"].append((f"[KPI_Target] {file_basename}", error_msg_short))#🧩       
            logger.error(f"Error importing {file_basename}: {error_msg_short}", exc_info=True) # Log the full error traceback.
            print_colored(f"❗️Error importing {file_basename}: {error_msg_short}", "OrangeRed")
            # Add information to the log list for this run (failed).
            # Note: ModifiedDate here is previous_modified_date, as this file wasn't successfully imported with current_modified_date.
            log_entries.append({"FileName": file_basename,"ModifiedDate": previous_modified_date,"Error": error_msg_short})
    else: #📃 If the file is not new and its modification date hasn't changed.
          #📃 Update Skip file to Summary
        summary_results["skipped_files_unchanged"].append(f"[KPI_Target] {file_basename}")#🧩
        print_colored(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.", "Aquamarine")
        logger.info(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.")
#💡 After iterating through all files in the directory.
display_summary("KPI_Target", error_count) #🧩 Display a summary of the results (number of errors).
process_and_save_log(log_df, log_entries, log_KPI_Target_path) #🧩 Update the Excel log file with information
logger.info("===== Processing of the KPI_Target data source is complete =====") #🧩 Log finish data processing.

In [ ]:
# 1️⃣9️⃣[BKN]LogoutCount💾
logger.info("===== Start LogoutCount Process =====")  #🧩 Log the start of the data processing.
log_df = read_or_create_log(log_LogoutCount_path) #🧩 Read the existing Excel log file or create a new one if it doesn't exist.
log_entries, error_count = Default_variable() #📃 Initialize a list to store log results for the current run and an error counter.
#💡 Iterate through all files in the specified folder.
for filename in Folder_LogoutCount.glob("*.csv"): #🧩 Get files in the Folder directory.
    #📃 Get the current modification timestamp and filename of the current file.
    current_modified_date_microseconds = datetime.datetime.fromtimestamp(filename.stat().st_mtime) # Get modification timestamp, including microseconds.
    current_modified_date = current_modified_date_microseconds.replace(microsecond=0) # Remove microseconds for easier comparison.
    file_basename = filename.name # Get the base name of the file (e.g., "data_20230101.csv").
    #📃 File is being processed.
    print_colored(f"Processing file🔄️: {file_basename}", "NavajoWhite")
    logger.info(f"Processing file🔄️: {file_basename}")
    #📃 Check if this file has been logged before.
    log_entry = log_df.filter(pl.col("FileName") == file_basename) # Filter the old log_df to find an entry for this filename.
    is_new_file = log_entry.is_empty() # If no entry is found (log_entry is empty), it's a new file.
    previous_modified_date = datetime.datetime(1900, 1, 1, 0, 0, 1)  # Initialize the previous modification date with a very old date.
    #📃 If this file was processed before (exists in the log).  
    if not is_new_file:
        previous_modified_date = log_entry.select("ModifiedDate").item() # Get the modification date recorded in the log from the previous processing.
    #📃 Condition to process the file:
    # 1. The file is new (not in the log).
    # 2. Or, the file is old (in the log), but its current modification date (current_modified_date) is more recent than the modification date recorded in the log (previous_modified_date).
    if is_new_file or current_modified_date > previous_modified_date:
        if not is_new_file: # If it's an old file but has changed (ModifiedDate is newer).
            print_colored(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.", "Coral")
            logger.warning(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.")
        else: # If it's a completely new file.
            print_colored(f"New file detected🚦: {file_basename}. Importing.", "Coral")
            logger.info(f"New file detected🚦: {file_basename}. Importing.")
        try: # Start a try-except block to handle potential errors during the import process.
            if not is_new_file: # If this file already exists in the DB (because it's not new and its ModifiedDate changed) then delete the old data for this file from the df.
                delete_data(engine, LogoutCount_TABLE_NAME, file_basename)  #🧩 Call the function to delete data.
            #📃 Read data from the excel file using Polars.
            LogoutCount = (pl.read_csv(filename, infer_schema_length=None) #🧩 Read the entire file to infer the schema.Add two new columns to the DataFrame: FileName and ModifiedDate.
                   .select(pl.all(),pl.lit(file_basename).alias("FileName"),pl.lit(current_modified_date).dt.cast_time_unit("ms").alias("ModifiedDate")
                   )) #🧩
            #📃 Validate if the columns in the DataFrame match the defined schema
            has_critical_error, critical_schema_error_msg = validate_schema(LogoutCount, LogoutCount_schema, file_basename) #🧩
            if has_critical_error: # If critical columns are missing.
                raise ValueError(critical_schema_error_msg) # Raise an error to stop processing this file.
            #📃 Adjust the column structure.
            print_colored(f"⚙️Adjust column structure", "Olive")
            logger.warning(f"⚙️Adjust column structure")
            LogoutCount = LogoutCount.with_columns(pl.col('TimeDimension').str.strptime(pl.Date, format='%m/%d/%Y'),
                                                   pl.col('KPI Value Formatted').cast(pl.Int64)) #🧩
            LogoutCount = LogoutCount.select(LogoutCount_schema) #🧩 Select columns in the correct order and names as per df_schema.
            print_colored(f"⚙️Column structure adjustment completed", "Olive")
            logger.info(f"⚙️Column structure adjustment completed")
            info_polars(LogoutCount) #🧩 Display information about the DataFrame (shape, columns, dtypes).
            #📃 Start writing data to the database.
            print_colored(f"💾Start import file: {file_basename}", "Coral")
            logger.info(f"💾Start import file: {file_basename}")
            write_data(engine, LogoutCount_TABLE_NAME, LogoutCount) #🧩 Call the function to write the DataFrame to SQL.
            print_colored(f"Successfully imported✨: {file_basename}", "LimeGreen")
            logger.info(f"Successfully imported✨: {file_basename}")
            #📃 Record to summary
            if is_new_file:
                summary_results["new_files_imported"].append(f"[LogoutCount] {file_basename}")#🧩
            else:
                summary_results["updated_files_reimported"].append(f"[LogoutCount] {file_basename}")#🧩       
            #📃 Update log_entries for successfully import: Add information to the log list for this run (successful).
            log_entries.append({"FileName": file_basename, "ModifiedDate": current_modified_date, "Error": None})
            print_colored(f"Updated 'Modified' time for {file_basename}", "LimeGreen")
            logger.info(f"Updated 'Modified' time for {file_basename}")
            #📃 Handle exceptions if any error occurred in the try block.
        except (
            FileNotFoundError, PermissionError, UnicodeDecodeError, IOError,                
            pl.exceptions.NoDataError, pl.exceptions.ComputeError, pl.exceptions.SchemaError, sa.exc.SQLAlchemyError,
            Exception # Catch various types of potential errors.
        ) as e:
            error_count += 1 # Increment the error counter.        
            error_msg_short = f"{type(e).__name__}: {str(e).splitlines()[0]}"  # Get a short error message (the first line).
            #📃 Update Failed file to Summary
            summary_results["failed_imports"].append((f"[LogoutCount] {file_basename}", error_msg_short))#🧩
            logger.error(f"Error importing {file_basename}: {error_msg_short}", exc_info=True) # Log the full error traceback.
            print_colored(f"❗️Error importing {file_basename}: {error_msg_short}", "OrangeRed")
            # Add information to the log list for this run (failed).
            # Note: ModifiedDate here is previous_modified_date, as this file wasn't successfully imported with current_modified_date.
            log_entries.append({"FileName": file_basename,"ModifiedDate": previous_modified_date,"Error": error_msg_short})
    else: #📃 If the file is not new and its modification date hasn't changed.
          #📃 Update Skip file to Summary
        summary_results["skipped_files_unchanged"].append(f"[LogoutCount] {file_basename}")#🧩 
        print_colored(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.", "Aquamarine")
        logger.info(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.")
#💡 After iterating through all files in the directory.
display_summary("LogoutCount", error_count) #🧩 Display a summary of the results (number of errors).
process_and_save_log(log_df, log_entries, log_LogoutCount_path) #🧩 Update the Excel log file with information
logger.info("===== Processing of the LogoutCount data source is complete =====") #🧩 Log finish data processing.

In [ ]:
# 2️⃣0️⃣[BKN]WpDetail💾
logger.info("===== Start WpDetail Process =====")  #🧩 Log the start of the data processing.
log_df = read_or_create_log(log_WpDetail_path) #🧩 Read the existing Excel log file or create a new one if it doesn't exist.
log_entries, error_count = Default_variable() #📃 Initialize a list to store log results for the current run and an error counter.
#💡 Iterate through all files in the specified folder.
for filename in Folder_WpDetail.glob("*.xlsx"): #🧩 Get files in the Folder directory.
    #📃 Get the current modification timestamp and filename of the current file.
    current_modified_date_microseconds = datetime.datetime.fromtimestamp(filename.stat().st_mtime) # Get modification timestamp, including microseconds.
    current_modified_date = current_modified_date_microseconds.replace(microsecond=0) # Remove microseconds for easier comparison.
    file_basename = filename.name # Get the base name of the file (e.g., "data_20230101.csv").
    #📃 File is being processed.
    print_colored(f"Processing file🔄️: {file_basename}", "NavajoWhite")
    logger.info(f"Processing file🔄️: {file_basename}")
    #📃 Check if this file has been logged before.
    log_entry = log_df.filter(pl.col("FileName") == file_basename) # Filter the old log_df to find an entry for this filename.
    is_new_file = log_entry.is_empty() # If no entry is found (log_entry is empty), it's a new file.
    previous_modified_date = datetime.datetime(1900, 1, 1, 0, 0, 1)  # Initialize the previous modification date with a very old date.
    #📃 If this file was processed before (exists in the log).  
    if not is_new_file:
        previous_modified_date = log_entry.select("ModifiedDate").item() # Get the modification date recorded in the log from the previous processing.
    #📃 Condition to process the file:
    # 1. The file is new (not in the log).
    # 2. Or, the file is old (in the log), but its current modification date (current_modified_date) is more recent than the modification date recorded in the log (previous_modified_date).
    if is_new_file or current_modified_date > previous_modified_date:
        if not is_new_file: # If it's an old file but has changed (ModifiedDate is newer).
            print_colored(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.", "Coral")
            logger.warning(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.")
        else: # If it's a completely new file.
            print_colored(f"New file detected🚦: {file_basename}. Importing.", "Coral")
            logger.info(f"New file detected🚦: {file_basename}. Importing.")
        try: # Start a try-except block to handle potential errors during the import process.
            if not is_new_file: # If this file already exists in the DB (because it's not new and its ModifiedDate changed) then delete the old data for this file from the df.
                delete_data(engine, WpDetail_TABLE_NAME, file_basename)  #🧩 Call the function to delete data.
            #📃 Read data from the excel file using Polars.
            WpDetail = (pl.read_excel(filename, infer_schema_length=None, engine="calamine", sheet_name="Sheet1") #🧩 Read the entire file to infer the schema.Add two new columns to the DataFrame: FileName and ModifiedDate.
                   .select(pl.all(),pl.lit(file_basename).alias("FileName"),
                           pl.lit(current_modified_date).dt.cast_time_unit("ms").alias("ModifiedDate")
                   ))
            time_cols = ['Start1', 'End1', 'Start2', 'End2']
            expressions = []      
            for col_name in time_cols:
                # Biểu thức này sẽ thử nhiều cách để chuyển đổi và lấy kết quả hợp lệ đầu tiên
                # 1. Thử chuyển đổi chuỗi thời gian có định dạng 12-giờ (AM/PM)
                # 2. Nếu thất bại, thử ép kiểu trực tiếp (dành cho định dạng 24h hoặc các kiểu khác)
                expression = pl.coalesce(
                    pl.col(col_name).str.strptime(pl.Time, format="%I:%M:%S %p", strict=False),
                    pl.col(col_name).cast(pl.Time, strict=False)
                ).alias(col_name)
                expressions.append(expression)      
            # Apply expressions
            WpDetail = WpDetail.with_columns(
                pl.col('Date').cast(pl.Date),
                *expressions
            )                    
            threshold = t(15, 0, 0) # Time threshold definition (15:00:00)
            WpDetail = WpDetail.with_columns( # create Date_end
                pl.when(pl.col("Start1").cast(pl.Time) >= threshold)
                    .then(pl.col("Date").cast(pl.Date) + timedelta(days=1)) 
                    .otherwise(pl.col("Date").cast(pl.Date)).alias("Date_end"))
            WpDetail = WpDetail.with_columns( # create DateTime_Start & DateTime_End
                (pl.col("Date").dt.strftime("%Y-%m-%d") + " " + pl.col("Start1").cast(str)).str.strptime(pl.Datetime, "%Y-%m-%d %H:%M:%S").alias("DateTime_Start"),
                (pl.col("Date_end").dt.strftime("%Y-%m-%d") + " " + pl.col("End1").cast(str)).str.strptime(pl.Datetime, "%Y-%m-%d %H:%M:%S").alias("DateTime_End"))
            WpDetail = WpDetail.filter(pl.col("Start2").is_not_null() & pl.col("End2").is_not_null()) # Filter for non-null values
            WpDetail = WpDetail.with_columns( # Create 'Date_Act_Start'
                pl.when((pl.col("Start1") >= t(15, 0, 0)) & (pl.col("Start2") <= t(15, 0, 0)))
                .then(pl.col("Date") + pl.duration(days=1))  # Directly add a duration to the Date column
                .otherwise(pl.col("Date")).alias("Date_Act_Start"))
            WpDetail = WpDetail.with_columns( # Create 'DateTime_Act_Start' column
                (pl.col("Date_Act_Start").dt.strftime("%Y-%m-%d") + " " + pl.col("Start2").cast(str))
                .str.strptime(pl.Datetime, "%Y-%m-%d %H:%M:%S").alias("DateTime_Act_Start"))
            WpDetail = WpDetail.with_columns( # Create 'Date_Act_End' column
                pl.when((pl.col("Start1") >= t(15, 0, 0)) & (pl.col("End2") <= t(15, 0, 0)))
                .then(pl.col("Date") + pl.duration(days=1))  # Add duration directly to Date column
                .otherwise(pl.col("Date")).alias("Date_Act_End"))
            WpDetail = WpDetail.with_columns( # Create 'DateTime_Act_End' column
                (pl.col("Date_Act_End").dt.strftime("%Y-%m-%d") + " " + pl.col("End2").cast(str))
                .str.strptime(pl.Datetime, "%Y-%m-%d %H:%M:%S").alias("DateTime_Act_End"))
            WpDetail = WpDetail.with_columns(( # Convert data and calculate Dur (Hrs)
                (pl.col("DateTime_End") - pl.col("DateTime_Start")).dt.total_seconds() / 3600).alias("Dur"))
            WpDetail = WpDetail.with_columns(( # Convert data and calculate Act_Dur (Hrs)
                (pl.col("DateTime_Act_End") - pl.col("DateTime_Act_Start")).dt.total_seconds() / 3600).alias("Act_Dur"))
            WpDetail = WpDetail.rename({'Date':'Date_Start','Start1':'Time_Start','End1':'Time_End','Schedule Act':'Action',
                                        'Start2':'Time_Act_Start','End2':'Time_Act_End'}) #🧩
            #📃 Validate if the columns in the DataFrame match the defined schema
            has_critical_error, critical_schema_error_msg = validate_schema(WpDetail, WpDetail_schema, file_basename) #🧩
            if has_critical_error: # If critical columns are missing.
                raise ValueError(critical_schema_error_msg) # Raise an error to stop processing this file.
            #📃 Adjust the column structure.
            print_colored(f"⚙️Adjust column structure", "Olive")
            logger.warning(f"⚙️Adjust column structure")
            WpDetail = WpDetail.select(WpDetail_schema) #🧩 Select columns in the correct order and names as per df_schema.
            print_colored(f"⚙️Column structure adjustment completed", "Olive")
            logger.info(f"⚙️Column structure adjustment completed")
            info_polars(WpDetail) #🧩 Display information about the DataFrame (shape, columns, dtypes).
            #📃 Start writing data to the database.
            print_colored(f"💾Start import file: {file_basename}", "Coral")
            logger.info(f"💾Start import file: {file_basename}")
            write_data(engine, WpDetail_TABLE_NAME, WpDetail) #🧩 Call the function to write the DataFrame to SQL.
            print_colored(f"Successfully imported✨: {file_basename}", "LimeGreen")
            logger.info(f"Successfully imported✨: {file_basename}")
            #📃 Record to summary
            if is_new_file:
                summary_results["new_files_imported"].append(f"[WpDetail] {file_basename}")#🧩
            else:
                summary_results["updated_files_reimported"].append(f"[WpDetail] {file_basename}")#🧩 
            #📃 Update log_entries for successfully import: Add information to the log list for this run (successful).
            log_entries.append({"FileName": file_basename, "ModifiedDate": current_modified_date, "Error": None})
            print_colored(f"Updated 'Modified' time for {file_basename}", "LimeGreen")
            logger.info(f"Updated 'Modified' time for {file_basename}")
            #📃 Handle exceptions if any error occurred in the try block.
        except (
            FileNotFoundError, PermissionError, UnicodeDecodeError, IOError,                
            pl.exceptions.NoDataError, pl.exceptions.ComputeError, pl.exceptions.SchemaError, sa.exc.SQLAlchemyError,
            Exception # Catch various types of potential errors.
        ) as e:
            error_count += 1 # Increment the error counter.        
            error_msg_short = f"{type(e).__name__}: {str(e).splitlines()[0]}"  # Get a short error message (the first line).
            #📃 Update Failed file to Summary
            summary_results["failed_imports"].append((f"[WpDetail] {file_basename}", error_msg_short))#🧩      
            logger.error(f"Error importing {file_basename}: {error_msg_short}", exc_info=True) # Log the full error traceback.
            print_colored(f"❗️Error importing {file_basename}: {error_msg_short}", "OrangeRed")
            # Add information to the log list for this run (failed).
            # Note: ModifiedDate here is previous_modified_date, as this file wasn't successfully imported with current_modified_date.
            log_entries.append({"FileName": file_basename,"ModifiedDate": previous_modified_date,"Error": error_msg_short})
    else: #📃 If the file is not new and its modification date hasn't changed.
          #📃 Update Skip file to Summary
        summary_results["skipped_files_unchanged"].append(f"[WpDetail] {file_basename}")#🧩      
        print_colored(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.", "Aquamarine")
        logger.info(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.")
#💡 After iterating through all files in the directory.
display_summary("WpDetail", error_count) #🧩 Display a summary of the results (number of errors).
process_and_save_log(log_df, log_entries, log_WpDetail_path) #🧩 Update the Excel log file with information
logger.info("===== Processing of the WpDetail data source is complete =====") #🧩 Log finish data processing.

In [ ]:
# 2️⃣1️⃣[BKN]WpSummary💾
logger.info("===== Start WpSummary Process =====")  #🧩 Log the start of the data processing.
log_df = read_or_create_log(log_WpSummary_path) #🧩 Read the existing Excel log file or create a new one if it doesn't exist.
log_entries, error_count = Default_variable() #📃 Initialize a list to store log results for the current run and an error counter.
#💡 Iterate through all files in the specified folder.
for filename in Folder_WpSummary.glob("*.xlsx"): #🧩 Get files in the Folder directory.
    #📃 Get the current modification timestamp and filename of the current file.
    current_modified_date_microseconds = datetime.datetime.fromtimestamp(filename.stat().st_mtime) # Get modification timestamp, including microseconds.
    current_modified_date = current_modified_date_microseconds.replace(microsecond=0) # Remove microseconds for easier comparison.
    file_basename = filename.name # Get the base name of the file (e.g., "data_20230101.csv").
    #📃 File is being processed.
    print_colored(f"Processing file🔄️: {file_basename}", "NavajoWhite")
    logger.info(f"Processing file🔄️: {file_basename}")
    #📃 Check if this file has been logged before.
    log_entry = log_df.filter(pl.col("FileName") == file_basename) # Filter the old log_df to find an entry for this filename.
    is_new_file = log_entry.is_empty() # If no entry is found (log_entry is empty), it's a new file.
    previous_modified_date = datetime.datetime(1900, 1, 1, 0, 0, 1)  # Initialize the previous modification date with a very old date.
    #📃 If this file was processed before (exists in the log).  
    if not is_new_file:
        previous_modified_date = log_entry.select("ModifiedDate").item() # Get the modification date recorded in the log from the previous processing.
    #📃 Condition to process the file:
    # 1. The file is new (not in the log).
    # 2. Or, the file is old (in the log), but its current modification date (current_modified_date) is more recent than the modification date recorded in the log (previous_modified_date).
    if is_new_file or current_modified_date > previous_modified_date:
        if not is_new_file: # If it's an old file but has changed (ModifiedDate is newer).
            print_colored(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.", "Coral")
            logger.warning(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.")
        else: # If it's a completely new file.
            print_colored(f"New file detected🚦: {file_basename}. Importing.", "Coral")
            logger.info(f"New file detected🚦: {file_basename}. Importing.")
        try: # Start a try-except block to handle potential errors during the import process.
            if not is_new_file: # If this file already exists in the DB (because it's not new and its ModifiedDate changed) then delete the old data for this file from the df.
                delete_data(engine, WpSummary_TABLE_NAME, file_basename)  #🧩 Call the function to delete data.
            #📃 Read data from the excel file using Polars.
            WpSummary = (pl.read_excel(filename, infer_schema_length=None, engine="calamine", sheet_name="Sheet1") #🧩 Read the entire file to infer the schema.Add two new columns to the DataFrame: FileName and ModifiedDate.
                    .select(pl.all(),pl.lit(file_basename).alias("FileName"),pl.lit(current_modified_date).dt.cast_time_unit("ms").alias("ModifiedDate")
                   )) #🧩
            #📃 Validate if the columns in the DataFrame match the defined schema
            has_critical_error, critical_schema_error_msg = validate_schema(WpSummary, WpSummary_schema, file_basename) #🧩
            if has_critical_error: # If critical columns are missing.
                raise ValueError(critical_schema_error_msg) # Raise an error to stop processing this file.
            #📃 Adjust the column structure.
            print_colored(f"⚙️Adjust column structure", "Olive")
            logger.warning(f"⚙️Adjust column structure")
            WpSummary = WpSummary.with_columns(pl.col("Date").cast(pl.Date)) #🧩
            WpSummary = WpSummary.with_columns(pl.col('Length', 'Percent').cast(pl.Float64)) #🧩
            WpSummary = WpSummary.select(WpSummary_schema) #🧩 Select columns in the correct order and names as per df_schema.
            print_colored(f"⚙️Column structure adjustment completed", "Olive")
            logger.info(f"⚙️Column structure adjustment completed")
            info_polars(WpSummary) #🧩 Display information about the DataFrame (shape, columns, dtypes).
            #📃 Start writing data to the database.
            print_colored(f"💾Start import file: {file_basename}", "Coral")
            logger.info(f"💾Start import file: {file_basename}")
            write_data(engine, WpSummary_TABLE_NAME, WpSummary) #🧩 Call the function to write the DataFrame to SQL.
            print_colored(f"Successfully imported✨: {file_basename}", "LimeGreen")
            logger.info(f"Successfully imported✨: {file_basename}")
            #📃 Record to summary
            if is_new_file:
                summary_results["new_files_imported"].append(f"[WpSummary] {file_basename}")#🧩
            else:
                summary_results["updated_files_reimported"].append(f"[WpSummary] {file_basename}")#🧩  
            #📃 Update log_entries for successfully import: Add information to the log list for this run (successful).
            log_entries.append({"FileName": file_basename, "ModifiedDate": current_modified_date, "Error": None})
            print_colored(f"Updated 'Modified' time for {file_basename}", "LimeGreen")
            logger.info(f"Updated 'Modified' time for {file_basename}")
            #📃 Handle exceptions if any error occurred in the try block.
        except (
            FileNotFoundError, PermissionError, UnicodeDecodeError, IOError,                
            pl.exceptions.NoDataError, pl.exceptions.ComputeError, pl.exceptions.SchemaError, sa.exc.SQLAlchemyError,
            Exception # Catch various types of potential errors.
        ) as e:
            error_count += 1 # Increment the error counter.        
            error_msg_short = f"{type(e).__name__}: {str(e).splitlines()[0]}"  # Get a short error message (the first line).
            #📃 Update Failed file to Summary
            summary_results["failed_imports"].append((f"[WpSummary] {file_basename}", error_msg_short))#🧩          
            logger.error(f"Error importing {file_basename}: {error_msg_short}", exc_info=True) # Log the full error traceback.
            print_colored(f"❗️Error importing {file_basename}: {error_msg_short}", "OrangeRed")
            # Add information to the log list for this run (failed).
            # Note: ModifiedDate here is previous_modified_date, as this file wasn't successfully imported with current_modified_date.
            log_entries.append({"FileName": file_basename,"ModifiedDate": previous_modified_date,"Error": error_msg_short})
    else: #📃 If the file is not new and its modification date hasn't changed.
          #📃 Update Skip file to Summary
        summary_results["skipped_files_unchanged"].append(f"[WpSummary] {file_basename}")#🧩      
        print_colored(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.", "Aquamarine")
        logger.info(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.")
#💡 After iterating through all files in the directory.
display_summary("WpSummary", error_count) #🧩 Display a summary of the results (number of errors).
process_and_save_log(log_df, log_entries, log_WpSummary_path) #🧩 Update the Excel log file with information
logger.info("===== Processing of the WpSummary data source is complete =====") #🧩 Log finish data processing.

In [ ]:
# 2️⃣2️⃣[BKN]RegisteredOT💾
logger.info("===== Start RegisteredOT Process =====")  #🧩 Log the start of the data processing.
log_df = read_or_create_log(log_RegisteredOT_path) #🧩 Read the existing Excel log file or create a new one if it doesn't exist.
log_entries, error_count = Default_variable() #📃 Initialize a list to store log results for the current run and an error counter.
#💡 Iterate through all files in the specified folder.
for filename in Folder_RegisteredOT.glob("*.xlsx"): #🧩 Get files in the Folder directory.
    #📃 Get the current modification timestamp and filename of the current file.
    current_modified_date_microseconds = datetime.datetime.fromtimestamp(filename.stat().st_mtime) # Get modification timestamp, including microseconds.
    current_modified_date = current_modified_date_microseconds.replace(microsecond=0) # Remove microseconds for easier comparison.
    file_basename = filename.name # Get the base name of the file (e.g., "data_20230101.csv").
    #📃 File is being processed.
    print_colored(f"Processing file🔄️: {file_basename}", "NavajoWhite")
    logger.info(f"Processing file🔄️: {file_basename}")
    #📃 Check if this file has been logged before.
    log_entry = log_df.filter(pl.col("FileName") == file_basename) # Filter the old log_df to find an entry for this filename.
    is_new_file = log_entry.is_empty() # If no entry is found (log_entry is empty), it's a new file.
    previous_modified_date = datetime.datetime(1900, 1, 1, 0, 0, 1)  # Initialize the previous modification date with a very old date.
    #📃 If this file was processed before (exists in the log).  
    if not is_new_file:
        previous_modified_date = log_entry.select("ModifiedDate").item() # Get the modification date recorded in the log from the previous processing.
    #📃 Condition to process the file:
    # 1. The file is new (not in the log).
    # 2. Or, the file is old (in the log), but its current modification date (current_modified_date) is more recent than the modification date recorded in the log (previous_modified_date).
    if is_new_file or current_modified_date > previous_modified_date:
        if not is_new_file: # If it's an old file but has changed (ModifiedDate is newer).
            print_colored(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.", "Coral")
            logger.warning(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.")
        else: # If it's a completely new file.
            print_colored(f"New file detected🚦: {file_basename}. Importing.", "Coral")
            logger.info(f"New file detected🚦: {file_basename}. Importing.")
        try: # Start a try-except block to handle potential errors during the import process.
            if not is_new_file: # If this file already exists in the DB (because it's not new and its ModifiedDate changed) then delete the old data for this file from the df.
                delete_data(engine, RegisteredOT_TABLE_NAME, file_basename)  #🧩 Call the function to delete data.
            #📃 Read data from the excel file using Polars.
            RegisteredOT = (pl.read_excel(filename, infer_schema_length=None, engine="calamine", sheet_name="Sheet1") #🧩 Read the entire file to infer the schema.Add two new columns to the DataFrame: FileName and ModifiedDate.
                    .select(pl.all(),pl.lit(file_basename).alias("FileName"),pl.lit(current_modified_date).dt.cast_time_unit("ms").alias("ModifiedDate")
                   )) #🧩
            #📃 Validate if the columns in the DataFrame match the defined schema
            has_critical_error, critical_schema_error_msg = validate_schema(RegisteredOT, RegisteredOT_schema, file_basename) #🧩
            if has_critical_error: # If critical columns are missing.
                raise ValueError(critical_schema_error_msg) # Raise an error to stop processing this file.
            #📃 Adjust the column structure.
            print_colored(f"⚙️Adjust column structure", "Olive")
            logger.warning(f"⚙️Adjust column structure")
            RegisteredOT = RegisteredOT.with_columns(pl.col("Date").cast(pl.Date)) #🧩
            RegisteredOT = RegisteredOT.with_columns(pl.col("OT").cast(pl.Float64)) #🧩
            RegisteredOT = RegisteredOT.select(RegisteredOT_schema) #🧩 Select columns in the correct order and names as per df_schema.
            print_colored(f"⚙️Column structure adjustment completed", "Olive")
            logger.info(f"⚙️Column structure adjustment completed")
            info_polars(RegisteredOT) #🧩 Display information about the DataFrame (shape, columns, dtypes).
            #📃 Start writing data to the database.
            print_colored(f"💾Start import file: {file_basename}", "Coral")
            logger.info(f"💾Start import file: {file_basename}")
            write_data(engine, RegisteredOT_TABLE_NAME, RegisteredOT) #🧩 Call the function to write the DataFrame to SQL.
            print_colored(f"Successfully imported✨: {file_basename}", "LimeGreen")
            logger.info(f"Successfully imported✨: {file_basename}")
            #📃 Record to summary
            if is_new_file:
                summary_results["new_files_imported"].append(f"[RegisteredOT] {file_basename}")#🧩
            else:
                summary_results["updated_files_reimported"].append(f"[RegisteredOT] {file_basename}")#🧩     
            #📃 Update log_entries for successfully import: Add information to the log list for this run (successful).
            log_entries.append({"FileName": file_basename, "ModifiedDate": current_modified_date, "Error": None})
            print_colored(f"Updated 'Modified' time for {file_basename}", "LimeGreen")
            logger.info(f"Updated 'Modified' time for {file_basename}")
            #📃 Handle exceptions if any error occurred in the try block.
        except (
            FileNotFoundError, PermissionError, UnicodeDecodeError, IOError,                
            pl.exceptions.NoDataError, pl.exceptions.ComputeError, pl.exceptions.SchemaError, sa.exc.SQLAlchemyError,
            Exception # Catch various types of potential errors.
        ) as e:
            error_count += 1 # Increment the error counter.        
            error_msg_short = f"{type(e).__name__}: {str(e).splitlines()[0]}"  # Get a short error message (the first line).
            #📃 Update Failed file to Summary
            summary_results["failed_imports"].append((f"[RegisteredOT] {file_basename}", error_msg_short))#🧩
            logger.error(f"Error importing {file_basename}: {error_msg_short}", exc_info=True) # Log the full error traceback.
            print_colored(f"❗️Error importing {file_basename}: {error_msg_short}", "OrangeRed")
            # Add information to the log list for this run (failed).
            # Note: ModifiedDate here is previous_modified_date, as this file wasn't successfully imported with current_modified_date.
            log_entries.append({"FileName": file_basename,"ModifiedDate": previous_modified_date,"Error": error_msg_short})
    else: #📃 If the file is not new and its modification date hasn't changed.
          #📃 Update Skip file to Summary
        summary_results["skipped_files_unchanged"].append(f"[RegisteredOT] {file_basename}")#🧩
        print_colored(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.", "Aquamarine")
        logger.info(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.")
#💡 After iterating through all files in the directory.
display_summary("RegisteredOT", error_count) #🧩 Display a summary of the results (number of errors).
process_and_save_log(log_df, log_entries, log_RegisteredOT_path) #🧩 Update the Excel log file with information
logger.info("===== Processing of the RegisteredOT data source is complete =====") #🧩 Log finish data processing.

In [ ]:
# 2️⃣3️⃣[BKN]CSAT_TP💾
logger.info("===== Start CSAT_TP Process =====")  #🧩 Log the start of the data processing.
log_df = read_or_create_log(log_CSAT_TP_path) #🧩 Read the existing Excel log file or create a new one if it doesn't exist.
log_entries, error_count = Default_variable() #📃 Initialize a list to store log results for the current run and an error counter.
#💡 Iterate through all files in the specified folder.
for filename in Folder_CSAT_TP.glob("*.csv"): #🧩 Get files in the Folder directory.
    #📃 Get the current modification timestamp and filename of the current file.
    current_modified_date_microseconds = datetime.datetime.fromtimestamp(filename.stat().st_mtime) # Get modification timestamp, including microseconds.
    current_modified_date = current_modified_date_microseconds.replace(microsecond=0) # Remove microseconds for easier comparison.
    file_basename = filename.name # Get the base name of the file (e.g., "data_20230101.csv").
    #📃 File is being processed.
    print_colored(f"Processing file🔄️: {file_basename}", "NavajoWhite")
    logger.info(f"Processing file🔄️: {file_basename}")
    #📃 Check if this file has been logged before.
    log_entry = log_df.filter(pl.col("FileName") == file_basename) # Filter the old log_df to find an entry for this filename.
    is_new_file = log_entry.is_empty() # If no entry is found (log_entry is empty), it's a new file.
    previous_modified_date = datetime.datetime(1900, 1, 1, 0, 0, 1)  # Initialize the previous modification date with a very old date.
    #📃 If this file was processed before (exists in the log).  
    if not is_new_file:
        previous_modified_date = log_entry.select("ModifiedDate").item() # Get the modification date recorded in the log from the previous processing.
    #📃 Condition to process the file:
    # 1. The file is new (not in the log).
    # 2. Or, the file is old (in the log), but its current modification date (current_modified_date) is more recent than the modification date recorded in the log (previous_modified_date).
    if is_new_file or current_modified_date > previous_modified_date:
        if not is_new_file: # If it's an old file but has changed (ModifiedDate is newer).
            print_colored(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.", "Coral")
            logger.warning(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.")
        else: # If it's a completely new file.
            print_colored(f"New file detected🚦: {file_basename}. Importing.", "Coral")
            logger.info(f"New file detected🚦: {file_basename}. Importing.")
        try: # Start a try-except block to handle potential errors during the import process.
            if not is_new_file: # If this file already exists in the DB (because it's not new and its ModifiedDate changed) then delete the old data for this file from the df.
                delete_data(engine, CSAT_TP_TABLE_NAME, file_basename)  #🧩 Call the function to delete data.
            #📃 Read data from the excel file using Polars.
            CSAT_TP = (pl.read_csv(filename, infer_schema_length=None) #🧩 Read the entire file to infer the schema.Add two new columns to the DataFrame: FileName and ModifiedDate.
                   .select(pl.all(),pl.lit(file_basename).alias("FileName"),pl.lit(current_modified_date).dt.cast_time_unit("ms").alias("ModifiedDate")
                   ))       
            CSAT_TP = (CSAT_TP.rename({'Date ': 'Date', '""Comment""': '"Comment"'})) #🧩
            #📃 Validate if the columns in the DataFrame match the defined schema
            has_critical_error, critical_schema_error_msg = validate_schema(CSAT_TP, CSAT_TP_schema, file_basename) #🧩
            if has_critical_error: # If critical columns are missing.
                raise ValueError(critical_schema_error_msg) # Raise an error to stop processing this file.
            #📃 Adjust the column structure.
            print_colored(f"⚙️Adjust column structure", "Olive")
            logger.warning(f"⚙️Adjust column structure")
            CSAT_TP = CSAT_TP.with_columns(
                parse_date(pl.col("Sort by Dimension")).alias("Sort by Dimension"),
                parse_date(pl.col("Date")).alias("Date"),
                parse_date(pl.col("Max. Sort by Dimension")).alias("Max. Sort by Dimension"),
                pl.col('Sort by Dimension (copy)').cast(pl.Float64)) #🧩
            CSAT_TP = CSAT_TP.select(CSAT_TP_schema) #🧩 Select columns in the correct order and names as per df_schema.
            print_colored(f"⚙️Column structure adjustment completed", "Olive")
            logger.info(f"⚙️Column structure adjustment completed")
            info_polars(CSAT_TP) #🧩 Display information about the DataFrame (shape, columns, dtypes).
            #📃 Start writing data to the database.
            print_colored(f"💾Start import file: {file_basename}", "Coral")
            logger.info(f"💾Start import file: {file_basename}")
            write_data(engine, CSAT_TP_TABLE_NAME, CSAT_TP) #🧩 Call the function to write the DataFrame to SQL.
            print_colored(f"Successfully imported✨: {file_basename}", "LimeGreen")
            logger.info(f"Successfully imported✨: {file_basename}")
            #📃 Record to summary
            if is_new_file:
                summary_results["new_files_imported"].append(f"[CSAT_TP] {file_basename}")#🧩
            else:
                summary_results["updated_files_reimported"].append(f"[CSAT_TP] {file_basename}")#🧩          
            #📃 Update log_entries for successfully import: Add information to the log list for this run (successful).
            log_entries.append({"FileName": file_basename, "ModifiedDate": current_modified_date, "Error": None})
            print_colored(f"Updated 'Modified' time for {file_basename}", "LimeGreen")
            logger.info(f"Updated 'Modified' time for {file_basename}")
            #📃 Handle exceptions if any error occurred in the try block.
        except (
            FileNotFoundError, PermissionError, UnicodeDecodeError, IOError,                
            pl.exceptions.NoDataError, pl.exceptions.ComputeError, pl.exceptions.SchemaError, sa.exc.SQLAlchemyError,
            Exception # Catch various types of potential errors.
        ) as e:
            error_count += 1 # Increment the error counter.        
            error_msg_short = f"{type(e).__name__}: {str(e).splitlines()[0]}"  # Get a short error message (the first line).
            #📃 Update Failed file to Summary
            summary_results["failed_imports"].append((f"[CSAT_TP] {file_basename}", error_msg_short))#🧩         
            logger.error(f"Error importing {file_basename}: {error_msg_short}", exc_info=True) # Log the full error traceback.
            print_colored(f"❗️Error importing {file_basename}: {error_msg_short}", "OrangeRed")
            # Add information to the log list for this run (failed).
            # Note: ModifiedDate here is previous_modified_date, as this file wasn't successfully imported with current_modified_date.
            log_entries.append({"FileName": file_basename,"ModifiedDate": previous_modified_date,"Error": error_msg_short})
    else: #📃 If the file is not new and its modification date hasn't changed.
          #📃 Update Skip file to Summary
        summary_results["skipped_files_unchanged"].append(f"[CSAT_TP] {file_basename}")#🧩 
        print_colored(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.", "Aquamarine")
        logger.info(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.")
#💡 After iterating through all files in the directory.
display_summary("CSAT_TP", error_count) #🧩 Display a summary of the results (number of errors).
process_and_save_log(log_df, log_entries, log_CSAT_TP_path) #🧩 Update the Excel log file with information
logger.info("===== Processing of the CSAT_TP data source is complete =====") #🧩 Log finish data processing.

In [ ]:
# 2️⃣4️⃣[BKN]CSAT_RS💾
logger.info("===== Start CSAT_RS Process =====")  #🧩 Log the start of the data processing.
log_df = read_or_create_log(log_CSAT_RS_path) #🧩 Read the existing Excel log file or create a new one if it doesn't exist.
log_entries, error_count = Default_variable() #📃 Initialize a list to store log results for the current run and an error counter.
#💡 Iterate through all files in the specified folder.
for filename in Folder_CSAT_RS.glob("*.csv"): #🧩 Get files in the Folder directory.
    #📃 Get the current modification timestamp and filename of the current file.
    current_modified_date_microseconds = datetime.datetime.fromtimestamp(filename.stat().st_mtime) # Get modification timestamp, including microseconds.
    current_modified_date = current_modified_date_microseconds.replace(microsecond=0) # Remove microseconds for easier comparison.
    file_basename = filename.name # Get the base name of the file (e.g., "data_20230101.csv").
    #📃 File is being processed.
    print_colored(f"Processing file🔄️: {file_basename}", "NavajoWhite")
    logger.info(f"Processing file🔄️: {file_basename}")
    #📃 Check if this file has been logged before.
    log_entry = log_df.filter(pl.col("FileName") == file_basename) # Filter the old log_df to find an entry for this filename.
    is_new_file = log_entry.is_empty() # If no entry is found (log_entry is empty), it's a new file.
    previous_modified_date = datetime.datetime(1900, 1, 1, 0, 0, 1)  # Initialize the previous modification date with a very old date.
    #📃 If this file was processed before (exists in the log).  
    if not is_new_file:
        previous_modified_date = log_entry.select("ModifiedDate").item() # Get the modification date recorded in the log from the previous processing.
    #📃 Condition to process the file:
    # 1. The file is new (not in the log).
    # 2. Or, the file is old (in the log), but its current modification date (current_modified_date) is more recent than the modification date recorded in the log (previous_modified_date).
    if is_new_file or current_modified_date > previous_modified_date:
        if not is_new_file: # If it's an old file but has changed (ModifiedDate is newer).
            print_colored(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.", "Coral")
            logger.warning(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.")
        else: # If it's a completely new file.
            print_colored(f"New file detected🚦: {file_basename}. Importing.", "Coral")
            logger.info(f"New file detected🚦: {file_basename}. Importing.")
        try: # Start a try-except block to handle potential errors during the import process.
            if not is_new_file: # If this file already exists in the DB (because it's not new and its ModifiedDate changed) then delete the old data for this file from the df.
                delete_data(engine, CSAT_RS_TABLE_NAME, file_basename)  #🧩 Call the function to delete data.
            #📃 Read data from the excel file using Polars.
            CSAT_RS = (pl.read_csv(filename, infer_schema_length=None) #🧩 Read the entire file to infer the schema.Add two new columns to the DataFrame: FileName and ModifiedDate.
                   .select(pl.all(),pl.lit(file_basename).alias("FileName"),pl.lit(current_modified_date).dt.cast_time_unit("ms").alias("ModifiedDate")
                   ))
            CSAT_RS = (CSAT_RS.rename({'Date ': 'Date', '""Comment""': '"Comment"'})) #🧩
            #📃 Validate if the columns in the DataFrame match the defined schema
            has_critical_error, critical_schema_error_msg = validate_schema(CSAT_RS, CSAT_RS_schema, file_basename) #🧩
            if has_critical_error: # If critical columns are missing.
                raise ValueError(critical_schema_error_msg) # Raise an error to stop processing this file.
            #📃 Adjust the column structure.
            print_colored(f"⚙️Adjust column structure", "Olive")
            logger.warning(f"⚙️Adjust column structure")
            CSAT_RS = CSAT_RS.with_columns(
                parse_date(pl.col("Sort by Dimension")).alias("Sort by Dimension"),
                parse_date(pl.col("Max. Sort by Dimension")).alias("Max. Sort by Dimension"),
                pl.col("Sort by Dimension (copy)").cast(pl.Float64),).select(CSAT_RS_schema) #🧩 Select columns in the correct order and names as per df_schema.
            print_colored(f"⚙️Column structure adjustment completed", "Olive")
            logger.info(f"⚙️Column structure adjustment completed")
            info_polars(CSAT_RS) #🧩 Display information about the DataFrame (shape, columns, dtypes).
            #📃 Start writing data to the database.
            print_colored(f"💾Start import file: {file_basename}", "Coral")
            logger.info(f"💾Start import file: {file_basename}")
            write_data(engine, CSAT_RS_TABLE_NAME, CSAT_RS) #🧩 Call the function to write the DataFrame to SQL.
            print_colored(f"Successfully imported✨: {file_basename}", "LimeGreen")
            logger.info(f"Successfully imported✨: {file_basename}")
            #📃 Record to summary
            if is_new_file:
                summary_results["new_files_imported"].append(f"[CSAT_RS] {file_basename}")#🧩
            else:
                summary_results["updated_files_reimported"].append(f"[CSAT_RS] {file_basename}")#🧩            
            #📃 Update log_entries for successfully import: Add information to the log list for this run (successful).
            log_entries.append({"FileName": file_basename, "ModifiedDate": current_modified_date, "Error": None})
            print_colored(f"Updated 'Modified' time for {file_basename}", "LimeGreen")
            logger.info(f"Updated 'Modified' time for {file_basename}")
            #📃 Handle exceptions if any error occurred in the try block.
        except (
            FileNotFoundError, PermissionError, UnicodeDecodeError, IOError,                
            pl.exceptions.NoDataError, pl.exceptions.ComputeError, pl.exceptions.SchemaError, sa.exc.SQLAlchemyError,
            Exception # Catch various types of potential errors.
        ) as e:
            error_count += 1 # Increment the error counter.        
            error_msg_short = f"{type(e).__name__}: {str(e).splitlines()[0]}"  # Get a short error message (the first line).
            #📃 Update Failed file to Summary
            summary_results["failed_imports"].append((f"[CSAT_RS] {file_basename}", error_msg_short))#🧩    
            logger.error(f"Error importing {file_basename}: {error_msg_short}", exc_info=True) # Log the full error traceback.
            print_colored(f"❗️Error importing {file_basename}: {error_msg_short}", "OrangeRed")
            # Add information to the log list for this run (failed).
            # Note: ModifiedDate here is previous_modified_date, as this file wasn't successfully imported with current_modified_date.
            log_entries.append({"FileName": file_basename,"ModifiedDate": previous_modified_date,"Error": error_msg_short})
    else: #📃 If the file is not new and its modification date hasn't changed.
          #📃 Update Skip file to Summary
        summary_results["skipped_files_unchanged"].append(f"[CSAT_RS] {file_basename}")#🧩
        print_colored(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.", "Aquamarine")
        logger.info(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.")
#💡 After iterating through all files in the directory.
display_summary("CSAT_RS", error_count) #🧩 Display a summary of the results (number of errors).
process_and_save_log(log_df, log_entries, log_CSAT_RS_path) #🧩 Update the Excel log file with information
logger.info("===== Processing of the CSAT_RS data source is complete =====") #🧩 Log finish data processing.

In [ ]:
# 2️⃣5️⃣[BKN]PSAT💾
logger.info("===== Start PSAT Process =====")  #🧩 Log the start of the data processing.
log_df = read_or_create_log(log_PSAT_path) #🧩 Read the existing Excel log file or create a new one if it doesn't exist.
log_entries, error_count = Default_variable() #📃 Initialize a list to store log results for the current run and an error counter.
#💡 Iterate through all files in the specified folder.
for filename in Folder_PSAT.glob("*.csv"): #🧩 Get files in the Folder directory.
    #📃 Get the current modification timestamp and filename of the current file.
    current_modified_date_microseconds = datetime.datetime.fromtimestamp(filename.stat().st_mtime) # Get modification timestamp, including microseconds.
    current_modified_date = current_modified_date_microseconds.replace(microsecond=0) # Remove microseconds for easier comparison.
    file_basename = filename.name # Get the base name of the file (e.g., "data_20230101.csv").
    #📃 File is being processed.
    print_colored(f"Processing file🔄️: {file_basename}", "NavajoWhite")
    logger.info(f"Processing file🔄️: {file_basename}")
    #📃 Check if this file has been logged before.
    log_entry = log_df.filter(pl.col("FileName") == file_basename) # Filter the old log_df to find an entry for this filename.
    is_new_file = log_entry.is_empty() # If no entry is found (log_entry is empty), it's a new file.
    previous_modified_date = datetime.datetime(1900, 1, 1, 0, 0, 1)  # Initialize the previous modification date with a very old date.
    #📃 If this file was processed before (exists in the log).  
    if not is_new_file:
        previous_modified_date = log_entry.select("ModifiedDate").item() # Get the modification date recorded in the log from the previous processing.
    #📃 Condition to process the file:
    # 1. The file is new (not in the log).
    # 2. Or, the file is old (in the log), but its current modification date (current_modified_date) is more recent than the modification date recorded in the log (previous_modified_date).
    if is_new_file or current_modified_date > previous_modified_date:
        if not is_new_file: # If it's an old file but has changed (ModifiedDate is newer).
            print_colored(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.", "Coral")
            logger.warning(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.")
        else: # If it's a completely new file.
            print_colored(f"New file detected🚦: {file_basename}. Importing.", "Coral")
            logger.info(f"New file detected🚦: {file_basename}. Importing.")
        try: # Start a try-except block to handle potential errors during the import process.
            if not is_new_file: # If this file already exists in the DB (because it's not new and its ModifiedDate changed) then delete the old data for this file from the df.
                delete_data(engine, PSAT_TABLE_NAME, file_basename)  #🧩 Call the function to delete data.
            #📃 Read data from the excel file using Polars.
            PSAT = (pl.read_csv(filename, infer_schema_length=None) #🧩 Read the entire file to infer the schema.Add two new columns to the DataFrame: FileName and ModifiedDate.
                   .select(pl.all(),pl.lit(file_basename).alias("FileName"),pl.lit(current_modified_date).dt.cast_time_unit("ms").alias("ModifiedDate")
                   )) 
            PSAT = (PSAT.rename({'""Comment""': '"Comment"'})) #🧩
            #📃 Validate if the columns in the DataFrame match the defined schema
            has_critical_error, critical_schema_error_msg = validate_schema(PSAT, PSAT_schema, file_basename) #🧩
            if has_critical_error: # If critical columns are missing.
                raise ValueError(critical_schema_error_msg) # Raise an error to stop processing this file.
            #📃 Adjust the column structure.
            print_colored(f"⚙️Adjust column structure", "Olive")
            logger.warning(f"⚙️Adjust column structure")
            PSAT = PSAT.with_columns(
                parse_date(pl.col("Sorted By Dimension")).alias("Sorted By Dimension"),
                parse_date(pl.col("Date")).alias("Date"),
                pl.col("Sorted BY Dimension (copy)").cast(pl.Float64),).select(PSAT_schema) #🧩 Select columns in the correct order and names as per df_schema.
            print_colored(f"⚙️Column structure adjustment completed", "Olive")
            logger.info(f"⚙️Column structure adjustment completed")
            info_polars(PSAT) #🧩 Display information about the DataFrame (shape, columns, dtypes).
            #📃 Start writing data to the database.
            print_colored(f"💾Start import file: {file_basename}", "Coral")
            logger.info(f"💾Start import file: {file_basename}")
            write_data(engine, PSAT_TABLE_NAME, PSAT) #🧩 Call the function to write the DataFrame to SQL.
            print_colored(f"Successfully imported✨: {file_basename}", "LimeGreen")
            logger.info(f"Successfully imported✨: {file_basename}")
            #📃 Record to summary
            if is_new_file:
                summary_results["new_files_imported"].append(f"[PSAT] {file_basename}")#🧩
            else:
                summary_results["updated_files_reimported"].append(f"[PSAT] {file_basename}")#🧩
            #📃 Update log_entries for successfully import: Add information to the log list for this run (successful).
            log_entries.append({"FileName": file_basename, "ModifiedDate": current_modified_date, "Error": None})
            print_colored(f"Updated 'Modified' time for {file_basename}", "LimeGreen")
            logger.info(f"Updated 'Modified' time for {file_basename}")
            #📃 Handle exceptions if any error occurred in the try block.
        except (
            FileNotFoundError, PermissionError, UnicodeDecodeError, IOError,                
            pl.exceptions.NoDataError, pl.exceptions.ComputeError, pl.exceptions.SchemaError, sa.exc.SQLAlchemyError,
            Exception # Catch various types of potential errors.
        ) as e:
            error_count += 1 # Increment the error counter.        
            error_msg_short = f"{type(e).__name__}: {str(e).splitlines()[0]}"  # Get a short error message (the first line).
            #📃 Update Failed file to Summary
            summary_results["failed_imports"].append((f"[PSAT] {file_basename}", error_msg_short))#🧩
            logger.error(f"Error importing {file_basename}: {error_msg_short}", exc_info=True) # Log the full error traceback.
            print_colored(f"❗️Error importing {file_basename}: {error_msg_short}", "OrangeRed")
            # Add information to the log list for this run (failed).
            # Note: ModifiedDate here is previous_modified_date, as this file wasn't successfully imported with current_modified_date.
            log_entries.append({"FileName": file_basename,"ModifiedDate": previous_modified_date,"Error": error_msg_short})
    else: #📃 If the file is not new and its modification date hasn't changed.
          #📃 Update Skip file to Summary
        summary_results["skipped_files_unchanged"].append(f"[PSAT] {file_basename}")#🧩   
        print_colored(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.", "Aquamarine")
        logger.info(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.")
#💡 After iterating through all files in the directory.
display_summary("PSAT", error_count) #🧩 Display a summary of the results (number of errors).
process_and_save_log(log_df, log_entries, log_PSAT_path) #🧩 Update the Excel log file with information
logger.info("===== Processing of the PSAT data source is complete =====") #🧩 Log finish data processing.

In [ ]:
# 2️⃣6️⃣[BKN]IEX_Hrs💾
logger.info("===== Start IEX_Hrs Process =====")  #🧩 Log the start of the data processing.
log_df = read_or_create_log(log_IEX_Hrs_path) #🧩 Read the existing Excel log file or create a new one if it doesn't exist.
log_entries, error_count = Default_variable() #📃 Initialize a list to store log results for the current run and an error counter.
#💡 Iterate through all files in the specified folder.
for filename in Folder_IEX_Hrs.glob("*.xlsx"): #🧩 Get files in the Folder directory.
    #📃 Get the current modification timestamp and filename of the current file.
    current_modified_date_microseconds = datetime.datetime.fromtimestamp(filename.stat().st_mtime) # Get modification timestamp, including microseconds.
    current_modified_date = current_modified_date_microseconds.replace(microsecond=0) # Remove microseconds for easier comparison.
    file_basename = filename.name # Get the base name of the file (e.g., "data_20230101.csv").
    #📃 File is being processed.
    print_colored(f"Processing file🔄️: {file_basename}", "NavajoWhite")
    logger.info(f"Processing file🔄️: {file_basename}")
    #📃 Check if this file has been logged before.
    log_entry = log_df.filter(pl.col("FileName") == file_basename) # Filter the old log_df to find an entry for this filename.
    is_new_file = log_entry.is_empty() # If no entry is found (log_entry is empty), it's a new file.
    previous_modified_date = datetime.datetime(1900, 1, 1, 0, 0, 1)  # Initialize the previous modification date with a very old date.
    #📃 If this file was processed before (exists in the log).  
    if not is_new_file:
        previous_modified_date = log_entry.select("ModifiedDate").item() # Get the modification date recorded in the log from the previous processing.
    #📃 Condition to process the file:
    # 1. The file is new (not in the log).
    # 2. Or, the file is old (in the log), but its current modification date (current_modified_date) is more recent than the modification date recorded in the log (previous_modified_date).
    if is_new_file or current_modified_date > previous_modified_date:
        if not is_new_file: # If it's an old file but has changed (ModifiedDate is newer).
            print_colored(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.", "Coral")
            logger.warning(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.")
        else: # If it's a completely new file.
            print_colored(f"New file detected🚦: {file_basename}. Importing.", "Coral")
            logger.info(f"New file detected🚦: {file_basename}. Importing.")
        try: # Start a try-except block to handle potential errors during the import process.
            if not is_new_file: # If this file already exists in the DB (because it's not new and its ModifiedDate changed) then delete the old data for this file from the df.
                delete_data(engine, IEX_Hrs_TABLE_NAME, file_basename)  #🧩 Call the function to delete data.
            #📃 Read data from the excel file using Polars.
            IEX_Hrs = (pl.read_excel(filename, infer_schema_length=None, engine="calamine", sheet_name="Sheet1") #🧩 Read the entire file to infer the schema.Add two new columns to the DataFrame: FileName and ModifiedDate.
                    .select(pl.all(),pl.lit(file_basename).alias("FileName"),pl.lit(current_modified_date).dt.cast_time_unit("ms").alias("ModifiedDate")
                   )) #🧩
            #📃 Validate if the columns in the DataFrame match the defined schema
            has_critical_error, critical_schema_error_msg = validate_schema(IEX_Hrs, IEX_Hrs_schema, file_basename) #🧩
            if has_critical_error: # If critical columns are missing.
                raise ValueError(critical_schema_error_msg) # Raise an error to stop processing this file.
            #📃 Adjust the column structure.
            print_colored(f"⚙️Adjust column structure", "Olive")
            logger.warning(f"⚙️Adjust column structure")
            IEX_Hrs = IEX_Hrs.with_columns(pl.col("VNT", "CET").cast(pl.Datetime)) #🧩
            IEX_Hrs = IEX_Hrs.with_columns(pl.col("HC", "Hour").cast(pl.Float64)) #🧩
            IEX_Hrs = IEX_Hrs.select(IEX_Hrs_schema) #🧩 Select columns in the correct order and names as per df_schema.
            print_colored(f"⚙️Column structure adjustment completed", "Olive")
            logger.info(f"⚙️Column structure adjustment completed")
            info_polars(IEX_Hrs) #🧩 Display information about the DataFrame (shape, columns, dtypes).
            #📃 Start writing data to the database.
            print_colored(f"💾Start import file: {file_basename}", "Coral")
            logger.info(f"💾Start import file: {file_basename}")
            write_data(engine, IEX_Hrs_TABLE_NAME, IEX_Hrs) #🧩 Call the function to write the DataFrame to SQL.
            print_colored(f"Successfully imported✨: {file_basename}", "LimeGreen")
            logger.info(f"Successfully imported✨: {file_basename}")
            #📃 Record to summary
            if is_new_file:
                summary_results["new_files_imported"].append(f"[IEX_Hrs] {file_basename}")#🧩
            else:
                summary_results["updated_files_reimported"].append(f"[IEX_Hrs] {file_basename}")#🧩   
            #📃 Update log_entries for successfully import: Add information to the log list for this run (successful).
            log_entries.append({"FileName": file_basename, "ModifiedDate": current_modified_date, "Error": None})
            print_colored(f"Updated 'Modified' time for {file_basename}", "LimeGreen")
            logger.info(f"Updated 'Modified' time for {file_basename}")
            #📃 Handle exceptions if any error occurred in the try block.
        except (
            FileNotFoundError, PermissionError, UnicodeDecodeError, IOError,                
            pl.exceptions.NoDataError, pl.exceptions.ComputeError, pl.exceptions.SchemaError, sa.exc.SQLAlchemyError,
            Exception # Catch various types of potential errors.
        ) as e:
            error_count += 1 # Increment the error counter.        
            error_msg_short = f"{type(e).__name__}: {str(e).splitlines()[0]}"  # Get a short error message (the first line).
            #📃 Update Failed file to Summary
            summary_results["failed_imports"].append((f"[IEX_Hrs] {file_basename}", error_msg_short))#🧩
            logger.error(f"Error importing {file_basename}: {error_msg_short}", exc_info=True) # Log the full error traceback.
            print_colored(f"❗️Error importing {file_basename}: {error_msg_short}", "OrangeRed")
            # Add information to the log list for this run (failed).
            # Note: ModifiedDate here is previous_modified_date, as this file wasn't successfully imported with current_modified_date.
            log_entries.append({"FileName": file_basename,"ModifiedDate": previous_modified_date,"Error": error_msg_short})
    else: #📃 If the file is not new and its modification date hasn't changed.
          #📃 Update Skip file to Summary
        summary_results["skipped_files_unchanged"].append(f"[IEX_Hrs] {file_basename}")#🧩
        print_colored(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.", "Aquamarine")
        logger.info(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.")
#💡 After iterating through all files in the directory.
display_summary("IEX_Hrs", error_count) #🧩 Display a summary of the results (number of errors).
process_and_save_log(log_df, log_entries, log_IEX_Hrs_path) #🧩 Update the Excel log file with information
logger.info("===== Processing of the IEX_Hrs data source is complete =====") #🧩 Log finish data processing.

In [ ]:
# 2️⃣7️⃣[BKN]IntervalReq💾
logger.info("===== Start IntervalReq Process =====")  #🧩 Log the start of the data processing.
log_df = read_or_create_log(log_IntervalReq_path) #🧩 Read the existing Excel log file or create a new one if it doesn't exist.
log_entries, error_count = Default_variable() #📃 Initialize a list to store log results for the current run and an error counter.
#💡 Iterate through all files in the specified folder.
for filename in Folder_IntervalReq.glob("*.xlsx"): #🧩 Get files in the Folder directory.
    #📃 Get the current modification timestamp and filename of the current file.
    current_modified_date_microseconds = datetime.datetime.fromtimestamp(filename.stat().st_mtime) # Get modification timestamp, including microseconds.
    current_modified_date = current_modified_date_microseconds.replace(microsecond=0) # Remove microseconds for easier comparison.
    file_basename = filename.name # Get the base name of the file (e.g., "data_20230101.csv").
    #📃 File is being processed.
    print_colored(f"Processing file🔄️: {file_basename}", "NavajoWhite")
    logger.info(f"Processing file🔄️: {file_basename}")
    #📃 Check if this file has been logged before.
    log_entry = log_df.filter(pl.col("FileName") == file_basename) # Filter the old log_df to find an entry for this filename.
    is_new_file = log_entry.is_empty() # If no entry is found (log_entry is empty), it's a new file.
    previous_modified_date = datetime.datetime(1900, 1, 1, 0, 0, 1)  # Initialize the previous modification date with a very old date.
    #📃 If this file was processed before (exists in the log).  
    if not is_new_file:
        previous_modified_date = log_entry.select("ModifiedDate").item() # Get the modification date recorded in the log from the previous processing.
    #📃 Condition to process the file:
    # 1. The file is new (not in the log).
    # 2. Or, the file is old (in the log), but its current modification date (current_modified_date) is more recent than the modification date recorded in the log (previous_modified_date).
    if is_new_file or current_modified_date > previous_modified_date:
        if not is_new_file: # If it's an old file but has changed (ModifiedDate is newer).
            print_colored(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.", "Coral")
            logger.warning(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.")
        else: # If it's a completely new file.
            print_colored(f"New file detected🚦: {file_basename}. Importing.", "Coral")
            logger.info(f"New file detected🚦: {file_basename}. Importing.")
        try: # Start a try-except block to handle potential errors during the import process.
            if not is_new_file: # If this file already exists in the DB (because it's not new and its ModifiedDate changed) then delete the old data for this file from the df.
                delete_data(engine, IntervalReq_TABLE_NAME, file_basename)  #🧩 Call the function to delete data.
            #📃 Read data from the excel file using Polars.
            IntervalReq = (pl.read_excel(filename, infer_schema_length=None, engine="calamine", sheet_name="Sheet1") #🧩 Read the entire file to infer the schema.Add two new columns to the DataFrame: FileName and ModifiedDate.
                    .select(pl.all(),pl.lit(file_basename).alias("FileName"),pl.lit(current_modified_date).dt.cast_time_unit("ms").alias("ModifiedDate")
                   )) 
            IntervalReq = IntervalReq.with_columns(pl.col('Attribute').dt.strftime('%Y-%m-%d'))
            IntervalReq = IntervalReq.with_columns(pl.col('Forecast').dt.strftime('%H:%M:%S'))
            IntervalReq = (
                IntervalReq.with_columns(
                    (pl.col("Value") / 95 * 100).alias("delivery_requirement"))
                .with_columns(
                    pl.col("delivery_requirement").round(2).alias("Delivery_Req")))
            IntervalReq = IntervalReq.with_columns(
                pl.format("{} {}", pl.col("Attribute"), pl.col("Forecast")).alias("Datetime_CET"))
            IntervalReq = IntervalReq.with_columns(pl.col('Datetime_CET'
                                ).str.strptime(pl.Datetime, format='%Y-%m-%d %H:%M:%S'
                                ).dt.replace_time_zone("Europe/Berlin", ambiguous="earliest"
                                ).dt.convert_time_zone("Asia/Bangkok").alias("Datetime_VN").dt.strftime('%Y-%m-%d %H:%M:%S')) #🧩
            IntervalReq = IntervalReq.with_columns(pl.col('Datetime_CET'
                                ).str.strptime(pl.Datetime, format='%Y-%m-%d %H:%M:%S').dt.strftime('%Y-%m-%d %H:%M:%S')
                                ).with_columns(pl.col("Datetime_CET").str.to_datetime("%Y-%m-%d %H:%M:%S").cast(pl.Datetime)
                                ).with_columns(pl.col("Datetime_VN").str.to_datetime("%Y-%m-%d %H:%M:%S").cast(pl.Datetime)) #🧩
            #📃 Validate if the columns in the DataFrame match the defined schema
            has_critical_error, critical_schema_error_msg = validate_schema(IntervalReq, IntervalReq_schema, file_basename) #🧩
            if has_critical_error: # If critical columns are missing.
                raise ValueError(critical_schema_error_msg) # Raise an error to stop processing this file.
            #📃 Adjust the column structure.
            print_colored(f"⚙️Adjust column structure", "Olive")
            logger.warning(f"⚙️Adjust column structure")
            IntervalReq = IntervalReq.select(IntervalReq_schema) #🧩 Select columns in the correct order and names as per df_schema.
            print_colored(f"⚙️Column structure adjustment completed", "Olive")
            logger.info(f"⚙️Column structure adjustment completed")
            info_polars(IntervalReq) #🧩 Display information about the DataFrame (shape, columns, dtypes).
            #📃 Start writing data to the database.
            print_colored(f"💾Start import file: {file_basename}", "Coral")
            logger.info(f"💾Start import file: {file_basename}")
            write_data(engine, IntervalReq_TABLE_NAME, IntervalReq) #🧩 Call the function to write the DataFrame to SQL.
            print_colored(f"Successfully imported✨: {file_basename}", "LimeGreen")
            logger.info(f"Successfully imported✨: {file_basename}")
            #📃 Record to summary
            if is_new_file:
                summary_results["new_files_imported"].append(f"[IntervalReq] {file_basename}")#🧩
            else:
                summary_results["updated_files_reimported"].append(f"[IntervalReq] {file_basename}")#🧩
            #📃 Update log_entries for successfully import: Add information to the log list for this run (successful).
            log_entries.append({"FileName": file_basename, "ModifiedDate": current_modified_date, "Error": None})
            print_colored(f"Updated 'Modified' time for {file_basename}", "LimeGreen")
            logger.info(f"Updated 'Modified' time for {file_basename}")
            #📃 Handle exceptions if any error occurred in the try block.
        except (
            FileNotFoundError, PermissionError, UnicodeDecodeError, IOError,                
            pl.exceptions.NoDataError, pl.exceptions.ComputeError, pl.exceptions.SchemaError, sa.exc.SQLAlchemyError,
            Exception # Catch various types of potential errors.
        ) as e:
            error_count += 1 # Increment the error counter.        
            error_msg_short = f"{type(e).__name__}: {str(e).splitlines()[0]}"  # Get a short error message (the first line).
            #📃 Update Failed file to Summary
            summary_results["failed_imports"].append((f"[IntervalReq] {file_basename}", error_msg_short))#🧩      
            logger.error(f"Error importing {file_basename}: {error_msg_short}", exc_info=True) # Log the full error traceback.
            print_colored(f"❗️Error importing {file_basename}: {error_msg_short}", "OrangeRed")
            # Add information to the log list for this run (failed).
            # Note: ModifiedDate here is previous_modified_date, as this file wasn't successfully imported with current_modified_date.
            log_entries.append({"FileName": file_basename,"ModifiedDate": previous_modified_date,"Error": error_msg_short})
    else: #📃 If the file is not new and its modification date hasn't changed.
          #📃 Update Skip file to Summary
        summary_results["skipped_files_unchanged"].append(f"[IntervalReq] {file_basename}")#🧩        
        print_colored(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.", "Aquamarine")
        logger.info(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.")
#💡 After iterating through all files in the directory.
display_summary("IntervalReq", error_count) #🧩 Display a summary of the results (number of errors).
process_and_save_log(log_df, log_entries, log_IntervalReq_path) #🧩 Update the Excel log file with information
logger.info("===== Processing of the IntervalReq data source is complete =====") #🧩 Log finish data processing.

In [ ]:
# 2️⃣8️⃣[BKN]ExceptionReq💾
logger.info("===== Start ExceptionReq Process =====")  #🧩 Log the start of the data processing.
log_df = read_or_create_log(log_ExceptionReq_path) #🧩 Read the existing Excel log file or create a new one if it doesn't exist.
log_entries, error_count = Default_variable() #📃 Initialize a list to store log results for the current run and an error counter.
#💡 Iterate through all files in the specified folder.
for filename in Folder_ExceptionReq.glob("*.xlsx"): #🧩 Get files in the Folder directory.
    #📃 Get the current modification timestamp and filename of the current file.
    current_modified_date_microseconds = datetime.datetime.fromtimestamp(filename.stat().st_mtime) # Get modification timestamp, including microseconds.
    current_modified_date = current_modified_date_microseconds.replace(microsecond=0) # Remove microseconds for easier comparison.
    file_basename = filename.name # Get the base name of the file (e.g., "data_20230101.csv").
    #📃 File is being processed.
    print_colored(f"Processing file🔄️: {file_basename}", "NavajoWhite")
    logger.info(f"Processing file🔄️: {file_basename}")
    #📃 Check if this file has been logged before.
    log_entry = log_df.filter(pl.col("FileName") == file_basename) # Filter the old log_df to find an entry for this filename.
    is_new_file = log_entry.is_empty() # If no entry is found (log_entry is empty), it's a new file.
    previous_modified_date = datetime.datetime(1900, 1, 1, 0, 0, 1)  # Initialize the previous modification date with a very old date.
    #📃 If this file was processed before (exists in the log).  
    if not is_new_file:
        previous_modified_date = log_entry.select("ModifiedDate").item() # Get the modification date recorded in the log from the previous processing.
    #📃 Condition to process the file:
    # 1. The file is new (not in the log).
    # 2. Or, the file is old (in the log), but its current modification date (current_modified_date) is more recent than the modification date recorded in the log (previous_modified_date).
    if is_new_file or current_modified_date > previous_modified_date:
        if not is_new_file: # If it's an old file but has changed (ModifiedDate is newer).
            print_colored(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.", "Coral")
            logger.warning(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.")
        else: # If it's a completely new file.
            print_colored(f"New file detected🚦: {file_basename}. Importing.", "Coral")
            logger.info(f"New file detected🚦: {file_basename}. Importing.")
        try: # Start a try-except block to handle potential errors during the import process.
            if not is_new_file: # If this file already exists in the DB (because it's not new and its ModifiedDate changed) then delete the old data for this file from the df.
                delete_data(engine, ExceptionReq_TABLE_NAME, file_basename)  #🧩 Call the function to delete data.
            #📃 Read data from the excel file using Polars.
            ExceptionReq = (pl.read_excel(filename, infer_schema_length=None, engine="calamine", sheet_name="Sheet1") #🧩 Read the entire file to infer the schema.Add two new columns to the DataFrame: FileName and ModifiedDate.
                    .select(pl.all(),pl.lit(file_basename).alias("FileName"),pl.lit(current_modified_date).dt.cast_time_unit("ms").alias("ModifiedDate")
                   )) #🧩
            #📃 Validate if the columns in the DataFrame match the defined schema
            has_critical_error, critical_schema_error_msg = validate_schema(ExceptionReq, ExceptionReq_schema, file_basename) #🧩
            if has_critical_error: # If critical columns are missing.
                raise ValueError(critical_schema_error_msg) # Raise an error to stop processing this file.
            #📃 Adjust the column structure.
            print_colored(f"⚙️Adjust column structure", "Olive")
            logger.warning(f"⚙️Adjust column structure")
            ExceptionReq = ExceptionReq.with_columns(pl.col("Date (MM/DD/YYYY)").cast(pl.Date)) #🧩
            ExceptionReq = ExceptionReq.with_columns(pl.col("Exception request (Minute)").cast(pl.Float64)) #🧩
            ExceptionReq = ExceptionReq.select(ExceptionReq_schema) #🧩 Select columns in the correct order and names as per df_schema.
            print_colored(f"⚙️Column structure adjustment completed", "Olive")
            logger.info(f"⚙️Column structure adjustment completed")
            info_polars(ExceptionReq) #🧩 Display information about the DataFrame (shape, columns, dtypes).
            #📃 Start writing data to the database.
            print_colored(f"💾Start import file: {file_basename}", "Coral")
            logger.info(f"💾Start import file: {file_basename}")
            write_data(engine, ExceptionReq_TABLE_NAME, ExceptionReq) #🧩 Call the function to write the DataFrame to SQL.
            print_colored(f"Successfully imported✨: {file_basename}", "LimeGreen")
            logger.info(f"Successfully imported✨: {file_basename}")
            #📃 Record to summary
            if is_new_file:
                summary_results["new_files_imported"].append(f"[ExceptionReq] {file_basename}")#🧩
            else:
                summary_results["updated_files_reimported"].append(f"[ExceptionReq] {file_basename}")#🧩    
            #📃 Update log_entries for successfully import: Add information to the log list for this run (successful).
            log_entries.append({"FileName": file_basename, "ModifiedDate": current_modified_date, "Error": None})
            print_colored(f"Updated 'Modified' time for {file_basename}", "LimeGreen")
            logger.info(f"Updated 'Modified' time for {file_basename}")
            #📃 Handle exceptions if any error occurred in the try block.
        except (
            FileNotFoundError, PermissionError, UnicodeDecodeError, IOError,                
            pl.exceptions.NoDataError, pl.exceptions.ComputeError, pl.exceptions.SchemaError, sa.exc.SQLAlchemyError,
            Exception # Catch various types of potential errors.
        ) as e:
            error_count += 1 # Increment the error counter.        
            error_msg_short = f"{type(e).__name__}: {str(e).splitlines()[0]}"  # Get a short error message (the first line).
            #📃 Update Failed file to Summary
            summary_results["failed_imports"].append((f"[ExceptionReq] {file_basename}", error_msg_short))#🧩      
            logger.error(f"Error importing {file_basename}: {error_msg_short}", exc_info=True) # Log the full error traceback.
            print_colored(f"❗️Error importing {file_basename}: {error_msg_short}", "OrangeRed")
            # Add information to the log list for this run (failed).
            # Note: ModifiedDate here is previous_modified_date, as this file wasn't successfully imported with current_modified_date.
            log_entries.append({"FileName": file_basename,"ModifiedDate": previous_modified_date,"Error": error_msg_short})
    else: #📃 If the file is not new and its modification date hasn't changed.
          #📃 Update Skip file to Summary
        summary_results["skipped_files_unchanged"].append(f"[ExceptionReq] {file_basename}")#🧩      
        print_colored(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.", "Aquamarine")
        logger.info(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.")
#💡 After iterating through all files in the directory.
display_summary("ExceptionReq", error_count) #🧩 Display a summary of the results (number of errors).
process_and_save_log(log_df, log_entries, log_ExceptionReq_path) #🧩 Update the Excel log file with information
logger.info("===== Processing of the ExceptionReq data source is complete =====") #🧩 Log finish data processing.

In [ ]:
# 2️⃣9️⃣[BKN]LTTransfers💾
logger.info("===== Start LTTransfers Process =====")  #🧩 Log the start of the data processing.
log_df = read_or_create_log(log_LTTransfers_path) #🧩 Read the existing Excel log file or create a new one if it doesn't exist.
log_entries, error_count = Default_variable() #📃 Initialize a list to store log results for the current run and an error counter.
#💡 Iterate through all files in the specified folder.
for filename in Folder_LTTransfers.glob("*.xlsx"): #🧩 Get files in the Folder directory.
    #📃 Get the current modification timestamp and filename of the current file.
    current_modified_date_microseconds = datetime.datetime.fromtimestamp(filename.stat().st_mtime) # Get modification timestamp, including microseconds.
    current_modified_date = current_modified_date_microseconds.replace(microsecond=0) # Remove microseconds for easier comparison.
    file_basename = filename.name # Get the base name of the file (e.g., "data_20230101.csv").
    #📃 File is being processed.
    print_colored(f"Processing file🔄️: {file_basename}", "NavajoWhite")
    logger.info(f"Processing file🔄️: {file_basename}")
    #📃 Check if this file has been logged before.
    log_entry = log_df.filter(pl.col("FileName") == file_basename) # Filter the old log_df to find an entry for this filename.
    is_new_file = log_entry.is_empty() # If no entry is found (log_entry is empty), it's a new file.
    previous_modified_date = datetime.datetime(1900, 1, 1, 0, 0, 1)  # Initialize the previous modification date with a very old date.
    #📃 If this file was processed before (exists in the log).  
    if not is_new_file:
        previous_modified_date = log_entry.select("ModifiedDate").item() # Get the modification date recorded in the log from the previous processing.
    #📃 Condition to process the file:
    # 1. The file is new (not in the log).
    # 2. Or, the file is old (in the log), but its current modification date (current_modified_date) is more recent than the modification date recorded in the log (previous_modified_date).
    if is_new_file or current_modified_date > previous_modified_date:
        if not is_new_file: # If it's an old file but has changed (ModifiedDate is newer).
            print_colored(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.", "Coral")
            logger.warning(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.")
        else: # If it's a completely new file.
            print_colored(f"New file detected🚦: {file_basename}. Importing.", "Coral")
            logger.info(f"New file detected🚦: {file_basename}. Importing.")
        try: # Start a try-except block to handle potential errors during the import process.
            if not is_new_file: # If this file already exists in the DB (because it's not new and its ModifiedDate changed) then delete the old data for this file from the df.
                delete_data(engine, LTTransfers_TABLE_NAME, file_basename)  #🧩 Call the function to delete data.
            #📃 Read data from the excel file using Polars.
            LTTransfers = (pl.read_excel(filename, infer_schema_length=None, engine="calamine", sheet_name="Sheet1") #🧩 Read the entire file to infer the schema.Add two new columns to the DataFrame: FileName and ModifiedDate.
                    .select(pl.all(),pl.lit(file_basename).alias("FileName"),pl.lit(current_modified_date).dt.cast_time_unit("ms").alias("ModifiedDate")
                   )) #🧩
            #📃 Validate if the columns in the DataFrame match the defined schema
            has_critical_error, critical_schema_error_msg = validate_schema(LTTransfers, LTTransfers_schema, file_basename) #🧩
            if has_critical_error: # If critical columns are missing.
                raise ValueError(critical_schema_error_msg) # Raise an error to stop processing this file.
            #📃 Adjust the column structure.
            print_colored(f"⚙️Adjust column structure", "Olive")
            logger.warning(f"⚙️Adjust column structure")
            LTTransfers = LTTransfers.with_columns(pl.col("LWD").cast(pl.Date)) #🧩
            LTTransfers = LTTransfers.select(LTTransfers_schema) #🧩 Select columns in the correct order and names as per df_schema.
            print_colored(f"⚙️Column structure adjustment completed", "Olive")
            logger.info(f"⚙️Column structure adjustment completed")
            info_polars(LTTransfers) #🧩 Display information about the DataFrame (shape, columns, dtypes).
            #📃 Start writing data to the database.
            print_colored(f"💾Start import file: {file_basename}", "Coral")
            logger.info(f"💾Start import file: {file_basename}")
            write_data(engine, LTTransfers_TABLE_NAME, LTTransfers) #🧩 Call the function to write the DataFrame to SQL.
            print_colored(f"Successfully imported✨: {file_basename}", "LimeGreen")
            logger.info(f"Successfully imported✨: {file_basename}")
            #📃 Record to summary
            if is_new_file:
                summary_results["new_files_imported"].append(f"[LTTransfers] {file_basename}")#🧩
            else:
                summary_results["updated_files_reimported"].append(f"[LTTransfers] {file_basename}")#🧩     
            #📃 Update log_entries for successfully import: Add information to the log list for this run (successful).
            log_entries.append({"FileName": file_basename, "ModifiedDate": current_modified_date, "Error": None})
            print_colored(f"Updated 'Modified' time for {file_basename}", "LimeGreen")
            logger.info(f"Updated 'Modified' time for {file_basename}")
            #📃 Handle exceptions if any error occurred in the try block.
        except (
            FileNotFoundError, PermissionError, UnicodeDecodeError, IOError,                
            pl.exceptions.NoDataError, pl.exceptions.ComputeError, pl.exceptions.SchemaError, sa.exc.SQLAlchemyError,
            Exception # Catch various types of potential errors.
        ) as e:
            error_count += 1 # Increment the error counter.        
            error_msg_short = f"{type(e).__name__}: {str(e).splitlines()[0]}"  # Get a short error message (the first line).
            #📃 Update Failed file to Summary
            summary_results["failed_imports"].append((f"[LTTransfers] {file_basename}", error_msg_short))#🧩    
            logger.error(f"Error importing {file_basename}: {error_msg_short}", exc_info=True) # Log the full error traceback.
            print_colored(f"❗️Error importing {file_basename}: {error_msg_short}", "OrangeRed")
            # Add information to the log list for this run (failed).
            # Note: ModifiedDate here is previous_modified_date, as this file wasn't successfully imported with current_modified_date.
            log_entries.append({"FileName": file_basename,"ModifiedDate": previous_modified_date,"Error": error_msg_short})
    else: #📃 If the file is not new and its modification date hasn't changed.
          #📃 Update Skip file to Summary
        summary_results["skipped_files_unchanged"].append(f"[LTTransfers] {file_basename}")#🧩
        print_colored(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.", "Aquamarine")
        logger.info(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.")
#💡 After iterating through all files in the directory.
display_summary("LTTransfers", error_count) #🧩 Display a summary of the results (number of errors).
process_and_save_log(log_df, log_entries, log_LTTransfers_path) #🧩 Update the Excel log file with information
logger.info("===== Processing of the LTTransfers data source is complete =====") #🧩 Log finish data processing.

In [ ]:
# 3️⃣0️⃣[BKN]DailyReq💾
logger.info("===== Start DailyReq Process =====")  #🧩 Log the start of the data processing.
log_df = read_or_create_log(log_DailyReq_path) #🧩 Read the existing Excel log file or create a new one if it doesn't exist.
log_entries, error_count = Default_variable() #📃 Initialize a list to store log results for the current run and an error counter.
#💡 Iterate through all files in the specified folder.
for filename in Folder_DailyReq.glob("*.xlsx"): #🧩 Get files in the Folder directory.
    #📃 Get the current modification timestamp and filename of the current file.
    current_modified_date_microseconds = datetime.datetime.fromtimestamp(filename.stat().st_mtime) # Get modification timestamp, including microseconds.
    current_modified_date = current_modified_date_microseconds.replace(microsecond=0) # Remove microseconds for easier comparison.
    file_basename = filename.name # Get the base name of the file (e.g., "data_20230101.csv").
    #📃 File is being processed.
    print_colored(f"Processing file🔄️: {file_basename}", "NavajoWhite")
    logger.info(f"Processing file🔄️: {file_basename}")
    #📃 Check if this file has been logged before.
    log_entry = log_df.filter(pl.col("FileName") == file_basename) # Filter the old log_df to find an entry for this filename.
    is_new_file = log_entry.is_empty() # If no entry is found (log_entry is empty), it's a new file.
    previous_modified_date = datetime.datetime(1900, 1, 1, 0, 0, 1)  # Initialize the previous modification date with a very old date.
    #📃 If this file was processed before (exists in the log).  
    if not is_new_file:
        previous_modified_date = log_entry.select("ModifiedDate").item() # Get the modification date recorded in the log from the previous processing.
    #📃 Condition to process the file:
    # 1. The file is new (not in the log).
    # 2. Or, the file is old (in the log), but its current modification date (current_modified_date) is more recent than the modification date recorded in the log (previous_modified_date).
    if is_new_file or current_modified_date > previous_modified_date:
        if not is_new_file: # If it's an old file but has changed (ModifiedDate is newer).
            print_colored(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.", "Coral")
            logger.warning(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.")
        else: # If it's a completely new file.
            print_colored(f"New file detected🚦: {file_basename}. Importing.", "Coral")
            logger.info(f"New file detected🚦: {file_basename}. Importing.")
        try: # Start a try-except block to handle potential errors during the import process.
            if not is_new_file: # If this file already exists in the DB (because it's not new and its ModifiedDate changed) then delete the old data for this file from the df.
                delete_data(engine, DailyReq_TABLE_NAME, file_basename)  #🧩 Call the function to delete data.
            #📃 Read data from the excel file using Polars.
            DailyReq = (pl.read_excel(filename, infer_schema_length=None, engine="calamine", sheet_name="Sheet1") #🧩 Read the entire file to infer the schema.Add two new columns to the DataFrame: FileName and ModifiedDate.
                    .select(pl.all(),pl.lit(file_basename).alias("FileName"),pl.lit(current_modified_date).dt.cast_time_unit("ms").alias("ModifiedDate")
                   )) #🧩
            #📃 Validate if the columns in the DataFrame match the defined schema
            has_critical_error, critical_schema_error_msg = validate_schema(DailyReq, DailyReq_schema, file_basename) #🧩
            if has_critical_error: # If critical columns are missing.
                raise ValueError(critical_schema_error_msg) # Raise an error to stop processing this file.
            #📃 Adjust the column structure.
            print_colored(f"⚙️Adjust column structure", "Olive")
            logger.warning(f"⚙️Adjust column structure")
            DailyReq = DailyReq.with_columns(pl.col("Date").cast(pl.Date)) #🧩
            DailyReq = DailyReq.with_columns(pl.col("Daily Requirement", "Prod Requirement").cast(pl.Float64)) #🧩
            DailyReq = DailyReq.select(DailyReq_schema) #🧩 Select columns in the correct order and names as per df_schema.
            print_colored(f"⚙️Column structure adjustment completed", "Olive")
            logger.info(f"⚙️Column structure adjustment completed")
            info_polars(DailyReq) #🧩 Display information about the DataFrame (shape, columns, dtypes).
            #📃 Start writing data to the database.
            print_colored(f"💾Start import file: {file_basename}", "Coral")
            logger.info(f"💾Start import file: {file_basename}")
            write_data(engine, DailyReq_TABLE_NAME, DailyReq) #🧩 Call the function to write the DataFrame to SQL.
            print_colored(f"Successfully imported✨: {file_basename}", "LimeGreen")
            logger.info(f"Successfully imported✨: {file_basename}")
            #📃 Record to summary
            if is_new_file:
                summary_results["new_files_imported"].append(f"[DailyReq] {file_basename}")#🧩
            else:
                summary_results["updated_files_reimported"].append(f"[DailyReq] {file_basename}")#🧩     
            #📃 Update log_entries for successfully import: Add information to the log list for this run (successful).
            log_entries.append({"FileName": file_basename, "ModifiedDate": current_modified_date, "Error": None})
            print_colored(f"Updated 'Modified' time for {file_basename}", "LimeGreen")
            logger.info(f"Updated 'Modified' time for {file_basename}")
            #📃 Handle exceptions if any error occurred in the try block.
        except (
            FileNotFoundError, PermissionError, UnicodeDecodeError, IOError,                
            pl.exceptions.NoDataError, pl.exceptions.ComputeError, pl.exceptions.SchemaError, sa.exc.SQLAlchemyError,
            Exception # Catch various types of potential errors.
        ) as e:
            error_count += 1 # Increment the error counter.        
            error_msg_short = f"{type(e).__name__}: {str(e).splitlines()[0]}"  # Get a short error message (the first line).
            #📃 Update Failed file to Summary
            summary_results["failed_imports"].append((f"[DailyReq] {file_basename}", error_msg_short))#🧩            
            logger.error(f"Error importing {file_basename}: {error_msg_short}", exc_info=True) # Log the full error traceback.
            print_colored(f"❗️Error importing {file_basename}: {error_msg_short}", "OrangeRed")
            # Add information to the log list for this run (failed).
            # Note: ModifiedDate here is previous_modified_date, as this file wasn't successfully imported with current_modified_date.
            log_entries.append({"FileName": file_basename,"ModifiedDate": previous_modified_date,"Error": error_msg_short})
    else: #📃 If the file is not new and its modification date hasn't changed.
          #📃 Update Skip file to Summary
        summary_results["skipped_files_unchanged"].append(f"[DailyReq] {file_basename}")#🧩  
        print_colored(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.", "Aquamarine")
        logger.info(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.")
#💡 After iterating through all files in the directory.
display_summary("DailyReq", error_count) #🧩 Display a summary of the results (number of errors).
process_and_save_log(log_df, log_entries, log_DailyReq_path) #🧩 Update the Excel log file with information
logger.info("===== Processing of the DailyReq data source is complete =====") #🧩 Log finish data processing.

In [ ]:
# 3️⃣1️⃣[BKN]ProjectedShrink💾
logger.info("===== Start ProjectedShrink Process =====")  #🧩 Log the start of the data processing.
log_df = read_or_create_log(log_ProjectedShrink_path) #🧩 Read the existing Excel log file or create a new one if it doesn't exist.
log_entries, error_count = Default_variable() #📃 Initialize a list to store log results for the current run and an error counter.
#💡 Iterate through all files in the specified folder.
for filename in Folder_ProjectedShrink.glob("*.xlsx"): #🧩 Get files in the Folder directory.
    #📃 Get the current modification timestamp and filename of the current file.
    current_modified_date_microseconds = datetime.datetime.fromtimestamp(filename.stat().st_mtime) # Get modification timestamp, including microseconds.
    current_modified_date = current_modified_date_microseconds.replace(microsecond=0) # Remove microseconds for easier comparison.
    file_basename = filename.name # Get the base name of the file (e.g., "data_20230101.csv").
    #📃 File is being processed.
    print_colored(f"Processing file🔄️: {file_basename}", "NavajoWhite")
    logger.info(f"Processing file🔄️: {file_basename}")
    #📃 Check if this file has been logged before.
    log_entry = log_df.filter(pl.col("FileName") == file_basename) # Filter the old log_df to find an entry for this filename.
    is_new_file = log_entry.is_empty() # If no entry is found (log_entry is empty), it's a new file.
    previous_modified_date = datetime.datetime(1900, 1, 1, 0, 0, 1)  # Initialize the previous modification date with a very old date.
    #📃 If this file was processed before (exists in the log).  
    if not is_new_file:
        previous_modified_date = log_entry.select("ModifiedDate").item() # Get the modification date recorded in the log from the previous processing.
    #📃 Condition to process the file:
    # 1. The file is new (not in the log).
    # 2. Or, the file is old (in the log), but its current modification date (current_modified_date) is more recent than the modification date recorded in the log (previous_modified_date).
    if is_new_file or current_modified_date > previous_modified_date:
        if not is_new_file: # If it's an old file but has changed (ModifiedDate is newer).
            print_colored(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.", "Coral")
            logger.warning(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.")
        else: # If it's a completely new file.
            print_colored(f"New file detected🚦: {file_basename}. Importing.", "Coral")
            logger.info(f"New file detected🚦: {file_basename}. Importing.")
        try: # Start a try-except block to handle potential errors during the import process.
            if not is_new_file: # If this file already exists in the DB (because it's not new and its ModifiedDate changed) then delete the old data for this file from the df.
                delete_data(engine, ProjectedShrink_TABLE_NAME, file_basename)  #🧩 Call the function to delete data.
            #📃 Read data from the excel file using Polars.
            ProjectedShrink = (pl.read_excel(filename, infer_schema_length=None, engine="calamine", sheet_name="Sheet1") #🧩 Read the entire file to infer the schema.Add two new columns to the DataFrame: FileName and ModifiedDate.
                    .select(pl.all(),pl.lit(file_basename).alias("FileName"),pl.lit(current_modified_date).dt.cast_time_unit("ms").alias("ModifiedDate")
                   )) #🧩
            #📃 Validate if the columns in the DataFrame match the defined schema
            has_critical_error, critical_schema_error_msg = validate_schema(ProjectedShrink, ProjectedShrink_schema, file_basename) #🧩
            if has_critical_error: # If critical columns are missing.
                raise ValueError(critical_schema_error_msg) # Raise an error to stop processing this file.
            #📃 Adjust the column structure.
            print_colored(f"⚙️Adjust column structure", "Olive")
            logger.warning(f"⚙️Adjust column structure")
            ProjectedShrink = ProjectedShrink.with_columns(pl.col("Week").cast(pl.Int64)) #🧩
            ProjectedShrink = ProjectedShrink.with_columns(pl.col("Ratio").cast(pl.Float64)) #🧩
            ProjectedShrink = ProjectedShrink.select(ProjectedShrink_schema) #🧩 Select columns in the correct order and names as per df_schema.
            print_colored(f"⚙️Column structure adjustment completed", "Olive")
            logger.info(f"⚙️Column structure adjustment completed")
            info_polars(ProjectedShrink) #🧩 Display information about the DataFrame (shape, columns, dtypes).
            #📃 Start writing data to the database.
            print_colored(f"💾Start import file: {file_basename}", "Coral")
            logger.info(f"💾Start import file: {file_basename}")
            write_data(engine, ProjectedShrink_TABLE_NAME, ProjectedShrink) #🧩 Call the function to write the DataFrame to SQL.
            print_colored(f"Successfully imported✨: {file_basename}", "LimeGreen")
            logger.info(f"Successfully imported✨: {file_basename}")
            #📃 Record to summary
            if is_new_file:
                summary_results["new_files_imported"].append(f"[ProjectedShrink] {file_basename}")#🧩
            else:
                summary_results["updated_files_reimported"].append(f"[ProjectedShrink] {file_basename}")#🧩
            #📃 Update log_entries for successfully import: Add information to the log list for this run (successful).
            log_entries.append({"FileName": file_basename, "ModifiedDate": current_modified_date, "Error": None})
            print_colored(f"Updated 'Modified' time for {file_basename}", "LimeGreen")
            logger.info(f"Updated 'Modified' time for {file_basename}")
            #📃 Handle exceptions if any error occurred in the try block.
        except (
            FileNotFoundError, PermissionError, UnicodeDecodeError, IOError,                
            pl.exceptions.NoDataError, pl.exceptions.ComputeError, pl.exceptions.SchemaError, sa.exc.SQLAlchemyError,
            Exception # Catch various types of potential errors.
        ) as e:
            error_count += 1 # Increment the error counter.        
            error_msg_short = f"{type(e).__name__}: {str(e).splitlines()[0]}"  # Get a short error message (the first line).
            #📃 Update Failed file to Summary
            summary_results["failed_imports"].append((f"[ProjectedShrink] {file_basename}", error_msg_short))#🧩
            logger.error(f"Error importing {file_basename}: {error_msg_short}", exc_info=True) # Log the full error traceback.
            print_colored(f"❗️Error importing {file_basename}: {error_msg_short}", "OrangeRed")
            # Add information to the log list for this run (failed).
            # Note: ModifiedDate here is previous_modified_date, as this file wasn't successfully imported with current_modified_date.
            log_entries.append({"FileName": file_basename,"ModifiedDate": previous_modified_date,"Error": error_msg_short})
    else: #📃 If the file is not new and its modification date hasn't changed.
          #📃 Update Skip file to Summary
        summary_results["skipped_files_unchanged"].append(f"[ProjectedShrink] {file_basename}")#🧩
        print_colored(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.", "Aquamarine")
        logger.info(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.")
#💡 After iterating through all files in the directory.
display_summary("ProjectedShrink", error_count) #🧩 Display a summary of the results (number of errors).
process_and_save_log(log_df, log_entries, log_ProjectedShrink_path) #🧩 Update the Excel log file with information
logger.info("===== Processing of the ProjectedShrink data source is complete =====") #🧩 Log finish data processing.

In [ ]:
# 3️⃣2️⃣[BKN]OTReq💾
logger.info("===== Start OTReq Process =====")  #🧩 Log the start of the data processing.
log_df = read_or_create_log(log_OTReq_path) #🧩 Read the existing Excel log file or create a new one if it doesn't exist.
log_entries, error_count = Default_variable() #📃 Initialize a list to store log results for the current run and an error counter.
#💡 Iterate through all files in the specified folder.
for filename in Folder_OTReq.glob("*.xlsx"): #🧩 Get files in the Folder directory.
    #📃 Get the current modification timestamp and filename of the current file.
    current_modified_date_microseconds = datetime.datetime.fromtimestamp(filename.stat().st_mtime) # Get modification timestamp, including microseconds.
    current_modified_date = current_modified_date_microseconds.replace(microsecond=0) # Remove microseconds for easier comparison.
    file_basename = filename.name # Get the base name of the file (e.g., "data_20230101.csv").
    #📃 File is being processed.
    print_colored(f"Processing file🔄️: {file_basename}", "NavajoWhite")
    logger.info(f"Processing file🔄️: {file_basename}")
    #📃 Check if this file has been logged before.
    log_entry = log_df.filter(pl.col("FileName") == file_basename) # Filter the old log_df to find an entry for this filename.
    is_new_file = log_entry.is_empty() # If no entry is found (log_entry is empty), it's a new file.
    previous_modified_date = datetime.datetime(1900, 1, 1, 0, 0, 1)  # Initialize the previous modification date with a very old date.
    #📃 If this file was processed before (exists in the log).  
    if not is_new_file:
        previous_modified_date = log_entry.select("ModifiedDate").item() # Get the modification date recorded in the log from the previous processing.
    #📃 Condition to process the file:
    # 1. The file is new (not in the log).
    # 2. Or, the file is old (in the log), but its current modification date (current_modified_date) is more recent than the modification date recorded in the log (previous_modified_date).
    if is_new_file or current_modified_date > previous_modified_date:
        if not is_new_file: # If it's an old file but has changed (ModifiedDate is newer).
            print_colored(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.", "Coral")
            logger.warning(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.")
        else: # If it's a completely new file.
            print_colored(f"New file detected🚦: {file_basename}. Importing.", "Coral")
            logger.info(f"New file detected🚦: {file_basename}. Importing.")
        try: # Start a try-except block to handle potential errors during the import process.
            if not is_new_file: # If this file already exists in the DB (because it's not new and its ModifiedDate changed) then delete the old data for this file from the df.
                delete_data(engine, OTReq_TABLE_NAME, file_basename)  #🧩 Call the function to delete data.
            #📃 Read data from the excel file using Polars.
            OTReq = (pl.read_excel(filename, infer_schema_length=None, engine="calamine", sheet_name="Sheet1") #🧩 Read the entire file to infer the schema.Add two new columns to the DataFrame: FileName and ModifiedDate.
                    .select(pl.all(),pl.lit(file_basename).alias("FileName"),pl.lit(current_modified_date).dt.cast_time_unit("ms").alias("ModifiedDate")
                   )) # Import Schema🧩
            #📃 Validate if the columns in the DataFrame match the defined schema
            has_critical_error, critical_schema_error_msg = validate_schema(OTReq, OTReq_schema, file_basename) #🧩
            if has_critical_error: # If critical columns are missing.
                raise ValueError(critical_schema_error_msg) # Raise an error to stop processing this file.
            #📃 Adjust the column structure.
            print_colored(f"⚙️Adjust column structure", "Olive")
            logger.warning(f"⚙️Adjust column structure")
            OTReq = OTReq.with_columns(pl.col("Date").cast(pl.Date)) #🧩
            OTReq = OTReq.with_columns(pl.col("OT Hour").cast(pl.Float64)) #🧩
            OTReq = OTReq.select(OTReq_schema) #🧩 Select columns in the correct order and names as per df_schema.
            print_colored(f"⚙️Column structure adjustment completed", "Olive")
            logger.info(f"⚙️Column structure adjustment completed")
            info_polars(OTReq) #🧩 Display information about the DataFrame (shape, columns, dtypes).
            #📃 Start writing data to the database.
            print_colored(f"💾Start import file: {file_basename}", "Coral")
            logger.info(f"💾Start import file: {file_basename}")
            write_data(engine, OTReq_TABLE_NAME, OTReq) #🧩 Call the function to write the DataFrame to SQL.
            print_colored(f"Successfully imported✨: {file_basename}", "LimeGreen")
            logger.info(f"Successfully imported✨: {file_basename}")
            #📃 Record to summary
            if is_new_file:
                summary_results["new_files_imported"].append(f"[OTReq] {file_basename}")#🧩
            else:
                summary_results["updated_files_reimported"].append(f"[OTReq] {file_basename}")#🧩   
            #📃 Update log_entries for successfully import: Add information to the log list for this run (successful).
            log_entries.append({"FileName": file_basename, "ModifiedDate": current_modified_date, "Error": None})
            print_colored(f"Updated 'Modified' time for {file_basename}", "LimeGreen")
            logger.info(f"Updated 'Modified' time for {file_basename}")
            #📃 Handle exceptions if any error occurred in the try block.
        except (
            FileNotFoundError, PermissionError, UnicodeDecodeError, IOError,                
            pl.exceptions.NoDataError, pl.exceptions.ComputeError, pl.exceptions.SchemaError, sa.exc.SQLAlchemyError,
            Exception # Catch various types of potential errors.
        ) as e:
            error_count += 1 # Increment the error counter.        
            error_msg_short = f"{type(e).__name__}: {str(e).splitlines()[0]}"  # Get a short error message (the first line).
            #📃 Update Failed file to Summary
            summary_results["failed_imports"].append((f"[OTReq] {file_basename}", error_msg_short))#🧩
            logger.error(f"Error importing {file_basename}: {error_msg_short}", exc_info=True) # Log the full error traceback.
            print_colored(f"❗️Error importing {file_basename}: {error_msg_short}", "OrangeRed")
            # Add information to the log list for this run (failed).
            # Note: ModifiedDate here is previous_modified_date, as this file wasn't successfully imported with current_modified_date.
            log_entries.append({"FileName": file_basename,"ModifiedDate": previous_modified_date,"Error": error_msg_short})
    else: #📃 If the file is not new and its modification date hasn't changed.
          #📃 Update Skip file to Summary
        summary_results["skipped_files_unchanged"].append(f"[OTReq] {file_basename}")#🧩   
        print_colored(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.", "Aquamarine")
        logger.info(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.")
#💡 After iterating through all files in the directory.
display_summary("OTReq", error_count) #🧩 Display a summary of the results (number of errors).
process_and_save_log(log_df, log_entries, log_OTReq_path) #🧩 Update the Excel log file with information
logger.info("===== Processing of the OTReq data source is complete =====") #🧩 Log finish data processing.

In [ ]:
# 3️⃣3️⃣[BKN]CapHC💾
logger.info("===== Start CapHC Process =====")  #🧩 Log the start of the data processing.
log_df = read_or_create_log(log_CapHC_path) #🧩 Read the existing Excel log file or create a new one if it doesn't exist.
log_entries, error_count = Default_variable() #📃 Initialize a list to store log results for the current run and an error counter.
#💡 Iterate through all files in the specified folder.
for filename in Folder_CapHC.glob("*.xlsx"): #🧩 Get files in the Folder directory.
    #📃 Get the current modification timestamp and filename of the current file.
    current_modified_date_microseconds = datetime.datetime.fromtimestamp(filename.stat().st_mtime) # Get modification timestamp, including microseconds.
    current_modified_date = current_modified_date_microseconds.replace(microsecond=0) # Remove microseconds for easier comparison.
    file_basename = filename.name # Get the base name of the file (e.g., "data_20230101.csv").
    #📃 File is being processed.
    print_colored(f"Processing file🔄️: {file_basename}", "NavajoWhite")
    logger.info(f"Processing file🔄️: {file_basename}")
    #📃 Check if this file has been logged before.
    log_entry = log_df.filter(pl.col("FileName") == file_basename) # Filter the old log_df to find an entry for this filename.
    is_new_file = log_entry.is_empty() # If no entry is found (log_entry is empty), it's a new file.
    previous_modified_date = datetime.datetime(1900, 1, 1, 0, 0, 1)  # Initialize the previous modification date with a very old date.
    #📃 If this file was processed before (exists in the log).  
    if not is_new_file:
        previous_modified_date = log_entry.select("ModifiedDate").item() # Get the modification date recorded in the log from the previous processing.
    #📃 Condition to process the file:
    # 1. The file is new (not in the log).
    # 2. Or, the file is old (in the log), but its current modification date (current_modified_date) is more recent than the modification date recorded in the log (previous_modified_date).
    if is_new_file or current_modified_date > previous_modified_date:
        if not is_new_file: # If it's an old file but has changed (ModifiedDate is newer).
            print_colored(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.", "Coral")
            logger.warning(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.")
        else: # If it's a completely new file.
            print_colored(f"New file detected🚦: {file_basename}. Importing.", "Coral")
            logger.info(f"New file detected🚦: {file_basename}. Importing.")
        try: # Start a try-except block to handle potential errors during the import process.
            if not is_new_file: # If this file already exists in the DB (because it's not new and its ModifiedDate changed) then delete the old data for this file from the df.
                delete_data(engine, CapHC_TABLE_NAME, file_basename)  #🧩 Call the function to delete data.
            #📃 Read data from the excel file using Polars.
            CapHC = (pl.read_excel(filename, infer_schema_length=None, engine="calamine", sheet_name="Sheet1") #🧩 Read the entire file to infer the schema.Add two new columns to the DataFrame: FileName and ModifiedDate.
                    .select(pl.all(),pl.lit(file_basename).alias("FileName"),pl.lit(current_modified_date).dt.cast_time_unit("ms").alias("ModifiedDate")
                   )) #🧩
            #📃 Validate if the columns in the DataFrame match the defined schema
            has_critical_error, critical_schema_error_msg = validate_schema(CapHC, CapHC_schema, file_basename) #🧩
            if has_critical_error: # If critical columns are missing.
                raise ValueError(critical_schema_error_msg) # Raise an error to stop processing this file.
            #📃 Adjust the column structure.
            print_colored(f"⚙️Adjust column structure", "Olive")
            logger.warning(f"⚙️Adjust column structure")
            CapHC = CapHC.with_columns(pl.col("Date").cast(pl.Date)) #🧩
            CapHC = CapHC.with_columns(pl.col("Client Requirement (Hours)").cast(pl.Float64)) #🧩
            CapHC = CapHC.select(CapHC_schema) #🧩 Select columns in the correct order and names as per df_schema.
            print_colored(f"⚙️Column structure adjustment completed", "Olive")
            logger.info(f"⚙️Column structure adjustment completed")
            info_polars(CapHC) #🧩 Display information about the DataFrame (shape, columns, dtypes).
            #📃 Start writing data to the database.
            print_colored(f"💾Start import file: {file_basename}", "Coral")
            logger.info(f"💾Start import file: {file_basename}")
            write_data(engine, CapHC_TABLE_NAME, CapHC) #🧩 Call the function to write the DataFrame to SQL.
            print_colored(f"Successfully imported✨: {file_basename}", "LimeGreen")
            logger.info(f"Successfully imported✨: {file_basename}")
            #📃 Record to summary
            if is_new_file:
                summary_results["new_files_imported"].append(f"[CapHC] {file_basename}")#🧩
            else:
                summary_results["updated_files_reimported"].append(f"[CapHC] {file_basename}")#🧩    
            #📃 Update log_entries for successfully import: Add information to the log list for this run (successful).
            log_entries.append({"FileName": file_basename, "ModifiedDate": current_modified_date, "Error": None})
            print_colored(f"Updated 'Modified' time for {file_basename}", "LimeGreen")
            logger.info(f"Updated 'Modified' time for {file_basename}")
            #📃 Handle exceptions if any error occurred in the try block.
        except (
            FileNotFoundError, PermissionError, UnicodeDecodeError, IOError,                
            pl.exceptions.NoDataError, pl.exceptions.ComputeError, pl.exceptions.SchemaError, sa.exc.SQLAlchemyError,
            Exception # Catch various types of potential errors.
        ) as e:
            error_count += 1 # Increment the error counter.        
            error_msg_short = f"{type(e).__name__}: {str(e).splitlines()[0]}"  # Get a short error message (the first line).
            #📃 Update Failed file to Summary
            summary_results["failed_imports"].append((f"[CapHC] {file_basename}", error_msg_short))#🧩
            logger.error(f"Error importing {file_basename}: {error_msg_short}", exc_info=True) # Log the full error traceback.
            print_colored(f"❗️Error importing {file_basename}: {error_msg_short}", "OrangeRed")
            # Add information to the log list for this run (failed).
            # Note: ModifiedDate here is previous_modified_date, as this file wasn't successfully imported with current_modified_date.
            log_entries.append({"FileName": file_basename,"ModifiedDate": previous_modified_date,"Error": error_msg_short})
    else: #📃 If the file is not new and its modification date hasn't changed.
          #📃 Update Skip file to Summary
        summary_results["skipped_files_unchanged"].append(f"[CapHC] {file_basename}")#🧩      
        print_colored(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.", "Aquamarine")
        logger.info(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.")
#💡 After iterating through all files in the directory.
display_summary("CapHC", error_count) #🧩 Display a summary of the results (number of errors).
process_and_save_log(log_df, log_entries, log_CapHC_path) #🧩 Update the Excel log file with information
logger.info("===== Processing of the CapHC data source is complete =====") #🧩 Log finish data processing.

In [ ]:
# 3️⃣4️⃣[BKN]ProjectedHC💾
logger.info("===== Start ProjectedHC Process =====")  #🧩 Log the start of the data processing.
log_df = read_or_create_log(log_ProjectedHC_path) #🧩 Read the existing Excel log file or create a new one if it doesn't exist.
log_entries, error_count = Default_variable() #📃 Initialize a list to store log results for the current run and an error counter.
#💡 Iterate through all files in the specified folder.
for filename in Folder_ProjectedHC.glob("*.xlsm"): #🧩 Get files in the Folder directory.
    #📃 Get the current modification timestamp and filename of the current file.
    current_modified_date_microseconds = datetime.datetime.fromtimestamp(filename.stat().st_mtime) # Get modification timestamp, including microseconds.
    current_modified_date = current_modified_date_microseconds.replace(microsecond=0) # Remove microseconds for easier comparison.
    file_basename = filename.name # Get the base name of the file (e.g., "data_20230101.csv").
    #📃 File is being processed.
    print_colored(f"Processing file🔄️: {file_basename}", "NavajoWhite")
    logger.info(f"Processing file🔄️: {file_basename}")
    #📃 Check if this file has been logged before.
    log_entry = log_df.filter(pl.col("FileName") == file_basename) # Filter the old log_df to find an entry for this filename.
    is_new_file = log_entry.is_empty() # If no entry is found (log_entry is empty), it's a new file.
    previous_modified_date = datetime.datetime(1900, 1, 1, 0, 0, 1)  # Initialize the previous modification date with a very old date.
    #📃 If this file was processed before (exists in the log).  
    if not is_new_file:
        previous_modified_date = log_entry.select("ModifiedDate").item() # Get the modification date recorded in the log from the previous processing.
    #📃 Condition to process the file:
    # 1. The file is new (not in the log).
    # 2. Or, the file is old (in the log), but its current modification date (current_modified_date) is more recent than the modification date recorded in the log (previous_modified_date).
    if is_new_file or current_modified_date > previous_modified_date:
        if not is_new_file: # If it's an old file but has changed (ModifiedDate is newer).
            print_colored(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.", "Coral")
            logger.warning(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.")
        else: # If it's a completely new file.
            print_colored(f"New file detected🚦: {file_basename}. Importing.", "Coral")
            logger.info(f"New file detected🚦: {file_basename}. Importing.")
        try: # Start a try-except block to handle potential errors during the import process.
            if not is_new_file: # If this file already exists in the DB (because it's not new and its ModifiedDate changed) then delete the old data for this file from the df.
                delete_data(engine, ProjectedHC_TABLE_NAME, file_basename)  #🧩 Call the function to delete data.
            #📃 Read data from the excel file using Polars.
            ProjectedHC = (pl.read_excel(filename, infer_schema_length=None, engine="calamine", sheet_name="rawDup") #🧩 Read the entire file to infer the schema.Add two new columns to the DataFrame: FileName and ModifiedDate.
                    .select(pl.all(),pl.lit(file_basename).alias("FileName"),pl.lit(current_modified_date).dt.cast_time_unit("ms").alias("ModifiedDate")
                   )) #🧩
            #📃 Validate if the columns in the DataFrame match the defined schema
            has_critical_error, critical_schema_error_msg = validate_schema(ProjectedHC, ProjectedHC_schema, file_basename) #🧩
            if has_critical_error: # If critical columns are missing.
                raise ValueError(critical_schema_error_msg) # Raise an error to stop processing this file.
            #📃 Adjust the column structure.
            print_colored(f"⚙️Adjust column structure", "Olive")
            logger.warning(f"⚙️Adjust column structure")
            ProjectedHC = ProjectedHC.with_columns(pl.col("Date").cast(pl.Date)) #🧩
            ProjectedHC = ProjectedHC.with_columns(pl.col('FTE Required', 'Projected HC', 'Plan Leave', 'Actual Projected HC', '%OO', '%IO', 
                                                          'Projected HC with Shrink', 'OT', 'Leave allow for Shrink', '% Deli').cast(pl.Float64)) #🧩
            ProjectedHC = ProjectedHC.select(ProjectedHC_schema) #🧩 Select columns in the correct order and names as per df_schema.
            print_colored(f"⚙️Column structure adjustment completed", "Olive")
            logger.info(f"⚙️Column structure adjustment completed")
            info_polars(ProjectedHC) #🧩 Display information about the DataFrame (shape, columns, dtypes).
            #📃 Start writing data to the database.
            print_colored(f"💾Start import file: {file_basename}", "Coral")
            logger.info(f"💾Start import file: {file_basename}")
            write_data(engine, ProjectedHC_TABLE_NAME, ProjectedHC) #🧩 Call the function to write the DataFrame to SQL.
            print_colored(f"Successfully imported✨: {file_basename}", "LimeGreen")
            logger.info(f"Successfully imported✨: {file_basename}")
            #📃 Record to summary
            if is_new_file:
                summary_results["new_files_imported"].append(f"[ProjectedHC] {file_basename}")#🧩
            else:
                summary_results["updated_files_reimported"].append(f"[ProjectedHC] {file_basename}")#🧩    
            #📃 Update log_entries for successfully import: Add information to the log list for this run (successful).
            log_entries.append({"FileName": file_basename, "ModifiedDate": current_modified_date, "Error": None})
            print_colored(f"Updated 'Modified' time for {file_basename}", "LimeGreen")
            logger.info(f"Updated 'Modified' time for {file_basename}")
            #📃 Handle exceptions if any error occurred in the try block.
        except (
            FileNotFoundError, PermissionError, UnicodeDecodeError, IOError,                
            pl.exceptions.NoDataError, pl.exceptions.ComputeError, pl.exceptions.SchemaError, sa.exc.SQLAlchemyError,
            Exception # Catch various types of potential errors.
        ) as e:
            error_count += 1 # Increment the error counter.        
            error_msg_short = f"{type(e).__name__}: {str(e).splitlines()[0]}"  # Get a short error message (the first line).
            #📃 Update Failed file to Summary
            summary_results["failed_imports"].append((f"[ProjectedHC] {file_basename}", error_msg_short))#🧩
            logger.error(f"Error importing {file_basename}: {error_msg_short}", exc_info=True) # Log the full error traceback.
            print_colored(f"❗️Error importing {file_basename}: {error_msg_short}", "OrangeRed")
            # Add information to the log list for this run (failed).
            # Note: ModifiedDate here is previous_modified_date, as this file wasn't successfully imported with current_modified_date.
            log_entries.append({"FileName": file_basename,"ModifiedDate": previous_modified_date,"Error": error_msg_short})
    else: #📃 If the file is not new and its modification date hasn't changed.

          #📃 Update Skip file to Summary
        summary_results["skipped_files_unchanged"].append(f"[ProjectedHC] {file_basename}")#🧩
        print_colored(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.", "Aquamarine")
        logger.info(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.")
#💡 After iterating through all files in the directory.
display_summary("ProjectedHC", error_count) #🧩 Display a summary of the results (number of errors).
process_and_save_log(log_df, log_entries, log_ProjectedHC_path) #🧩 Update the Excel log file with information
logger.info("===== Processing of the ProjectedHC data source is complete =====") #🧩 Log finish data processing.

In [ ]:
# 3️⃣5️⃣[BKN]RampHC💾
logger.info("===== Start RampHC Process =====")  #🧩 Log the start of the data processing.
log_df = read_or_create_log(log_RampHC_path) #🧩 Read the existing Excel log file or create a new one if it doesn't exist.
log_entries, error_count = Default_variable() #📃 Initialize a list to store log results for the current run and an error counter.
#💡 Iterate through all files in the specified folder.
for filename in Folder_RampHC.glob("*.xlsx"): #🧩 Get files in the Folder directory.
    #📃 Get the current modification timestamp and filename of the current file.
    current_modified_date_microseconds = datetime.datetime.fromtimestamp(filename.stat().st_mtime) # Get modification timestamp, including microseconds.
    current_modified_date = current_modified_date_microseconds.replace(microsecond=0) # Remove microseconds for easier comparison.
    file_basename = filename.name # Get the base name of the file (e.g., "data_20230101.csv").
    #📃 File is being processed.
    print_colored(f"Processing file🔄️: {file_basename}", "NavajoWhite")
    logger.info(f"Processing file🔄️: {file_basename}")
    #📃 Check if this file has been logged before.
    log_entry = log_df.filter(pl.col("FileName") == file_basename) # Filter the old log_df to find an entry for this filename.
    is_new_file = log_entry.is_empty() # If no entry is found (log_entry is empty), it's a new file.
    previous_modified_date = datetime.datetime(1900, 1, 1, 0, 0, 1)  # Initialize the previous modification date with a very old date.
    #📃 If this file was processed before (exists in the log).  
    if not is_new_file:
        previous_modified_date = log_entry.select("ModifiedDate").item() # Get the modification date recorded in the log from the previous processing.
    #📃 Condition to process the file:
    # 1. The file is new (not in the log).
    # 2. Or, the file is old (in the log), but its current modification date (current_modified_date) is more recent than the modification date recorded in the log (previous_modified_date).
    if is_new_file or current_modified_date > previous_modified_date:
        if not is_new_file: # If it's an old file but has changed (ModifiedDate is newer).
            print_colored(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.", "Coral")
            logger.warning(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.")
        else: # If it's a completely new file.
            print_colored(f"New file detected🚦: {file_basename}. Importing.", "Coral")
            logger.info(f"New file detected🚦: {file_basename}. Importing.")
        try: # Start a try-except block to handle potential errors during the import process.
            if not is_new_file: # If this file already exists in the DB (because it's not new and its ModifiedDate changed) then delete the old data for this file from the df.
                delete_data(engine, RampHC_TABLE_NAME, file_basename)  #🧩 Call the function to delete data.
            #📃 Read data from the excel file using Polars.
            RampHC = (pl.read_excel(filename, infer_schema_length=None, engine="calamine", sheet_name="Sheet1") #🧩 Read the entire file to infer the schema.Add two new columns to the DataFrame: FileName and ModifiedDate.
                    .select(pl.all(),pl.lit(file_basename).alias("FileName"),pl.lit(current_modified_date).dt.cast_time_unit("ms").alias("ModifiedDate")
                   )) #🧩
            #📃 Validate if the columns in the DataFrame match the defined schema
            has_critical_error, critical_schema_error_msg = validate_schema(RampHC, RampHC_schema, file_basename) #🧩
            if has_critical_error: # If critical columns are missing.
                raise ValueError(critical_schema_error_msg) # Raise an error to stop processing this file.
            #📃 Adjust the column structure.
            print_colored(f"⚙️Adjust column structure", "Olive")
            logger.warning(f"⚙️Adjust column structure")
            RampHC = RampHC.with_columns(pl.col("Date").cast(pl.Date)) #🧩
            RampHC = RampHC.select(RampHC_schema) #🧩 Select columns in the correct order and names as per df_schema.
            print_colored(f"⚙️Column structure adjustment completed", "Olive")
            logger.info(f"⚙️Column structure adjustment completed")
            info_polars(RampHC) #🧩 Display information about the DataFrame (shape, columns, dtypes).
            #📃 Start writing data to the database.
            print_colored(f"💾Start import file: {file_basename}", "Coral")
            logger.info(f"💾Start import file: {file_basename}")
            write_data(engine, RampHC_TABLE_NAME, RampHC) #🧩 Call the function to write the DataFrame to SQL.
            print_colored(f"Successfully imported✨: {file_basename}", "LimeGreen")
            logger.info(f"Successfully imported✨: {file_basename}")
            #📃 Record to summary
            if is_new_file:
                summary_results["new_files_imported"].append(f"[RampHC] {file_basename}")#🧩
            else:
                summary_results["updated_files_reimported"].append(f"[RampHC] {file_basename}")#🧩       
            #📃 Update log_entries for successfully import: Add information to the log list for this run (successful).
            log_entries.append({"FileName": file_basename, "ModifiedDate": current_modified_date, "Error": None})
            print_colored(f"Updated 'Modified' time for {file_basename}", "LimeGreen")
            logger.info(f"Updated 'Modified' time for {file_basename}")
            #📃 Handle exceptions if any error occurred in the try block.
        except (
            FileNotFoundError, PermissionError, UnicodeDecodeError, IOError,                
            pl.exceptions.NoDataError, pl.exceptions.ComputeError, pl.exceptions.SchemaError, sa.exc.SQLAlchemyError,
            Exception # Catch various types of potential errors.
        ) as e:
            error_count += 1 # Increment the error counter.        
            error_msg_short = f"{type(e).__name__}: {str(e).splitlines()[0]}"  # Get a short error message (the first line).
            #📃 Update Failed file to Summary
            summary_results["failed_imports"].append((f"[RampHC] {file_basename}", error_msg_short))#🧩      
            logger.error(f"Error importing {file_basename}: {error_msg_short}", exc_info=True) # Log the full error traceback.
            print_colored(f"❗️Error importing {file_basename}: {error_msg_short}", "OrangeRed")
            # Add information to the log list for this run (failed).
            # Note: ModifiedDate here is previous_modified_date, as this file wasn't successfully imported with current_modified_date.
            log_entries.append({"FileName": file_basename,"ModifiedDate": previous_modified_date,"Error": error_msg_short})
    else: #📃 If the file is not new and its modification date hasn't changed.
          #📃 Update Skip file to Summary
        summary_results["skipped_files_unchanged"].append(f"[RampHC] {file_basename}")#🧩     
        print_colored(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.", "Aquamarine")
        logger.info(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.")
#💡 After iterating through all files in the directory.
display_summary("RampHC", error_count) #🧩 Display a summary of the results (number of errors).
process_and_save_log(log_df, log_entries, log_RampHC_path) #🧩 Update the Excel log file with information
logger.info("===== Processing of the RampHC data source is complete =====") #🧩 Log finish data processing.

In [ ]:
# 3️⃣6️⃣[BKN]SEAT💾
logger.info("===== Start SEAT Process =====")  #🧩 Log the start of the data processing.
log_df = read_or_create_log(log_SEAT_path) #🧩 Read the existing Excel log file or create a new one if it doesn't exist.
log_entries, error_count = Default_variable() #📃 Initialize a list to store log results for the current run and an error counter.
all_seat_files = list(Folder_SEAT.glob("*.xlsx")) + list(Folder_SEAT2.glob("*.xlsx")) #🧩 Combine folder
#💡 Iterate through all files in the specified folder.
for filename in all_seat_files: #📃 Get files in the Folder directory.
    #📃 Get the current modification timestamp and filename of the current file.
    current_modified_date_microseconds = datetime.datetime.fromtimestamp(filename.stat().st_mtime) # Get modification timestamp, including microseconds.
    current_modified_date = current_modified_date_microseconds.replace(microsecond=0) # Remove microseconds for easier comparison.
    file_basename = filename.name # Get the base name of the file (e.g., "data_20230101.csv").
    #📃 File is being processed.
    print_colored(f"Processing file🔄️: {file_basename}", "NavajoWhite")
    logger.info(f"Processing file🔄️: {file_basename}")
    #📃 Check if this file has been logged before.
    log_entry = log_df.filter(pl.col("FileName") == file_basename) # Filter the old log_df to find an entry for this filename.
    is_new_file = log_entry.is_empty() # If no entry is found (log_entry is empty), it's a new file.
    previous_modified_date = datetime.datetime(1900, 1, 1, 0, 0, 1)  # Initialize the previous modification date with a very old date.
    #📃 If this file was processed before (exists in the log).  
    if not is_new_file:
        previous_modified_date = log_entry.select("ModifiedDate").item() # Get the modification date recorded in the log from the previous processing.
    #📃 Condition to process the file:
    # 1. The file is new (not in the log).
    # 2. Or, the file is old (in the log), but its current modification date (current_modified_date) is more recent than the modification date recorded in the log (previous_modified_date).
    if is_new_file or current_modified_date > previous_modified_date:
        if not is_new_file: # If it's an old file but has changed (ModifiedDate is newer).
            print_colored(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.", "Coral")
            logger.warning(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.")
        else: # If it's a completely new file.
            print_colored(f"New file detected🚦: {file_basename}. Importing.", "Coral")
            logger.info(f"New file detected🚦: {file_basename}. Importing.")
        try: # Start a try-except block to handle potential errors during the import process.
            if not is_new_file: # If this file already exists in the DB (because it's not new and its ModifiedDate changed) then delete the old data for this file from the df.
                delete_data(engine, SEAT_TABLE_NAME, file_basename)  #🧩 Call the function to delete data.
            #📃 Read data from the excel file using Polars.
            SEAT = (pl.read_excel(filename, infer_schema_length=None, engine="calamine", sheet_name="SEAT MAP") #🧩 Read the entire file to infer the schema.Add two new columns to the DataFrame: FileName and ModifiedDate.
                    .select(pl.all(),pl.lit(file_basename).alias("FileName"),pl.lit(current_modified_date).dt.cast_time_unit("ms").alias("ModifiedDate")
                   )) #🧩
            #📃 Validate if the columns in the DataFrame match the defined schema
            has_critical_error, critical_schema_error_msg = validate_schema(SEAT, SEAT_schema, file_basename) #🧩
            if has_critical_error: # If critical columns are missing.
                raise ValueError(critical_schema_error_msg) # Raise an error to stop processing this file.
            #📃 Adjust the column structure.
            print_colored(f"⚙️Adjust column structure", "Olive")
            logger.warning(f"⚙️Adjust column structure")
            SEAT = SEAT.with_columns(pl.col("Date").cast(pl.Date)) #🧩
            SEAT = SEAT.select(SEAT_schema) #🧩 Select columns in the correct order and names as per df_schema.
            print_colored(f"⚙️Column structure adjustment completed", "Olive")
            logger.info(f"⚙️Column structure adjustment completed")
            info_polars(SEAT) #🧩 Display information about the DataFrame (shape, columns, dtypes).
            #📃 Start writing data to the database.
            print_colored(f"💾Start import file: {file_basename}", "Coral")
            logger.info(f"💾Start import file: {file_basename}")
            write_data(engine, SEAT_TABLE_NAME, SEAT) #🧩 Call the function to write the DataFrame to SQL.
            print_colored(f"Successfully imported✨: {file_basename}", "LimeGreen")
            logger.info(f"Successfully imported✨: {file_basename}")
            #📃 Record to summary
            if is_new_file:
                summary_results["new_files_imported"].append(f"[SEAT] {file_basename}")#🧩
            else:
                summary_results["updated_files_reimported"].append(f"[SEAT] {file_basename}")#🧩    
            #📃 Update log_entries for successfully import: Add information to the log list for this run (successful).
            log_entries.append({"FileName": file_basename, "ModifiedDate": current_modified_date, "Error": None})
            print_colored(f"Updated 'Modified' time for {file_basename}", "LimeGreen")
            logger.info(f"Updated 'Modified' time for {file_basename}")
            #📃 Handle exceptions if any error occurred in the try block.
        except (
            FileNotFoundError, PermissionError, UnicodeDecodeError, IOError,                
            pl.exceptions.NoDataError, pl.exceptions.ComputeError, pl.exceptions.SchemaError, sa.exc.SQLAlchemyError,
            Exception # Catch various types of potential errors.
        ) as e:
            error_count += 1 # Increment the error counter.        
            error_msg_short = f"{type(e).__name__}: {str(e).splitlines()[0]}"  # Get a short error message (the first line).
            #📃 Update Failed file to Summary
            summary_results["failed_imports"].append((f"[SEAT] {file_basename}", error_msg_short))#🧩   
            logger.error(f"Error importing {file_basename}: {error_msg_short}", exc_info=True) # Log the full error traceback.
            print_colored(f"❗️Error importing {file_basename}: {error_msg_short}", "OrangeRed")
            # Add information to the log list for this run (failed).
            # Note: ModifiedDate here is previous_modified_date, as this file wasn't successfully imported with current_modified_date.
            log_entries.append({"FileName": file_basename,"ModifiedDate": previous_modified_date,"Error": error_msg_short})
    else: #📃 If the file is not new and its modification date hasn't changed.
          #📃 Update Skip file to Summary
        summary_results["skipped_files_unchanged"].append(f"[SEAT] {file_basename}")#🧩
        print_colored(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.", "Aquamarine")
        logger.info(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.")
#💡 After iterating through all files in the directory.
display_summary("SEAT", error_count) #🧩 Display a summary of the results (number of errors).
process_and_save_log(log_df, log_entries, log_SEAT_path) #🧩 Update the Excel log file with information
logger.info("===== Processing of the SEAT data source is complete =====") #🧩 Log finish data processing.

In [ ]:
# 3️⃣7️⃣[BKN]SEAT_AVAIL💾
logger.info("===== Start SEAT_AVAIL Process =====")  #🧩 Log the start of the data processing.
log_df = read_or_create_log(log_SEAT_AVAIL_path) #🧩 Read the existing Excel log file or create a new one if it doesn't exist.
log_entries, error_count = Default_variable() #📃 Initialize a list to store log results for the current run and an error counter.
all_seat_files = list(Folder_SEAT_AVAIL.glob("*.xlsx")) + list(Folder_SEAT_AVAIL2.glob("*.xlsx")) #🧩 Combine folder
#💡 Iterate through all files in the specified folder.
for filename in all_seat_files: #📃 Get files in the Folder directory.
    #📃 Get the current modification timestamp and filename of the current file.
    current_modified_date_microseconds = datetime.datetime.fromtimestamp(filename.stat().st_mtime) # Get modification timestamp, including microseconds.
    current_modified_date = current_modified_date_microseconds.replace(microsecond=0) # Remove microseconds for easier comparison.
    file_basename = filename.name # Get the base name of the file (e.g., "data_20230101.csv").
    #📃 File is being processed.
    print_colored(f"Processing file🔄️: {file_basename}", "NavajoWhite")
    logger.info(f"Processing file🔄️: {file_basename}")
    #📃 Check if this file has been logged before.
    log_entry = log_df.filter(pl.col("FileName") == file_basename) # Filter the old log_df to find an entry for this filename.
    is_new_file = log_entry.is_empty() # If no entry is found (log_entry is empty), it's a new file.
    previous_modified_date = datetime.datetime(1900, 1, 1, 0, 0, 1)  # Initialize the previous modification date with a very old date.
    #📃 If this file was processed before (exists in the log).  
    if not is_new_file:
        previous_modified_date = log_entry.select("ModifiedDate").item() # Get the modification date recorded in the log from the previous processing.
    #📃 Condition to process the file:
    # 1. The file is new (not in the log).
    # 2. Or, the file is old (in the log), but its current modification date (current_modified_date) is more recent than the modification date recorded in the log (previous_modified_date).
    if is_new_file or current_modified_date > previous_modified_date:
        if not is_new_file: # If it's an old file but has changed (ModifiedDate is newer).
            print_colored(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.", "Coral")
            logger.warning(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.")
        else: # If it's a completely new file.
            print_colored(f"New file detected🚦: {file_basename}. Importing.", "Coral")
            logger.info(f"New file detected🚦: {file_basename}. Importing.")
        try: # Start a try-except block to handle potential errors during the import process.
            if not is_new_file: # If this file already exists in the DB (because it's not new and its ModifiedDate changed) then delete the old data for this file from the df.
                delete_data(engine, SEAT_AVAIL_TABLE_NAME, file_basename)  #🧩 Call the function to delete data.
            #📃 Read data from the excel file using Polars.
            SEAT_AVAIL = (pl.read_excel(filename, infer_schema_length=None, engine="calamine", sheet_name="SEAT AVAILABLE") #🧩 Read the entire file to infer the schema.Add two new columns to the DataFrame: FileName and ModifiedDate.
                    .select(pl.all(),pl.lit(file_basename).alias("FileName"),pl.lit(current_modified_date).dt.cast_time_unit("ms").alias("ModifiedDate")
                   )) #🧩
            #📃 Validate if the columns in the DataFrame match the defined schema
            has_critical_error, critical_schema_error_msg = validate_schema(SEAT_AVAIL, SEAT_AVAIL_schema, file_basename) #🧩
            if has_critical_error: # If critical columns are missing.
                raise ValueError(critical_schema_error_msg) # Raise an error to stop processing this file.
            #📃 Adjust the column structure.
            print_colored(f"⚙️Adjust column structure", "Olive")
            logger.warning(f"⚙️Adjust column structure")
            SEAT_AVAIL = SEAT_AVAIL.with_columns(pl.col("Date").cast(pl.Date)) #🧩
            SEAT_AVAIL = SEAT_AVAIL.select(SEAT_AVAIL_schema) #🧩 Select columns in the correct order and names as per df_schema.
            print_colored(f"⚙️Column structure adjustment completed", "Olive")
            logger.info(f"⚙️Column structure adjustment completed")
            info_polars(SEAT_AVAIL) #🧩 Display information about the DataFrame (shape, columns, dtypes).
            #📃 Start writing data to the database.
            print_colored(f"💾Start import file: {file_basename}", "Coral")
            logger.info(f"💾Start import file: {file_basename}")
            write_data(engine, SEAT_AVAIL_TABLE_NAME, SEAT_AVAIL) #🧩 Call the function to write the DataFrame to SQL.
            print_colored(f"Successfully imported✨: {file_basename}", "LimeGreen")
            logger.info(f"Successfully imported✨: {file_basename}")
            #📃 Record to summary
            if is_new_file:
                summary_results["new_files_imported"].append(f"[SEAT_AVAIL] {file_basename}")#🧩
            else:
                summary_results["updated_files_reimported"].append(f"[SEAT_AVAIL] {file_basename}")#🧩   
            #📃 Update log_entries for successfully import: Add information to the log list for this run (successful).
            log_entries.append({"FileName": file_basename, "ModifiedDate": current_modified_date, "Error": None})
            print_colored(f"Updated 'Modified' time for {file_basename}", "LimeGreen")
            logger.info(f"Updated 'Modified' time for {file_basename}")
            #📃 Handle exceptions if any error occurred in the try block.
        except (
            FileNotFoundError, PermissionError, UnicodeDecodeError, IOError,                
            pl.exceptions.NoDataError, pl.exceptions.ComputeError, pl.exceptions.SchemaError, sa.exc.SQLAlchemyError,
            Exception # Catch various types of potential errors.
        ) as e:
            error_count += 1 # Increment the error counter.        
            error_msg_short = f"{type(e).__name__}: {str(e).splitlines()[0]}"  # Get a short error message (the first line).
            #📃 Update Failed file to Summary
            summary_results["failed_imports"].append((f"[SEAT_AVAIL] {file_basename}", error_msg_short))#🧩     
            logger.error(f"Error importing {file_basename}: {error_msg_short}", exc_info=True) # Log the full error traceback.
            print_colored(f"❗️Error importing {file_basename}: {error_msg_short}", "OrangeRed")
            # Add information to the log list for this run (failed).
            # Note: ModifiedDate here is previous_modified_date, as this file wasn't successfully imported with current_modified_date.
            log_entries.append({"FileName": file_basename,"ModifiedDate": previous_modified_date,"Error": error_msg_short})
    else: #📃 If the file is not new and its modification date hasn't changed.
          #📃 Update Skip file to Summary
        summary_results["skipped_files_unchanged"].append(f"[SEAT_AVAIL] {file_basename}")#🧩
        print_colored(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.", "Aquamarine")
        logger.info(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.")
#💡 After iterating through all files in the directory.
display_summary("SEAT_AVAIL", error_count) #🧩 Display a summary of the results (number of errors).
process_and_save_log(log_df, log_entries, log_SEAT_AVAIL_path) #🧩 Update the Excel log file with information
logger.info("===== Processing of the SEAT_AVAIL data source is complete =====") #🧩 Log finish data processing.

In [ ]:
# 3️⃣8️⃣[BKN]CSAT_Comp💾
logger.info("===== Start CSAT_Comp Process =====")  #🧩 Log the start of the data processing.
log_df = read_or_create_log(log_CSAT_Comp_path) #🧩 Read the existing Excel log file or create a new one if it doesn't exist.
log_entries, error_count = Default_variable() #📃 Initialize a list to store log results for the current run and an error counter.
#💡 Iterate through all files in the specified folder.
for filename in Folder_CSAT_Comp.glob("*.xlsx"): #🧩 Get files in the Folder directory.
    #📃 Get the current modification timestamp and filename of the current file.
    current_modified_date_microseconds = datetime.datetime.fromtimestamp(filename.stat().st_mtime) # Get modification timestamp, including microseconds.
    current_modified_date = current_modified_date_microseconds.replace(microsecond=0) # Remove microseconds for easier comparison.
    file_basename = filename.name # Get the base name of the file (e.g., "data_20230101.csv").
    #📃 File is being processed.
    print_colored(f"Processing file🔄️: {file_basename}", "NavajoWhite")
    logger.info(f"Processing file🔄️: {file_basename}")
    #📃 Check if this file has been logged before.
    log_entry = log_df.filter(pl.col("FileName") == file_basename) # Filter the old log_df to find an entry for this filename.
    is_new_file = log_entry.is_empty() # If no entry is found (log_entry is empty), it's a new file.
    previous_modified_date = datetime.datetime(1900, 1, 1, 0, 0, 1)  # Initialize the previous modification date with a very old date.
    #📃 If this file was processed before (exists in the log).  
    if not is_new_file:
        previous_modified_date = log_entry.select("ModifiedDate").item() # Get the modification date recorded in the log from the previous processing.
    #📃 Condition to process the file:
    # 1. The file is new (not in the log).
    # 2. Or, the file is old (in the log), but its current modification date (current_modified_date) is more recent than the modification date recorded in the log (previous_modified_date).
    if is_new_file or current_modified_date > previous_modified_date:
        if not is_new_file: # If it's an old file but has changed (ModifiedDate is newer).
            print_colored(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.", "Coral")
            logger.warning(f"🧬ModifiedDate changed for {file_basename}. Deleting old data and re-importing.")
        else: # If it's a completely new file.
            print_colored(f"New file detected🚦: {file_basename}. Importing.", "Coral")
            logger.info(f"New file detected🚦: {file_basename}. Importing.")
        try: # Start a try-except block to handle potential errors during the import process.
            if not is_new_file: # If this file already exists in the DB (because it's not new and its ModifiedDate changed) then delete the old data for this file from the df.
                delete_data(engine, CSAT_Comp_TABLE_NAME, file_basename)  #🧩 Call the function to delete data.
            #📃 Read data from the excel file using Polars.
            CSAT_Comp = (pl.read_excel(filename, infer_schema_length=None, engine="calamine", sheet_name="Table1") #🧩 Read the entire file to infer the schema.Add two new columns to the DataFrame: FileName and ModifiedDate.
                    .select(pl.all(),pl.lit(file_basename).alias("FileName"),pl.lit(current_modified_date).dt.cast_time_unit("ms").alias("ModifiedDate")
                   )) #🧩
            #📃 Validate if the columns in the DataFrame match the defined schema
            has_critical_error, critical_schema_error_msg = validate_schema(CSAT_Comp, CSAT_Comp_schema, file_basename) #🧩
            if has_critical_error: # If critical columns are missing.
                raise ValueError(critical_schema_error_msg) # Raise an error to stop processing this file.
            #📃 Adjust the column structure.
            print_colored(f"⚙️Adjust column structure", "Olive")
            logger.warning(f"⚙️Adjust column structure")
            CSAT_Comp = CSAT_Comp.with_columns(pl.col("Date").cast(pl.Date)) #🧩
            CSAT_Comp = CSAT_Comp.select(CSAT_Comp_schema) #🧩 Select columns in the correct order and names as per df_schema.
            print_colored(f"⚙️Column structure adjustment completed", "Olive")
            logger.info(f"⚙️Column structure adjustment completed")
            info_polars(CSAT_Comp) #🧩 Display information about the DataFrame (shape, columns, dtypes).
            #📃 Start writing data to the database.
            print_colored(f"💾Start import file: {file_basename}", "Coral")
            logger.info(f"💾Start import file: {file_basename}")
            write_data(engine, CSAT_Comp_TABLE_NAME, CSAT_Comp) #🧩 Call the function to write the DataFrame to SQL.
            print_colored(f"Successfully imported✨: {file_basename}", "LimeGreen")
            logger.info(f"Successfully imported✨: {file_basename}")
            #📃 Record to summary
            if is_new_file:
                summary_results["new_files_imported"].append(f"[CSAT_Comp] {file_basename}")#🧩
            else:
                summary_results["updated_files_reimported"].append(f"[CSAT_Comp] {file_basename}")#🧩   
            #📃 Update log_entries for successfully import: Add information to the log list for this run (successful).
            log_entries.append({"FileName": file_basename, "ModifiedDate": current_modified_date, "Error": None})
            print_colored(f"Updated 'Modified' time for {file_basename}", "LimeGreen")
            logger.info(f"Updated 'Modified' time for {file_basename}")
            #📃 Handle exceptions if any error occurred in the try block.
        except (
            FileNotFoundError, PermissionError, UnicodeDecodeError, IOError,                
            pl.exceptions.NoDataError, pl.exceptions.ComputeError, pl.exceptions.SchemaError, sa.exc.SQLAlchemyError,
            Exception # Catch various types of potential errors.
        ) as e:
            error_count += 1 # Increment the error counter.        
            error_msg_short = f"{type(e).__name__}: {str(e).splitlines()[0]}"  # Get a short error message (the first line).
            #📃 Update Failed file to Summary
            summary_results["failed_imports"].append((f"[CSAT_Comp] {file_basename}", error_msg_short))#🧩       
            logger.error(f"Error importing {file_basename}: {error_msg_short}", exc_info=True) # Log the full error traceback.
            print_colored(f"❗️Error importing {file_basename}: {error_msg_short}", "OrangeRed")
            # Add information to the log list for this run (failed).
            # Note: ModifiedDate here is previous_modified_date, as this file wasn't successfully imported with current_modified_date.
            log_entries.append({"FileName": file_basename,"ModifiedDate": previous_modified_date,"Error": error_msg_short})
    else: #📃 If the file is not new and its modification date hasn't changed.
          #📃 Update Skip file to Summary
        summary_results["skipped_files_unchanged"].append(f"[CSAT_Comp] {file_basename}")#🧩        
        print_colored(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.", "Aquamarine")
        logger.info(f"ModifiedDate unchanged for {file_basename}. 🚀Skipping import.")
#💡 After iterating through all files in the directory.
display_summary("CSAT_Comp", error_count) #🧩 Display a summary of the results (number of errors).
process_and_save_log(log_df, log_entries, log_CSAT_Comp_path) #🧩 Update the Excel log file with information
logger.info("===== Processing of the CSAT_Comp data source is complete =====") #🧩 Log finish data processing.

In [ ]:
# Notify & Close DB📃
# Summary
summary_report = generate_summary_report(summary_results)
print(summary_report)
# Recort Summary
logger.info("===== FINAL SUMMARY REPORT =====")
logger.info(summary_report)
logger.info("================================")
try:
    with open(user_credential / r'DataBase//DataRaw//BKN//import_status_log.txt', 'w', encoding='utf-8') as file:
        file.write(summary_report)
except IOError as e:
    print(f"Error creating file: {e}")
engine.dispose()
print("Database connection closed.")
logger.info("Database connection closed.")
# Close logging
logger.info("Close logging system...")
logging.shutdown()
print("logging shutdown.")
%reset -f